In [ ]:
import pandas as pd
import sqlite3
import re
import unicodedata
from nltk.corpus import stopwords
import nltk

nltk.download("stopwords")

base_drive_path = "/content/drive/MyDrive/wifo dosyalar/"
db_path = base_drive_path + "reviews.db"

conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM Reviews", conn)
conn.close()

df = df.dropna(subset=["comment_text"])
df = df[df["comment_text"].str.strip() != ""]
df["comment_clean"] = df["comment_text"]

def remove_word_repeats(text):
    return re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

def fix_missing_space_after_dot(text):
    return re.sub(r'\.(?=[^\s\d])', '. ', text)

def remove_punctuation_noise(text):
    text = re.sub(r"\.{2,}", ".", text)
    text = re.sub(r"\s*[\.,!?;:]+\s*", " ", text)
    return text.strip()

def fix_merged_number_words(text):
    text = re.sub(r'(\d+)\s*tlye', r"\1 tl'ye", text)
    text = re.sub(r'(\d+)yi\b', r"\1'yi", text)
    return text

def normalize_unicode(text):
    return unicodedata.normalize("NFKC", text)

##bu hepsi siliniyor
empty_phrases = {
    "yorum yapmadım", "yorum yok", "daha kullanmadım",
    "henüz kullanmadım", "henüz denemedim", "kullanmadım",
    "denemedim", "daha denemedim", "yorum yazmadım", "kullanınca güncellerim",
    "daha denemedik", "deneyince yazarım"
}


custom_stopwords = {
   'kargo', 'paket', 'paketleme', 'satıcı', 'teşekkür', 'hediye',
    'ay', 'ya', 'yaa', 'yani', 'bence', 'aynen', 'fena değil', 'şey',
    'teşekkürler', 'teşekkür ederim', 'sorun yok', 'vs', 'vb', 'of',
    'replika', 'gönderdiler', 'güzelce', 'ayrı', 'gibi', 'ürün geldi', 'ürün elime ulaştı'

}

critical_words = {
    'ama', 'fakat', 'değil', 'ancak', 'yalnız', 'lakin', 'hiç', 'asla', 'gayet', 'ne', 'hiçbir', 'yok', 'olmayan',
    'eksik', 'zarar', 'yetersiz', 'aşırı', 'kesinlikle', 'tamamen', 'en', 'bile', 'sadece', 'hala', 'daha', 'yine', 'artık', 'tekrar'
}


nltk_stopwords = set(stopwords.words('turkish'))
final_stopwords = (nltk_stopwords | custom_stopwords | empty_phrases) - critical_words


neutral_keywords = [
    # Kargo & Teslimat
    "kargo", "teslimat", "zamanında geldi", "geç geldi", "hızlı teslimat",
    "mng", "yurtiçi", "arvato", "kargocu", "kargo şirketi", "hala gelmedi", "hala ulaşmadı",

    # Paketleme
    "paket", "paketleme", "ambalaj", "kutusu", "güzelce sarılmış", "özenli paketleme",
    "paketleme kötü", "paketleme başarılı", "süper paketleme", "güzel paketlenmiş",

    # Satıcı & Hediye
    "satıcı", "hediye", "satıcı ilgiliydi", "hediye göndermişler", "hediyelik", "hediye için teşekkürler",

    # Ürün yok / eksik / bozuk vs.
    "eksik ürün", "ürün eksik", "eksik geldi", "yanlış ürün", "yanlış geldi", "yanlış gönderildi",
    "ürün gelmedi", "bozuk geldi", "kırık geldi", "patlamış", "dökülmüş", "ürün dökülmüş", "ürün bozulmuş",
    "ürünün yarısı", "ürün elimde", "ürün elime ulaştı", "aldığım ürün", "ürün ulaşmadı",
    "farklı ürün", "hasarlı geldi", "kutusu açık", "ürün eksikti", "ürünün yarısı yoktu", "boş geldi",

    # İade süreci & fatura
    "iade", "iade süreci", "iade ettim", "iade ediyorum", "iptal", "ürün iptal", "fatura yok",
    "ürün orijinal değil", "sahte ürün", "ürün sahte", "replika ürün",

    # Süreç ile ilgili ama performans dışı ifadeler
    "sipariş süreci", "gönderildi", "ürün geldi", "ürün ulaştı", "siparişim geldi",
    "ürün elime geçti", "eksiksizdi", "tam geldi", "sorunsuz alışveriş", "kutu ezilmiş",
    "ürünüm yok", "hala teslim edilmedi"
]




pattern = '|'.join(neutral_keywords)

neutral_pattern = '|'.join(map(re.escape, neutral_keywords))  # Re.escape ile güvenli
df_neutral = df[df["comment_clean"].str.contains(neutral_pattern, case=False, na=False)]

df_perceptual = df[~df["comment_clean"].str.contains(neutral_pattern, case=False, na=False)]
df_perceptual = df_perceptual[~df_perceptual["comment_clean"].isin(empty_phrases)]


advanced_patterns = [
    r"yanlış.*(ürün|şey|şampuan|paket|gönderildi|geldi)",
    r"ürün.*gelmedi",
    r"eksik.*ürün",
    r"eksik.*geldi",
    r"hasarlı.*geldi",
    r"iade.*(ettim|ediyorum|süreci|ettik|edildi|aldılar)",
    r"(kargo|teslimat).*(geç|gelmedi|sorun|problem|bekledim)",
    r"farklı.*ürün",
    r"paket.*(yırtık|bozuk|kötü|açık)",
    r"ürün.*kırık",
    r"bozuk.*ürün",
    r"ürün.*iptal",
    r"ürün.*sahte",
    r"ürün.*replika",
    r"kutusu.*yok",
    r"fatura.*yok",
    r"ürün.*yoktu",
    r"ürün.*orijinal.*(değil|değildi)",
    r"marka.*(değil|sahte|uyduruk)",
    r"sadece.*bir.*ürün.*vardı",
    r"\d+\s*adet\s*aldım.*(sadece|ama|yalnız).*1.*geldi",
    r"(ürün|şampuan|sabun).*(patlamış|dökülmüş|boşalmış)",
    r"ürünün.*yarısı.*yok|dökülmüş",
    r"ürün.*boş.*geldi",
    r"ürün.*hala.*gelmedi",
    r"hala.*teslim.*edilmedi",
    r"aldığım.*ürün.*değil",
    r"ürün.*beklediğim.*gibi.*değil"
]


pattern = '|'.join(advanced_patterns)
df_advanced_exclude = df_perceptual[df_perceptual["comment_clean"].str.contains(pattern, case=False, na=False)]
df_final_perceptual = df_perceptual[~df_perceptual["comment_clean"].str.contains(pattern, case=False, na=False)]

typo_corrections1 = {
    "çokmu": "çok mu",
    "gerçektenmi": "gerçekten mi",
    "hiçmi": "hiç mi",
    "saglıyor": "sağlıyor",
    "örün": "ürün",
    "bumu": "bu mu",
    "böylemiydi": "böyle miydi",
    "bunlarmı": "bunlar mı",
    "yokartık": "yok artık",
    "yanimı": "yani mi",
    "değilmi": "değil mi",
    "engellermi": "engeller mi",
    "terkokutmuyor": "ter kokutmuyor",
    "oldspice": "old spice",
    "kokusuvar": "kokusu var",
   "dahada": "daha da",
   "güzelçbir":"güzel bir",
   "aldim":"aldım",
   "stokladim":"stokladım",
   "gercekten": "gerçekten",
   "boyle": "böyle",
   "sey": "şey",
   "ise": "işe",
   "mj": "mı",
   "öncedende": "önceden de",
   "bayilarak": "bayılarak",
   "kadin": "kadın",
   "kullaniyorum": "kullanıyorum",
   "başaralı": "başarılı",
   "sonrq":"sonra",
   "hisiyatı": "hissiyatı",
   "begenmesemde": "beğenmesem de",
   "hesiye": "hediye",
   "oglum":"oğlum",
   "orjınal": "orjinal",
   "once": "önce",
   "diger": "diğer",
   "bosune": "boşuna",
   "denemisim": "denemişim",
   "surekli": "sürekli",
   "kullaniyoruz": "kullanıyoruz",
   "kullanabılıyorsunuz": "kullanabiliyorsunuz",
   "güzelki": "güzel ki",
   "guzel": "güzel",
   "beyenmedimguzel": "beğenmedim güzel",
   "altiniz": "altınız",
   "cikmayor": "çıkmıyor",
   "hizli": "hızlı",
   "tafsiye": "tavsiye",
   "begendim": "beğendim",
   "Supereer": "süper",
   "kaldik": "kaldik",
   "hergün": "her gün",
   "geliyo": "geliyor",
   "alinir": "alınır",
   "kokuyo":"kokuyor",
   "inanilmaz": "inanılmaz",
   "almasin":"almasın",
   "lutfen":"lütfen",
   "saolsun":"sağolsun",
   "f/p": "fiyat  performans",
   "suresi": "süresi",
   "ozen": "özen",
   "ulaşti": "ulaştı",
   "sarmislardi": "sarmışlardı",
   "gozal": "gözal",
   "rolonla": "roll on'la",
   "kullandiği": "kullandığı",
   "almıstım": "almıştım",
   "ozellikle": "özellikle",
   "gatet ": "gayet",
   "herzaman":"her zaman",
   "urunler":"ürünler",
    "stıc": "stick",
    "faka": "fakat",
    "kiyafetlerimde": "kıyafetlerimde",
    "parasini": "parasını",
    "hicbir": "hiç bir",
    "yaramiyor": "yaramıyor",
    "komusu": "kokusu",
    "bense": "bence",
    "sürdügünüz": "sürdüğünüz",
   "begenmedim": "beğenmedim",
   "bnde": "ben de",
   "yazdigi": "yazdığı",
   "aldanmayin": "aldanmayın",
   "sanirim": "sanırım",
   "aliminyum": "alüminyum",
   "zman": "zaman",
   "tercıhım": "tercihim",
   "gb": "gibi",
   "kokusi": "kokusu",
   "girmiyo": "girmiyor",
   "şuan": "şu an",
   "inşlh": "inşallah",
   "stalk": "stok",
   "akıyo": "akıyor",
   "rolonun": "roll on'un",
   "fiyatina": "fiyatına",
   "fln": "falan",
   "tasarimi": "tasarımı",
   "uzerınde": "üzerinde",
   "sure": "süre",
   "kaliyor": "kalıyor",
   "esim":"eşim",
   "kullanıyoruz": "kullanıyoruz",
   "arkadasim": "arkadaşım",
   "begendi": "beğendi",
   "tatlisiniz": "tatlısınız",
   "tesekkurler": "teşekkürler",
   "ins": "inşallah",
   "fiyatida": "fiyatıda",
   "kullanisli": "kullanışlı",
   "pakartleme": "paketleme",
   "de vamlı": "devamlı",
   "gzüel": "güzel",
   "kacirmadigim": "kaçırmadığım",
   "hakkinda": "hakkında",
   "uybun": "uygun",
   "begendigi": "beğendiği",
   "herseyi": "her şeyi",
   "ğrün": "ürün",
   "yaomadı": "yapmadı",
   "deosu": "deodorantı",
   "bisey": "bir şey",
   "tsk": "teşekkür",
   "tşk": "teşekkür",
   "alin": "alın",
   "cok": "çok",
   "grcl": "güzel",
   "ürn": "ürün",
   "urun": "ürün",
   "siparis": "sipariş",
   "coh": "çok",
   "koti": "kötü",
   "deger" : "değer",
   "deo": "deodorant",
   "Hiçbir": "hiç bir",
   "ayni":"aynı",
   "markayi": "markayı",
   "begenmistim" : "beğenmiştim",
   "simdi": "şimdi",
   "rolon":  "roll on",
    "roll-on": "roll on",
   "bi" : "bir",
   "paketlenmisti": "paketlenmişti",
   "aynu": "aynı",
   "gelmiyo": "gelmiyor",
   "kokmadi": "kokmadı",
   "kaliciliğı": "kalıcılığı",
   "aynısınım": "aynısının",
   "atickini": "stickini",
   "buda": "bu da",
    "herşey":  "her şey",
   "stokladik": "stokladık",
   "urun.super":"ürün süper",
   "super": "süper",
   "kesfettigim":"keşfettiğim",
   "kokosu": "kokusu",
   "cokguzel": "çok güzel",
   "esime": "eşime",
   "akrgo": "kargo",
   "kullanılmamiş": "kullanmamış",
   "tabisye": "tavsiye",
   "yollamadiniz": "yollamadınız",
   "kalıtelı": "kaliteli",
   "degil": "değil",
   "nymara": "numara",
   "baskin": "baskın",
   "oyn": "on",
    "hızli": "hızlı",
   "hoi": "hoş",
   "hojj":"hoş",
   "kullaniyor": "kullanıyor",
   "sakiz":"sakız",
   "müq": "mükemmel",
   "muq":"mükemmel",
   "koki": "kokuyor",
   "muhtesem":"muhteşem",
   "uc": "üç",
   "gun": "gün",
   "yikanma": "yıkanma",
   "saticiyla": "satıcıyla",
   "dicem": "diyeceğim",
   "olmadıgı": "olmadığı",
   "oldugu": "olduğu",
   "alişim": "alışım",
   "kullandıgım": "kullandığım",
   "vqr": "var",
   "kalici": "kalıcı",
   "abartıldğı": "abartıldığı",
   "malesef":"maalesef",
   "ucin": "için",
   "veriyo": "veriyor",
   "adat": "adet",
   "bilr": "bile",
   "Sağladi":"sağladı",
   "agir": "ağır",
   "yok.begendim.tesekkurler": "yok beğendim teşekkürler",
   "degil.guzel.paketlemede": "değil güzel paketlemede",
   "ucmadan": "uçmadan",
   "gzuel": "güzel",
   "bı": "bir",
   "10numara": "10 numara",
   "5yıldız": "5 yıldız",
   "begenerekten": "beğenerekten",
   "TR'ye": "Türkiye'ye",
   "süreklı": "sürekli",
   "krgo": "kargo",
   "alinabilir": "alınabilir",
   "kokuısu": "kokusu",
   "mukkemel":"mükemmel",
   "çık": "çok",
   "isede": "işe de",
   "şampuanini": "şampuanını",
   "urin": "ürün",
   "begenildi":"beğenildi",
   "ediyırum":"ediyorum",
   "tesekkürler": "teşekkürler",
   "sodlaki":"soldaki",
   "yanlis": "yanlış",
   "gonderiorsunuz": "gönderiyorsunuz",
   "kullanıyım": "kullanayım",
   "birşey": "bir şey",
   "nıvea": "nivea",
   "bubtarz": "bu tarz",
   "diyer": "diğer",
   "bergolaeda": "bargello'da",
   "guvenli": "güvenli",
   "yuzden": "yüzden",
   "sarilmadan": "sarılmadan",
   "yirtildiğini": "yırtıldığını",
   "dusunuyorum": "düşünüyorum",
   "kisminda":"kısmında",
   "yirtik": "yırtık",
   "vardi":"vardı",
   "tuketim": "tüketim",
   "ici": "içi",
   "ellenmemisti": "ellenmemişti",
   "gerektigini": "gerektiğini",
   "dayaniyor": "dayanıyor",
   "bidaha": "bir daha",
   "basarili": "başarılı",
   "kalıteli": "kaliteli",
   "kullandiklarim": "kullandıklarım",
   "icinde": "içinde",
   "hemde": "hem de",
   "rahatlik": "rahatlık",
   "olrk": "olarak",
   "tşkler": "teşekkürler",
   "beyendik": "beğendik",
   "severel": "severek",
    "çocuklarim": "çocuklarım",
    "sahane": "şahane",
    "ce": "ve",
    "trk": "tek",
    "kadinlara": "kadınlara",
    "kendı": "kendi",
    "aska": "asla",
    "dhaa": "daha",
    "urunmus": "ürünmüş",
    "yapmalik": "yapmalık",
    "kardesim":"kardeşim",
    "aliminyumsuz": "alüminyumsuz",
    "tercij": "tercih",
    "freş": "freş",
    "kullanmiyoruz": "kullanmıyoruz",
    "muthis": "müthiş",
    "ıyı": "iyi",
    "cokmi": "çok mu",
    "wallla": "valla",
    "bişey": "bir şey",
    "bnce": "bence",
    "duzey": "düzey",
    "fazlq": "fazla",
    "yollanmıs": "yollanmış",
    "2gunde": "2 günde",
    "oldukca": "oldukça",
    "gunde": "gunde",
    "bıze": "bize",
    "guzellıgınden": "güzelliğinden",
    "benım": "benim",
    "filn": "falan",
    "kötüki": "kötü ki",
    "öle": "öyle",
    "bide": "bir de",
    "sulandiriyo": "sulandırıyor",
    "akiyo": "akıyor",
    "kmtü": "kötü",
    "kalıcılıgı": "kalıcılığı",
    "hıc": "hiç",
    "bosuna": "boşuna",
    "suruyorum": "sürüyorum",
    "oglene": "öğlene",
    "okdr": "o kadar",
    "surmeme": "sürmeme",
    "ragmen": "rağmen",
    "yuotuber": "youtuber",
    "gırgınım": "kırgınım",
    "gonderimis": "gönderilmiş",
    "kulanila bilir": "kullanılabilir",
    "esnitisini": "esintisini",
    "fhres": "fresh",
    "abartıliyor": "abartılıyor",
    "izyapıor": "iz yapıyor",
    "sari": "sarı",
    "bayildim": "bayıldım",
    "kadinlar": "kadınlar",
    "kullanabilir": "kullanabilir",
    "katkisix": "katkısız",
    "com": "çok",
    "amanda":"zamanda",
    "Suan a kadar ki": "şu ana kadar ki",
    "esimin": "eşimin",
    "sevdigi": "sevdiği",
    "isteyebiirsiniz": "isteyebilirsiniz",
    "alamiyorum": "alamıyorum",
    "praben": "paraben",
    "sikkat": "dikkat",
    "begenerek": "beğenerek",
    "Deodorantmı": "deodorant mı",
    "maddemi": "madde mi",
    "devamli": "devamlı",
    "alcagımı": "alacağımı",
    "lokusu": "kokusu",
    "tavsıye": "tavsiye",
    "ven":"ben",
    "direk": "direkt",
    "alsaumışım": "alsaymışım",
    "günun": "günün",
    "kurtaricisi": "kurtarıcısı",
    "sagolun": "sağolun",
    "mük": "mükemmel",
    "mumkun": "mümkün",
    "lazim": "lazım",
    "ab.": "abi",
    "rollon": "roll on",
    "paektleme": "paketleme",
    "gunlerce": "günlerce",
    "suruyor": "sürüyor",
    "oneririm": "öneririm",
    "oģlum": "oğlum",
    "saglam": "sağlam",
    "okadar": "o kadar",
    "guzeldi": "güzeldi",
    "rolonu": "roll on'u",
    "demleyip": "deneyip",
    "wolf thorn": "wolfthorn",
    "onlemede": "önlemede",
    "faydali": "faydalı",
    "gercektenn": "gerçekten",
    "güzell": "güzel",
    "çom": "çok",
    "hayatimda": "hayatımda",
    "fp": "fiyat performans",
    "bie":"bir",
    "okuanus": "okyanus",
    "markadayı": "markayı",
    "guzle": "güzel",
    "teşekkur": "teşekkür",
    "kaliciguzel": "kalıcı güzel",
    "deodorantlar a": "deodrantlara",
    "aldırt": "aldırın",
    "iyisüper": "iyi süper",
    "olmamasida": "olmaması da",
    "meyvelisabunsu": "meyveli sabunsu",
    "kullanimi": "kullanımı",
    "dtok": "stok",
    "yenı": "yeni",
    "keşfettım": "keşfettim",
    "arkadasım": "arkadaşım",
    "gıymeyı": "giyemyi",
    "istemıyorsanız": "istemiyorsanız",
    "kalıcıgılıda": "kalıcılığı da",
    "birakiyor": "bırakıyor",
    "terletmiyo": "terletmiyor",
    "dünyaın": "dünyanın",
    "kullandigim": "kullandığım",
    "aşşırı": "aşırı",
    "nvea": "nivea",
    "harıka": "harika",
    "yaglanmayi": "yağlanmayı",
    "azalltti": "azalttı",
    "fiyati ": "fiyatı",
    "perfecto": "mükemmel",
    "aödım": "aldım",
    "deostick": "deodorant stick",
    "kokisi": "kokusu",
    "alacagim": "alacağım",
    "baska": "başka",
    "urunun": "ürünün",
    "urunu": "ürünü",
    "kullanirim": "kullanırım",
    "satin": "satın",
    "aktifsiniz": "aktfiseniz",
    "baildım": "bayıldım",
    "agır": "ağır",
    "etkisin i":"etkisini",
    "yasamadim": "yaşamadım",
    "islenmis": "işlenmiş",
    "gormuyor": "görmüyor",
    "oyle": "öyle",
    "ayrica": "ayrıca",
    "kalip": "kalıp",
    "parfumlusunu": "parfümlüsünü",
    "dusunun": "düşünün",
    "hemencicik": "hemencecik",
    "ucuyor": "uçuyor",
    "puskurtmelilerden": "püskürtmelilerden",
    "abartıldıgı": "abartıldığı",
    "biryerden": "bir yerden",
    "rolonlarda": "roll on'larda",
    "erkeksii": "erkeksi",
    "hesapli": "hesaplı",
    "herkes e": "herkese",
    "esentisi": "esintisi",
    "zamanalrda": "zamanlarda",
    "sağladi": "sağladı",
    "tesekkur":"teşekkür",
    "yapardim": "yapardım",
    "basarılı": "başarılı",
    "kullanabilirsiniz": "kullanabilirsiniz",
    "tahrihi": "tarihini",
    "yaziyormu": "yazıyor mu",
    "şyi": "şey",
    "rockstae": "rockstar",
    "raziyim":"razıyım",
    "teşekur": "teşekkür",
    "guzelmis": "güzelmiş",
    "kokuso": "kokusu",
    "çook": "çok",
    "stoklanmalik": "stoklanmalık",
    "sıparıs": "sipariş",
    "werdıgım": "verdiğim",
    "gereksız": "gereksiz",
    "2adet": "2 adet",
    "idea": "idi",
    "kullanamz": "kullanamaz",
    "cokl": "çok",
    "arkadasimda": "arkadaşımda",
    "gormustum": "görmüştüm",
    "erkegimsi": "erkeğimsi",
    "hic":"hiç",
    "tereddut": "tereddüt",
    "yillardir": "yıllardır",
    "aliyorum": "alıyorum",
    "udun": "ürün",
    "kullaniliyor": "kullanılıyor",
    "ogullarima":"oğullarıma",
    "yetistiremedigim": "yetiştiremediğim",
    "imdadima": "imdadına",
    "yetisti": "yetişti",
    "killanilcsk": "kullanılacak",
    "eahat": "rahat",
    "disinda": "dışında",
    "hersey": "her şey",
    "alumina": "alüminyum",
    "yapiya": "yapıya",
    "bazli": "bazlı",
    "olmasi": "olması",
    "uzmus": "üzmüş",
    "kullanmaktayim": "kullanmaktayım",
    "beyazlik": "beyazlık",
    "birakmiyor": "bırakmıyor",
    "bircok": "bir çok",
    "uyğuna": "uyguna",
    "goatu": "en iyisi",
    "goat": "en iyi",
    "uugun": "uygun",
    "aldirin": "aldırın",
    "tsm": "tam",
    "olayi": "olayı",
    "abartmiyorum": "abartmıyorum",
    "gecerli": "geçerli",
    "amacina": "amacına",
    "gondericide": "gönderici de",
    "gonderdi": "gönderdi",
    "spise": "spice",
    "eşkkürler": "teşekkürler",
    "tesukurler": "teşekkürler",
    "güzäl": "güzel",
    "karşımı": "karışımı",
    "saticinin": "satıcının",
    "orjinall": "orijinal",
    "harbii": "harbi",
    "not bad": "kötü değil",
    "narika": "harika",
    "yapmyıor": "yapmıyor",
    "aradıgımı": "aradığımı",
    "yakalamislar": "yakalamışlar",
    "quality": "kalite",
    "kazinmadan": "kazınmadan",
    "kalicilik": "kalıcılık",
    "gorevini": "görevini",
    "mukemmel": "mükemmel",
    "kardesime": "kardeşime",
    "kullanmadim": "kullanmadım",
    "iahsen": "şahsen",
    "yanıtlmaz": "yanıltmaz",
    "paketlemefe": "paketlemede",
    "akşamgece": "akşam gece",
    "kulanimi": "kullanımı",
    "kokuyoor": "kokuyor",
    "sekmenti": "segmenti",
    "isin": "işin",
    "bayiliyoruz": "bayılıyoruz",
    "ekzotik": "egzotik",
    "keke": "keşke",
    "mok": "çok",
    "yildiz": "yıldız",
    "buýana": "bu yana",
    "ýaşdan": "yaştan",
    "tafsiyye": "tavsiye",
    "fıreş": "fresh",
    "olacagını":"olacağını",
    "acısından": "acısından",
    "acıldıktan": "açıldıktan",
    "altındaki":"altındaki",
    "dondurme":"döndürme",
    "ha ri ka": "harika",
    "ındırıme": "indirime",
    "gırdıkce": "girdikçe",
    "perfect": "mükemmel",
    "yaptiğim": "yaptığım",
    "kararmalarina": "kararmalarına",
    "dus":"duş",
    "sonrasi": "sonrası",
    "kaldi": "kaldı",
    "terettud": "tereddüt",
    "gerctkten":"gerçekten",
    "perfektıkıl": "mükemmel",
    "adla": "asla",
    "tabsiye": "tavsiye",
     "waoov": "iyi",
    "deodorantim": "deodorantım",
    "mukemmellik": "mükemmelik",
    "yapsanizda": "yapsanız da",
    "expisers": "express",
    "2yildir": "2 yıldır",
    "yildir": "yıldır",
    "yedekliyeceğim": "yedekleyeceğim",
    "sirekli": "sürekli",
    "bittikce": "bittikçe",
    "aylardir": "aylardır",
    "almistim": "almıştım",
    "yapicm": "yapacağım",
    "birsey": "bir şey",
    "vaad": "vaat",
    "ınstagram'da": "instagram'da",
    "ınstagram": "instagram",
    "kizima": "kızıma",
    "dogum": "doğum",
    "yaptiktan": "yaptıktan",
    "olusan": "oluşan",
    "kuvik": "küçük",
    "olusuyurdu": "oluşuyordu",
    "bikactane": "birkaç tane",
    "kurtarici": "kurtarıcı",
    "stoklanmali": "stoklanmalı",
    "sahene": "şahane",
    "calisicam": "çalışacağım",
    "gitmiuor": "gitmiyor",
    "kelimeylee": "kelimeyle",
    "mükkemeell": "mükemmel",
    "stoklayacagım": "stoklayacağım",
    "gördükce": "gördükçe",
    "stokladigim": "stokladığım",
    "urunn": "ürün",
    "sifir": "sıfır",
    "rulon": "roll-on",
    "herşeyi": "her şeyi",
    "hiçbişeymiş": "hiçbir şeymiş",
    "yapiyorum": "yapıyorum",
    "zamandir": "zamandır",
    "girdikce": "girdikçe",
    "yapin": "yapın",
    "harikaa": "harika",
    "yapilmalı": "yapılmalı",
    "misller": "misler",
    "geliyormus": "geliyormuş",
    "birşeymis": "bir şeymiş",
    "stoklsaymisim": "stoklasaymışım",
    "kotu": "kötü",
    "gecmis": "geçmiş",
    "oncede kullanmistim": "önceden kullanmıştım",
    "degildi": "değildi",
    "cöp": "çöp",
    "deoderantini": "deodorantını",
    "hemn": "hemen",
    "deil": "değil",
    "severk": "severek",
    "kullanöadoö": "kullanamadım",
    "alip": "alıp",
    "kullandim": "kullandım",
    "cevremede": "çevreme de",
    "koltukalti": "koltuk altı",
    "rulonu": "roll on’u",
    "yapilsin": "yapılsın",
    "kullandiginiz": "kullandığınız",
    "tum": "tüm",
    "deolari": "deodorantları",
    "bagimlilik": "bağımlılık",
    "yapiyo": "yapıyor",
    "stoklanmasi": "stoklanması",
    "aylarinin": "aylarının",
    "begeniyorum": "beğeniyorum",
    "diyincede": "deyince de",
    "duzeltme": "düzeltme",
    "2nci": "ikinci",
    "biseysin": "bir şeysin",
    "yakaladikca": "yakaladıkça",
    "ozel": "özel",
    "gunlerde": "günlerde",
    "fav": "favorim",
    "deom": "deodorantım",
    "stokladgım": "stokladığım",
    "bsey": "bir şey",
    "fyat": "fiyat",
    "gördgm": "gördüğüm",
    "stokluyrum": "stokluyorum",
    "dsarda": "dışarıda",
    "gezn": "gezin",
    "ınsanlara": "insanlara",
    "dagıtacam": "dağıtacağım",
    "medenıyet": "medeniyet",
    "görsnler": "görsünler",
    "guzell": "güzel",
    "bitrikce": "bittikçe",
    "beğen dim": "beğendim",
    "olmıcak": "olmayacak",
    "yapıorum": "yapıyorum",
    "kullaniyordum": "kullanıyordum",
    "şuana": "şu ana",
    "vazgecemiyeceksiniz": "vazgeçemeyeceksiniz",
    "aldiysam": "aldıysam",
    "bouu": "bu",
    "sıgır": "sıfır",
    "önlüyo": "önlüyor",
    "rexsona": "rexona",
    "tanidigomdan": "tanıdığımdan",
    "miss": "mis",
    "ulasti": "ulaştı",
    "hasarsiz": "hasarsız",
    "2ci": "ikinci",
    "koçuk": "küçük",
    "kucucuk": "küçücük",
    "yapışş": "yapış",
    "fres": "ferah",
    "kucuk": "küçük",
    "canta": "çanta",
    "icin": "için",
    "mlmi": "milimetresi",
    "gozume": "gözüme",
    "kullanmamistim": "kullanmamıştım",
    "gorünuyor": "görünüyor",
    "kullanilir": "kullanılır",
    "tm": "tam",
    "deneyıp": "deneyip",
    "kalıcılıgi": "kalıcılığı",
    "güzekk": "güzel",
    "super": "süper",
    "sekilde": "şekilde",
    "yayinindan": "yayınından",
    "bır": "bir",
    "şne": "ine",
    "rollom": "roll on'um",
    "cokk": "çok",
    "kolusu": "kokusu",
    "rolanlarım": "roll on’larım",
    "hatika": "harika",
    "otesi": "ötesi",
    "koltukaltı": "koltuk altı",
    "ııcn": "için",
    "öantalık": "çantalık",
    "begendimw": "beğendim",
    "harıka": "harika",
    "cik": "çık",
    "yillarin": "yılların",
    "uyi": "iyi",
    "minnak": "minik",
    "buyuk": "büyük",
    "minnaklar": "minikler",
    "sarrmıs": "sararmış",
    "altibir": "altı bir",
    "gonderiyorusunuz": "gönderiyorsunuz",
    "1yil": "1 yıl",
    "kalitəli": "kaliteli",
    "ürundur": "üründür",
    "yazilmisdi": "yazılmıştı",
    "gonderilmis": "gönderilmiş",
    "onceki": "önceki",
    "olusmuyor": "oluşmuyor",
    "onlemiyor": "önlemiyor",
    "kanallarini": "kanallarını",
    "tıkanmiyor": "tıkanmıyor",
    "yazin": "yazın",
    "dogru": "doğru",
    "onluyor": "önlüyor",
    "deniyecegim": "deneyeceğim",
    "kullaniyorm": "kullanıyorum",
    "yapisi": "yapısı",
    "stoklamalik": "stoklamalık",
    "fiyatteyken": "fiyattayken",
    "davamli": "devamlı",
    "zel": "güzel",
    "kardo": "kargo",
    "kullandıgim": "kullandığım",
    "vazgecilmez": "vazgeçilmez",
    "güzelürünü": "güzel ürünü",
    "alisim": "alışım",
    "kullanıyprum": "kullanıyorum",
    "aldimm": "aldım",
    "aşdim": "aldım",
    "rollonları": "roll-on’ları",
    "kıruyuculuğu": "kuruyuculuğu",
    "aman": "zaman",
    "don": "son",
    "reyona": "rexona",
    "bildigimiz": "bildiğimiz",
    "kaşar": "kadar",
    "olmazssa": "olmazsa",
    "olmazim": "olmazım",
    "duşında": "dışında",
    "tskler": "teşekkürler",
    "rexona nin": "rexona’nın",
    "ustun": "üstün",
    "rwxona": "rexona",
    "alinmali": "alınmalı",
    "azındanbir": "en azından bir",
    "paketlen": "paketleme",
    "ferahh": "ferah",
    "müg": "mükemmel",
    "kullundığım": "kullandığım",
    "ındırime": "indirime",
    "gırdıgı": "girdiği",
    "znm": "zaman",
    "yaptim": "yaptım",
    "kacirmayin": "kaçırmayın",
    "vazgecemedıgım": "vazgeçemediğim",
    "ferahlik": "ferahlık",
    "tertemizz": "tertemiz",
    "gidiyri": "gidiyor",
    "duzenli": "düzenli",
    "bilginizee": "bilginize",
    "kalmistim": "kalmıştım",
    "kokulari": "kokuları",
    "seviyorsaniz": "seviyorsanız",
    "indirmde": "indirimde",
    "hizliydi": "hızlıydı",
    "olmaz larım": "olmazlarım",
    "pudrali": "pudralı",
    "lekele se": "lekelese",
    "bayiliyorum": "bayılıyorum",
    "endirimle": "indirimle",
    "oakatleme": "paketleme",
    "miis": "mis",
    "canli": "canlı",
    "yayndan": "yayından",
    "paketlenmis": "paketlenmiş",
    "zaiyat": "hasarsız",
    "fiyatlari": "fiyatları",
    "karika": "harika",
    "şakamı": "şaka mı",
    "surmemissiniz": "sürmemişsiniz",
    "kac": "kaç",
    "kayboluyo": "kayboluyor",
    "ünlühakkıyla": "ünlü hakkıyla",
    "sekerli": "şekerli",
    "deneyecegım": "deneyeceğim",
    "guncelleyecegım": "güncelleyeceğim",
    "kalıcığıda": "kalıcılığı da",
    "edrr": "eder",
    "diverleri": "diğerleri",
    "gbi": "gibi",
    "yapmiyor": "yapmıyor",
    "koltulatında": "koltuk altında",
    "kokosuda": "kokusu da",
    "islak": "ıslak",
    "kargolandi": "kargolandı",
    "yogun": "yoğun",
    "ettiginiz": "ettiğiniz",
    "islakligi onlemiyor": "ıslaklığı önlemiyor",
    "frna": "fırına",
    "sayilmaz": "sayılmaz",
    "korunakliydi": "korunaklıydı",
    "buloko": "bloke",
    "elıme": "elime",
    "beyendim": "beğendim",
    "özenliydibence": "özenliydi bence",
    "kullanıyoru": "kullanıyorum",
    "kalıcılığıdq": "kalıcılığı da",
    "h3men": "hemen",
    "çokk": "çok",
    "paketlenmişdi": "paketlenmişti",
    "ba yıl dım": "bayıldım",
    "su an": "şu an",
    "parfğmğnü": "parfümünü",
    "beyazlatici": "beyazlatıcı",
    "kullandiktan": "kullandıktan",
    "guncellerim": "güncellerim",
    "denecağim": "deneyeceğim",
    "ğüzel": "güzel",
    "yapbiyor": "yapmıyor",
    "1adet": "1 adet",
    "yapmio": "yapmıyor",
    "gondermissiniz": "göndermişsiniz",
    "güzel123": "güzel",
    "duymamissiniz": "duymamışsınız",
    "aldik": "aldık",
    "ilkkez": "ilk kez",
    "ef sa ne": "efsane",
    "surup": "sürüp",
    "rezonans": "rexona",
    "rexona,": "rexona",
    "ciceksi": "çiçeksi",
    "terdiye": "ter diye",
    "aşşırıı": "aşırı",
    "ederim lt 3": "ederim",
    "indrimdeyken": "indirimdeyken",
    "ayri": "ayrı",
    "alin": "alın",
    "deodoranti": "deodorantı",
    "vazgecemiyorum": "vazgeçemiyorum",
    "saglamdi": "sağlamdı",
    "efasanee": "efsane",
    "geldiki": "geldi ki",
    "biran": "bir an",
    "isinize": "işinize",
    "hedıye": "hediye",
    "canıms": " ",
    "koyabilecegim": "koyabileceğim",
    "tşekur": "teşekkür",
    "deodarant": "deodorant",
    "inflz": "influencer",
    "yanni": "yani",
    "ttendyol": "trendyol",
   "dav": "dove",
   "kasinti": "kaşıntı",
   "2tane": "2 tane",
   "kokisu": "kokusu",
   "ztn": "zaten",
   "cook": "çok",
   "tsklr": "teşekkürler",
   "bicmsız": "biçimsiz",
   "indirmeden": "indirimden",
   "stokluyacağım": "stoklayacağım",
   "stoklicam": "stoklayacağım",
   "hos": "hoş",
   "roll ön": "roll on",
   "suan": "şu an",
   "rollonlar": "roll on’lar",
   "coconutli": "hindistancevizli",
   "pnluyor": "önlüyor",
   "konuyo": "kokuyot",
   "iyipaketleme": "iyi paketleme",
   "alinn": "alın",
   "bna": "bana",
   "fiyattıstokladım": "fiyattayken stokladım",
   "tiz kokmayan": "temiz kokan",
   "stoklanavak": "stoklanacak",
   "alacagım": "alacağım",
   "hedıyeler": "hediyeler",
   "ıcın": "için",
   "iyiki": "iyi ki",
   "kokuaunu": "kokusunu",
   "rollona": "roll-on’a",
   "urunlerden": "ürünlerden",
   "kullandiğım": "kullandığım",
   "uhteşem": "muhteşem",
   "rolonum": "roll-on’um",
    "bukadarmi": "bu kadar mı",
    "harikaa": "harika",
    "rolonlarıda": "roll-on’ları da",
    "birdan": "birden",
    "bayaa": "bayağı",
    "olurmu": "olur mu",
    "bileginize": "bileğinize",
    "ozaman": "o zaman",
    "olmadıgını": "olmadığını",
    "artti": "arttı",
    "kurumus": "kurumuş",
    "miadi": "tarihi",
    "kaliciligina": "kalıcılığına",
    "deyil": "değil",
    "saglik": "sağlık",
    "urundu": "üründü",
    "gıyebılıyorum": "giyebiliyorum",
    "orıginal": "orijinal",
    "herkewe": "herkese",
    "kullanniyorum": "kullanıyorum",
    "ihtiyaci": "ihtiyacı",
    "kesınlıkle": "kesinlikle",
    "ederım": "ederim",
    "yerını": "yerini",
    "ınanın": "inanın",
    "kesın": "kesin",
    "yeterlı": "yeterli",
    "yiilardir": "yıllardır",
    "kokuyot": "kokuyor",
    "tşklerr": "teşekkürler",
    "asiri": "aşırı",
    "teşekurlar": "teşekkürler",
    "kesfettikten": "keşfettikten",
    "surme": "sürme",
    "birde": "bir de",
    "yakin": "yakın",
    "arkadaslarim": "arkadaşlarım",
    "surdun": "sürdün",
    "olacakki": "olacak ki",
    "bulamadim": "bulamadım",
    "aldigim": "aldığım",
    "stixs": "stick",
    "gecen": "geçen",
    "keske": "keşke",
    "olacagini": "olacağını",
    "mütiş": "müthiş",
    "guvenilir": "güvenilir",
    "satici": "satıcı",
    "ekspres": "ekspres",
    "kullandiigim": "kullandığım",
    "korunakli": "korunaklı",
   "düsünmemden": "düşünmemden",
    "gerci": "gerçi",
    "şşekilde": "şekilde",
    "hizlı": "hızlı",
    "begendım": "beğendim",
    "sur": "sür",
    "ozenliydi": "özenliydi",
    "saticiya": "satıcıya",
    "doveun": "dove’un",
    "paketlenmişx": "paketlenmiş",
    "cox": "çok",
    "hediyyeside": "hediyesi de",
    "katmiyorlar": "katmıyorlar",
    "tertemis": "tertemiz",
    "piyasada ki": "piyasadaki",
   "kadar dir": "kadardır",
   "yanindaki": "yanındaki",
   "degeri": "değeri",
   "yayin": "yayın",
   "yayinlari": "yayınları",
   "alıaveriş": "alışveriş",
   "hakeden": "hak eden",
   "gediyeler": "hediyeler",
   "ederimm": "ederim",
   "sapasaglam": "sapa sağlam",
   "paketlenısı": "paketlenmesi",
   "tatli": "tatlı",
   "kullaninca": "kullanınca",
   "ustura": "ultra",
   "indirimdeiken": "indirimdeyken",
   "fırsat di": "fırsattı",
   "almisim": "almışım",
   "begenemedim": "beğenemedim",
   "hafig": "hafif",
   "aluminyum": "alüminyum",
   "islaklık": "ıslaklık",
   "pakatleme": "paketleme",
   "yanlışyanıltıcı": "yanlış ve yanıltıcı",
   "koyafetlere": "kıyafetlere",
   "sunsürsem" : "sürmesem",
"bumdan": "bundan",
"pahalndı": "pahalandı",
"şağım": "şahım",
"endirimde": "indirimde",
"küçuk": "küçük",
"begndk": "beğendik",
"ütün": "bütün",
"herşeyiyle": "her şeyiyle",
"vede": "ve de",
"hedıyenız": "hediyeniz",
"stokuk": "stokluk",
"orıjınal": "orijinal",
"urunude": "ürününü de",
"edecegim": "edeceğim",
"numarasin": "numarasını",
"gore": "göre",
"fiyati": "fiyatı",
"urunden": "üründen",
"yaptm": "yaptım",
"oldugundan": "olduğundan",
"kakar": "kadar",
"vazgecmedigim": "vazgeçmediğim",
"gonderim": "gönderim",
"donup": "dönüp",
"dolasip": "dolaşıp",
"tahris": "tahriş",
"4lu": "4’lü",
"kaçmazdi": "kaçmazdı",
"fşyatına": "fiyatına",
"guel": "güzel",
   "əladır" : "harikadır",
   "hər": "her",
    "pahalandilar": "pahalandılar",
    "almalik": "almalık",
    "sureklı": "sürekli",
    "islaklığın": "ıslaklığın",
    "begenerwk": "beğenerek",
    "kalıte": "kalite",
    "ihtiyacimi": "ihtiyacımı",
    "karsiladi": "karşıladı",
    "kızlarima": "kızlarıma",
    "kullandugım": "kullandığım",
    "rol on": "roll-on",
    "gorelim": "görelim",
    "bakalim": "bakalım",
    "guzal": "güzel",
    "döve": "dove",
    "yillardr": "yıllardır",
    "mü kem mel": "mükemmel",
    "simdilik": "şimdilik",
    "her kes": "herkes",
    "kalıcılıgını": "kalıcılığını",
    "onceiyiinsanolacagim": "önce iyi insan olacağım",
    "beklmeiyordum": "beklemiyordum",
    "stokk": "stok",
    "bnm": "benim",
    "hatika": "harika",
    "mukemmel": "mükemmel",
    "hezaman": "her zaman",
    "gəldi": "geldi",
    "yakaladıgimda": "yakaladığımda",
    "kaçirmadıgim": "kaçırmadığım",
    "sparis": "sipariş",
    "harikasiniz": "harikasınız",
    "coo": "çok",
    "kullanabikeceğimiz": "kullanabileceğimiz",
    "3lu": "3’lü",
    "sağkam": "sağlam",
    "twşwkkür": "teşekkür",
    "whatsons": "watsons",
    "kampanyasindan": "kampanyasından",
    "almisdim": "almıştım",
   "iyisi" : "iyidi",
   "xosdu" : "hoştu",
   "yeteekdir" : "yedektir",
   "şupesiz": "şüphesiz",
    "kaldımm": "kaldım",
    "muthissiin": "müthişsin",
    "sıkıyosunuz": "sıkıyorsunuz",
    "bişe": "bir şey",
    "teşekkür ller": "teşekkürler",
    "geçtikçede": "geçtikçe de",
    "bire bir": "birebir",
    "fiyatli": "fiyatlı",
    "tewekkurler": "teşekkürler",
    "hafih": "hafif",
    "dowe": "dove",
    "kulandigim": "kullandığım",
    "zatenbayılıyorum": "zaten bayılıyorum",
    "tesekkür": "teşekkür",
    "alı": "aldı",
    "güzel di": "güzeldi",
    "aşırıbir": "aşırı bir",
    "beğendimm": "beğendim",
    "geliyodu": "geliyordu",
    "parfum": "parfüm",
    "cicegi": "çiçeği",
    "markanin": "markanın",
    "tatlis": "tatlısı",
    "ındırımdeyken": "indirimdeyken",
    "kokacaksiniz": "kokacaksınız",
    "bulasmiyor": "bulaşmıyor",
    "degistirmem": "değiştirmem",
    "deodorantim": "deodorantım",
    "kullandiğim": "kullandığım",
    "sevdigim": "sevdiğim",
    "almalısınjz": "almalısınız",
    "alsaymısım": "alsaymışım",
    "butun": "bütün",
    "bos": "boş",
    "saçimi": "saçımı",
    "yağli": "yağlı",
   "çoksel" : "çok","siparisi": "siparişi",
    "siparisin": "siparişin",
    "siparis": "sipariş",
    "şipariş": "sipariş",
    "ürun": "ürün",
    "gosterilmedigini": "gösterilmediğini",
    "gostermiyor": "göstermiyor",
    "dun": "dün",
    "trendyyolgo": "trendyol",
    "edecem": "edeceğim",
    "kizginim": "kızgınım",
    "oof": "of",
    "kargom": "kargom",
    "alış veriş": "alışveriş",
    "alışverıs": "alışveriş",
    "sayfaya": "sayfada",
    "site girişi ne": "site girişine",
    "iade edemedım": "iade edemedim",
    "bukadarda": "bu kadar da",
    "dıkkat": "dikkat",
   "paketcisi": "paketçisi",
    "etsin": "etkisini",
    "onarıcıkuru": "onarıcı kuru",
    "birdaha": "bir daha",
    "keremiyle": "kremiyle",
    "artisi": "artısı",
    "bnu": "bunu",
    "bire bir": "birebir",
    "yapiyor": "yapıyor",
    "beyendi": "beğendi",
    "aıtoklama": "stoklama",
    "yapıyorumm": "yapıyorum",
    "sac": "saç",
    "kurtaricim": "kurtarıcım",
    "mükkemmel": "mükemmel",
    "ciktigindan": "çıktığından",
    "sverek": "severek",
    "alısim": "alışım",
    "kasintiya": "kaşıntıya",
    "oncede": "öncede",
    "egzema lı": "egzemalı",
    "ınanılmaz": "inanılmaz",
    "çağre": "çare",
    "acok": "çok",
    "asırı": "aşırı",
    "kremıyle": "kremiyle",
    "sonuc": "sonuç",
    "çox": "çok",
    "banasenin": "bana senin",
    "mucizededi": "mucizevi",
   "sadce" : "sadece",
    "aldgm": "aldığım",
    "yaramıyo": "yaramıyor",
    "onarici": "onarıcı",
    "yolluyorsun": "yolluyorsunuz",
    "kandiriyorsunuz": "kandırıyorsunuz",
    "yorumlardada": "yorumlarda da",
    "yollandigini": "yollandığını",
    "farkli": "farklı",
    "saticidanda": "satıcıdan da",
    "yollanmiş": "yollanmış",
    "açıkcası": "açıkçası",
    "gorecegiz": "göreceğiz",
    "gonderdikleri": "gönderdikleri",
    "gondermişsiniz": "göndermişsiniz",
    "çünki": "çünkü",
    "hissediyorun": "hissediyorum",
    "kaynımda": "kayınbiraderime de",
    "bnu": "bunu",
    "tabiki": "tabii ki",
    "gormedim": "görmedim",
    "yapiyor": "yapıyor",
    "kullaniyiruz": "kullanıyoruz",
    "beyendi": "beğendi",
    "arkadasima": "arkadaşıma",
    "aıtoklama": "stoklama",
    "yapıyorumm": "yapıyorum",
    "nerdese": "neredeyse",
    "bunuda": "bunu da",
    "görecez": "göreceğiz",
    "sac": "saç",
    "kurtaricim": "kurtarıcım",
   "orjinal": "orijinal",
    "mükkemmel": "mükemmel",
    "ciktigindan": "çıktığından",
    "sverek": "severek",
    "şaçlarım": "saçlarım",
    "banasenin": "bana senin",
    "mucizededi": "mucizevi",
    "sac": "saç",
    "çox": "çok",
    "ınanılmaz": "inanılmaz",
    "çağre": "çare",
    "acok": "çok",
    "asırı": "aşırı",
    "kremıyle": "kremiyle",
    "sonuc": "sonuç",
    "egzema lı": "egzemalı",
    "oncede": "öncede",
    "kasintiya": "kaşıntıya",
    "alısim": "alışım",
    "birdaha": "bir daha",
    "kepeğede": "kepeğe de",
    "muhtesem": "muhteşem",
    "guncelleyecegim": "güncelleyeceğim",
    "atopık": "atopik",
    "sampuana": "şampuana",
    "gectim": "geçtim",
    "yaglandirma": "yağlandırma",
    "kepegi": "kepeği",
    "onarmadi": "onarmadı",
    "fsfısl": "fısfıslı",
    "iydi": "iyiydi",
    "egzamalarim": "egzamalarım",
    "cozum": "çözüm",
    "donusumlu": "dönüşümlü",
    "hoss": "hoş",
    "ekleyecegim": "ekleyeceğim",
   "henuz": "henüz",
    "kaldim": "kaldım",
    "urun": "ürün",
    "alıcam": "alacağım",
    "kullanimda": "kullanımda",
    "pahali": "pahalı",
    "deneyiince": "deneyince",
    "yinede": "yine de",
    "saticiyi": "satıcıyı",
    "rahatliği": "rahatlığı",
    "yada": "ya da",
    "hemee": "hemen",
    "senedır": "senedir",
    "haberınız": "haberiniz",
    "çokk": "çok",
    "verdigim": "verdiğim",
    "bakıcam": "bakacağım",
    "paketlenemiş": "paketlenmemiş",
    "göndermicem": "göndermeyeceğim",
    "birdaha": "bir daha",
    "gerek varmıydı": "gerek var mıydı",
    "tabiki": "tabii ki",
    "cozum": "çözüm",
    "faydasi": "faydası",
    "mağza": "mağaza",
    "orjinal": "orijinal",
    "tabiki": "tabii ki",
   "gonderip": "gönderip",
    "gonderdiler": "gönderdiler",
    "gonderilmiş": "gönderilmiş",
    "sac": "saç",
    "interneten": "internet üzerinden",
    "çatdı": "geldi",
    "ədəd": "adet",
    "təşəkkürlər": "teşekkürler",
    "iyii": "iyi",
    "ziyan": "ziyan",
    "vəziyyətdə": "vaziyette",
     "saclarimdaki": "saçlarımdaki",
    "onledi": "önledi",
    "be": "ve",
    "ispatlamıs": "ispatlamış",
    "steçlenip": "streçlenip",
    "sifariş": "sipariş",
    "sampuanimdir": "şampuanımdır",
    "kargi": "kargo",
    "kaldk": "kaldık",
    "ağlıyorum": "alıyorum",
    "vücudmuda": "vücudumu da",
    "tavsıue": "tavsiye",
    "beyen dik": "beğendik",
    "memnu um": "memnunum",
    "indirmden": "indirimden",
    "firoy": "fırsat",
    "kabariklik": "kabarıklık",
    "vazgecmem": "vazgeçmem",
    "elimizi": "elimize",
   "çokbeğendim": "çok beğendim",
    "hizlisiniz": "hızlısınız",
    "kullanıyorız": "kullanıyoruz",
    "iyi hili": "iyi kaliteli",
    "zmaanki": "zamanki",
    "kullanıyo": "kullanıyor",
    "sampuan": "şampuan",
    "zmn": "zaman",
    "edemiyordm": "edemiyordum",
    "çokbeğendim": "çok beğendim",
    "hizlisiniz": "hızlısınız",
    "kullanıyorız": "kullanıyoruz",
    "iyi hili": "iyi kaliteli",
    "zmaanki": "zamanki",
    "kullanıyo": "kullanıyor",
    "sampuan": "şampuan",
    "zmn": "zaman",
    "edemiyordm": "edemiyordum",
     "ozenli": "özenli",
    "bantlayip": "bantlayıp",
    "gondermisler": "göndermişler",
    "kargolanmıs": "kargolanmış",
    "bida": "bir daha",
    "paeketlenmis": "paketlenmiş",
    "5degil": "5 değil",
    "10yildiz": "10 yıldız",
    "orjinal": "orijinal",
    "teşekürler": "teşekkürler",
    "paketlenme": "paketleme",
    "sampuanlarin": "şampuanların",
    "basi": "başı",
    "yariya": "yarıya",
    "bosh": "boş",
    "ferahlağı": "ferahlığı",
    "hissetirmiyor": "hissettirmiyor",
    "ayibdi": "ayıptır",
    "verdigim": "verdiğim",
    "odeme": "ödeme",
    "yalnış": "yanlış",
    "cakma": "çakma",
    "sümen": "sürmen",
    "dolun": "dolum",
    "aldm": "aldım",
    "yavasti": "yavaştı",
    "deyıl": "değil",
    "ıcı": "içi",
    "cakma": "çakma",
    "sankı": "sanki",
    "ustude": "üstü de",
    "kagolama": "kargolama",
    "kozel": "güzel",
    "kokuyoz": "kokuyoruz",
    "söylenmedigi": "söylenmediği",
    "keyfiyyətlidir": "kalitelidir",
    "paketlənməsi": "paketlenmesi",
    "kullanıyo": "kullanıyor",
    "şampun": "şampuan",
    "kaqo": "kargo",
    "paketlənmişdi": "paketlenmişti",
    "təşəkürlər": "teşekkürler",
    "sampuani": "şampuanı",
    "degerlendirdim": "değerlendirdim",
    "diebilirim": "diyebilirim",
    "çox": "çok",
    "gözəl": "güzel",
    "şampundur": "şampuandır",
    "təşəkkürlər": "teşekkürler",
    "ferahliyor": "ferahlıyor",
    "teşekkürlerr": "teşekkürler",
    "indirdeykem": "indirimdeyken",
    "tavsiyye": "tavsiye",
   "təəşəkkürlerre": "teşekkürler",
    "cox": "çok",
    "sagolun": "sağ olun",
    "kanfetede": "şekerleme",
    "gelior": "geliyor",
    "haldır şaldır": "head&shoulders",
    "indimdeyken": "indirimdeyken",
    "düz nli": "düzenli",
    "iyidi": "iyiydi",
    "endirimdə": "indirimde",
    "aldıq": "aldık",
    "kullanıyorr": "kullanıyor",
    "ucz": "ucuz",
    "yakıstı": "yakıştı",
    "teşekkürker": "teşekkürler",
    "ulastialar": "ulaştırdılar",
    "kullanıırım": "kullanırım",
    "sifariş": "sipariş",
    "tökülməmişdi": "dökülmemişti",
    "cüzi": "az",
    "yapışqanlıq": "yapışkanlık",
    "coox": "çok",
    "teşekkurler": "teşekkürler",
   "ulawdi": "ulaştı",
    "ferahlatiyor": "ferahlatıyor",
    "gonderi": "gönderi",
    "paketmeleme": "paketleme",
    "ozenlı": "özenli",
    "guzeldı": "güzeldi",
    "paketlenmisdi": "paketlenmişti",
 "salamlar": "selamlar",
    "gec": "geç",
    "amma": "ama",
    "gozel": "güzel",
    "poketleme": "paketleme",
    "stoklamalk": "stoklanmalık",
    "olmamishda": "olmamıştı",
    "k teşekkür": "çok teşekkür",
    "kargolanmisti": "kargolanmıştı",
    "ozenle": "özenle",
    "paketlenmsti": "paketlenmişti",
    "tesekkuer": "teşekkür",
    "paketlenmıstu": "paketlenmişti",
    "geçikmeli": "gecikmeli",
    "yoğunluktn": "yoğunluktan",
    "paketlemisler": "paketlemişler",
    "gorseldeki": "görseldeki",
    "paketlene": "paketleme",
"original": "orijinal",
    "iyidi": "iyiydi",
    "zaaten": "zaten",
    "sampuan": "şampuan",
    "akmisti": "akmıştı",
    "samsundan": "Samsun’dan",
    "gerek varmıymış": "gerek var mıymış",
    "yaşamadımsadece": "yaşamadım sadece",
    "başqa": "başka",
    "böye": "böyle",
    "şeymii": "şey mi",
    "olurya": "olur ya",
    "düşünemiyosunuz": "düşünemiyorsunuz",
    "ürünn": "ürünün",
    "yavas": "yavaş",
    "şapoan": "şampuan",
    "saten": "zaten",
    "kalite si": "kalitesi",
    "ispatlamıs": "ispatlamış",
    "yanj": "yani",
 "kiçik": "küçük",
    "hediyyeniz": "hediyeniz",
    "ove": "öve",
    "konusduki": "konuştu ki",
    "dayanamadim": "dayanamadım",
    "kullandigik": "kullandığımız",
    "dokmuyor": "dökmüyor",
    "saci": "saçı",
    "gercekden": "gerçekten",
    "guzl": "güzel",
    "trenyoldan": "trendyol’dan",
    "temın": "temin",
    "şekılde": "şekilde",
    "kullanıcağım": "kullanacağım",
    "guzel": "güzel",
 "fiyatolarakta": "fiyat olarak da",
    "kullanış olarakta": "kullanış olarak da",
    "ulasdi": "ulaştı",
    "fiyadı": "fiyatı",
    "en iyidi": "en iyiydi",
    "guzldi": "güzeldi",
    "tenk": "teşekkürler",
    "farki": "farkı",
    "gonderiyolar": "gönderiyorlar",
    "aldgima": "aldığıma",
    "traş": "tıraş",
"gzl": "güzel",
    "konusduki": "konuştu ki",
    "gercekden": "gerçekten",
    "kozmetikcilerde": "kozmetikçilerde",
    "açamadıharika": "açamadı harika",
    "teşekürler": "teşekkürler",
    "paketleeme": "paketleme",
    "pat pata": "patpata",
    "sioariş": "sipariş",
    "yarayack": "yarayacak",
    "fiyatıda": "fiyatı da",
    "marketde": "markette",
    "guvenemedigim": "güvenemediğim",
    "düşünüyorsunuz": "düşürüyorsunuz",
   "kapsğı": "kapağı",
    "dışınds": "dışında",
    "pisman": "pişman",
    "ettigimin": "ettiğimin",
    "alakasi": "alakası",
    "head shouldersin": "head & shoulders'ın",
    "gosteriyor": "gösteriyor",
    "ferahlatıyo": "ferahlatıyor",
    "headshoulders": "head & Shoulders",
    "diğecek": "diyecek",
    "pakatlemede": "paketlemesi de",
    "degistirmedim": "değiştirmedim",
    "degişebilir": "değişebilir",
    "tavsiye uzerıne": "tavsiye üzerine",
    "ıns": "inş",
  "sampuani": "şampuanı",
    "buldugum": "bulduğum",
    "mutluyumm": "mutluyum",
    "bayiliyotumk": "bayılıyorum",
    "sholders": "shoulders",
    "yumşaçık": "yumuşacık",
    "sampuan": "şampuan",
    "sacimi": "saçımı",
    "yumwacik": "yumuşacık",
    "yapdi": "yaptı",
    "aldğm": "aldığım",
    "önliyor": "önlüyor",
      "yalniz": "yalnız",
    "performansj": "performansı",
    "sampuan": "şampuan",
    "yumaşacık": "yumuşacık",
    "muhtemeşem": "muhteşem",
    "kullanmali": "kullanmalı",
    "sampuan": "şampuan",
    "yagli": "yağlı",
    "iyidi": "iyiydi",
    "efso": "efsane" ,
"yumwacik": "yumuşacık",
"memnunm": "memnunum",
    "temzlik": "temizlik",
    "ddkleri": "dedikleri",
    "sahtee": "sahte",
    "sıpariş": "sipariş",
    "aldm": "aldım",
    "ederiim": "ederim",
    "hiçbi": "hiçbir",
    "teşekürler": "teşekkürler",
    "biradan": "buradan",
    "benm": "benim",
    "dükkat": "dikkat",
    "paketlenmiştı": "paketlenmişti",
    "naisl": "nasıl",
    "yaglı": "yağlı",
    "gordugum": "gördüğüm",
    "verdigim": "verdiğim",
    "yagi": "yağı",
    "icermez": "içermez",
    "yazili": "yazılı",
    "zamanimiza": "zamanımıza",
    "spariş": "sipariş",
    "trentyol": "trendyol",
    "trenyolu": "trendyolu",
    "yapilacak": "yapılacak",
    "satcıdan": "satıcıdan",
    "ürüü": "ürünü",
    "aldıgım": "aldığım",
    "olmadıgıı": "olmadığını",
    "sbb": "sebebi",
    "sipariş edecegiz": "sipariş edeceğiz",
    "saha": "daha",
    "kopük": "köpük",
    "7nin": "7'nin",
    "tiel": "tl",
    "balon": "boş",
    "guncelleyecegim": "güncelleyeceğim",
    "sampuandan": "şampuandan",
    "ektili": "etkili",
    "etkiside": "etkisi de",
    "saçımda ki": "saçımdaki",
    "iş görür": "işe yarar",
    "açıkcası": "açıkçası",
    "çalşmalar": "çalışmalar",
    "kurutmus": "kurutmuş",
    "egzamaye": "egzamaya",
    "guncelleyecegim": "güncelleyeceğim",
    "görücez": "göreceğiz",
    "nüksediyor": "nüksetiyor",
    "bayıldım": "bayıldım",
    "sac": "saç",
    "kullanigim": "kullandığım",
    "sikayetim": "şikayetim",
    "cocuklarim": "çocuklarım",
    "pahali": "pahalı",
    "yatıştıryor": "yatıştırıyor",
    "seberoik": "seboreik",
    "bür": "bir",
    "egzamalarimiz": "egzamalarımız",
    "tamamiyle": "tamamıyla",
    "eğzamaları": "egzamaları",
    "dolaşıyodum": "dolaşıyordum",
    "kullanimda": "kullanımda",
    "gosterdi": "gösterdi",
    "guncelliyorum": "güncelliyorum",
    "orjinal": "orijinal",
    "memun": "memnun",
    "sacinizda": "saçınızda",
    "yılamadan": "yıkamadan",
    "qrtti": "arttı",
    "dercs": "Dercos",
    "hakediyor": "hak ediyor",
    "egzamami": "egzamamı",
    "alablirsiniz": "alabilirsiniz",
    "rastlamadim": "rastlamadım",
    "en iyidi": "en iyisiydi",
    "yıkamads": "yıkamada",
    "yağlanmayıda": "yağlanmayı da",
    "kokusuda": "kokusu da",
    "yiğenim": "yeğenim",
    "hiçbişey": "hiçbir şey",
    "pullandi": "pullandı",
    "kasimaktan": "kaşımaktan",
    "bulastigina": "bulaştığına",
    "qncak": "ancak",
    "gorebildim": "görebildim",
    "hicbise": "hiçbir şey",
    "kasurek": "kaşınarak",
    "kasurek": "kaşınarak",
    "saclarimda": "saçlarımda",
    "terlediginiz": "terlediğiniz",
    "dezavantaji": "dezavantajı",
    "bukadar": "bu kadar",
    "ufacığı": "ufacık",
    "memnum": "memnunum",
    "yanlız": "yalnız",
    "sampuan": "şampuan",
    "cozuldu": "çözüldü",
    "olucağınu": "olacağını",
    "konazol": "Konzol",
    "dercosu": "Dercos'u",
   "oda": "o da",
    "aldıgı": "aldığı",
    "hak ediyo": "hak ediyor",
    "ezgama": "egzama",
    "ediyorm": "ediyorum",
    "sampuan": "şampuan",
    "sac": "saç",
    "almadığım": "almadığım",
    "cıldırıyordum": "çıldırıyordum",
    "egzamasi": "egzaması",
    "seberoik": "seboreik",
    "seboroid": "seboreik",
    "kizımda": "kızımda",
    "kullnm": "kullanm",
    "gösteriyo": "gösteriyor",
    "indrim": "indirim",
    "alıyoruz": "alıyoruz",
    "emeginize": "emeğinize",
    "orjinal": "orijinal",
    "mükemmelbir": "mükemmel bir",
    "sampun": "şampuan",
    "yıllardir": "yıllardır",
    "biyik": "bıyık",
    "yaşiyordum": "yaşıyordum",
    "kaşimda": "kaşımda",
    "sacim": "saçım",
    "göstedi": "gösterdi",
    "headshoulders": "Head & Shoulders",
    "egzrmalı": "egzamalı",
      "sbye": "seboreik",
    "stoga": "stoğa",
    "köpekler": "kepekler",
    "iyiki": "iyi ki",
    "gecmeyen": "geçmeyen",
    "egzamasi": "egzaması",
    "davsiye": "tavsiye",
    "sac": "saç",
    "almali": "almalı",
    "saclarima": "saçlarıma",
    "kalmasisti": "kalmamıştı",
    "şambuan": "şampuan",
    "yuz": "yüz",
    "egzema": "egzama",
    "saçtski": "saçtaki",
    "hiçbirseyi": "hiçbir şeyi",
    "seboreit": "seboreik",
    "şsmpuan": "şampuan",
    "saçlarımıda": "saçlarımı da",
    "bıtırdım": "bitirdim",
    "sebatit": "seboreik",
    "egzemasınaniyi": "egzemasına iyi",
    "egzamayi": "egzamayı",
    "seboreit": "seboreik",
    "gittii": "gitti",
    "egzamayi": "egzamayı",
    "yalanların": "yaşayanların",
    "samimi": "samimiyetle",
    "yada": "ya da",
    "dene": "denenebilir",
    "özere": "üzerine",
    "markanım": "markanın",
    "sacıma": "saçıma",
    "sac": "saç",
    "dokulmesi": "dökülmesi",
    "yaglanma": "yağlanma",
    "bir kaç": "birkaç",
    "gûzel": "güzel",
    "oaket": "paket",
    "begendimm": "beğendim",
    "kasintiya": "kaşıntıya",
    "kasintiyi": "kaşıntıyı",
    "bioy": "boy",
    "yada": "ya da",
    "sampuan": "şampuan",
    "actım": "açtım",
    "ezgamam": "egzamam",
    "dəfədir": "defadır",
    "alıram": "alıyorum",
    "keyfiyyəti": "kalitesi",
    "qablaşması": "paketlenmesi",
    "özeniçin": "özeni için",
    "alışverişlər": "alışverişler",
    "busey": "bir şey",
    "saölara": "saçlara",
    "ilkdefa": "ilk defa",
    "kulanıyorum": "kullanıyorum",
    "bene": "bence",
    "geçirceğimi": "geçireceğimi",
    "cvp": "cevap",
    "varmı": "var mı",
    "beğendin": "beğendim",
      "teşekürler": "teşekkürler",
    "kaldim": "kaldım",
    "kasintim": "kaşıntım",
    "azldi": "azaldı",
    "salam": "selam",
    "ulasdi": "ulaştı",
    "görücez": "göreceğiz",
    "5yildiz": "5 yıldız",
    "bərbad": "berbat",
    "saclarimi": "saçlarımı",
    "dokdu": "döktü",
    "maelesef": "maalesef",
    "iyrenc": "iğrenç",
    "sacimda": "saçımda",
    "oluyo": "oluyor",
    "görmrdim": "görmedim",
    "donduğunu": "donduğunu",
    "düşüp": "düşünüp",
    "helal": "helâl",
    "abikirmi": "ispatlayabilir mi",
    "soylendigi": "söylendiği",
    "zmaan": "zaman",
    "kararsizim": "kararsızım",
    "karısık": "karışık",
    "seboroit": "seboreik",
    "gerckten": "gerçekten",
    "baştaki": "baş derisindeki",
    "elimizdeydipaketleme": "elimizdeydi paketleme",
    "orjinal": "orijinal",
    "tavsite": "tavsiye",
    "teşekür": "teşekkür",
    "azaldi": "azaldı",
    "mükemell": "mükemmel",
  "gercektende": "gerçekten de",
  "egzmalarim": "egzamalarım",
  "tesekur": "teşekkür",
  "egzama ya": "egzamaya",
  "sampuan": "şampuan",
  "urunu": "ürünü",
  "urün": "ürün",
  "sd": "seboreik dermatit",
  "köpeği": "kepeği",
  "sac": "saç",
  "saclarinda": "saçlarında",
  "sacima": "saçıma",
  "egzemayi": "egzamayı",
  "egzemamı": "egzamamı",
  "solusyonlar": "solüsyonlar",
  "şampuananlardan": "şampuanlardan",
  "kaşırken": "kaşıyarak",
  "menmun": "memnun",
  "kelliğe": "kel kalmaya",
    "sampuandan": "şampuandan",
  "egzamasina": "egzamasına",
  "hersey": "her şey",
  "şampuana": "şampuana",
  "egzamalarim": "egzamalarım",
  "egzamami": "egzamamı",
  "karşıda": "karşı da",
  "sapasağlam": "sapasağlam",
  "egzemayi": "egzamayı",
  "egzemamı": "egzamamı",
  "seboraik": "seboreik",
  "haketti": "hak etti",
  "kararsizim": "kararsızım",
  "menmun": "memnun",
  "egzemadan": "egzemadan",
  "kepeğe": "kepeğe",
  "başkasını": "başka birini",
  "birsey": "bir şey",
  "yüzü": "yüzünde",
  "kargosu": "kargo",
  "hop": "hemen",
  "yinede": "yine de",
  "egzema": "egzama",
  "egzemayi": "egzamayı",
  "vaşında": "başında",
  "cak": "çok",
  "herzamn": "her zaman",
  "bıraz": "biraz",
  "fıyat": "fiyat",
  "yuksek": "yüksek",
  "degıyor": "değiyor",
  "kepei": "kepeği",
  "arkadasimin": "arkadaşımın",
  "sampuan": "şampuan",
  "edecegiz": "edeceğiz",
   "urunnu": "ürününü",
  "kullanisim": "kullanışım",
  "sampuan": "şampuan",
  "tesekkrler": "teşekkürler",
  "herzamanki": "her zamanki",
  "sac": "saç",
  "dokulmem": "dökülmem",
  "bıtmeden": "bitmeden",
  "saclarımın": "saçlarımın",
  "sacimda": "saçımda",
  "egzemadan": "egzamadan",
  "kalmmamişti": "kalmamıştı",
  "süremediğğm": "süremediğim",
  "şambuan": "şampuan",
  "kayet": "gayet",
  "teşekürler": "teşekkürler",
  "trent yol": "Trendyol",
  "tekrarla": "tekrarlamıyor",
  "moruk": "moruk",
  "tamamem": "tamamen",
  "3ü1": "3ü 1",
  "bantlanmamişti": "bantlanmamıştı",
  "haldır şaldırdan": "head&Shoulders'dan",
  "ağzı acık": "ağzı açık",
  "fotoğrafda": "fotoğrafta",
  "sampun": "şampuan",
  "bəzisi": "bazısı",
  "yarımçıq": "yarımçık",
  "sifariş": "sipariş",
  "sampuanin": "şampuanın",
  "gözəldi": "güzeldi",
  "qaldım": "kaldım",
  "bantlanmamişti": "bantlanmamıştı",
  "bir taneti": "bir tanesi",
  "atdım": "attım",
  "çekə bilmədim": "çekemedim",
  "gedi": "geldi",
  "cvb": "cevap",
  "gördermişsiniz": "göndermişsiniz",
  "lətişimə keçəmiyorum": "iletişime geçemiyorum",
  "işətişimə keçin": "iletişime geçin",
  "atdım": "attım",
  "çekə bilmədim": "çekemedim",
  "gekmiw": "geçmiş",
  "tafsye": "tavsiye",
  "pesmanim": "pişmanım",
  "ödenis": "ödeme",
  "gondeeilmese": "gönderilmezse",
  "alabılırdım": "alabilirdim",
  "yazmışssınız": "yazmışsınız",
  "yazıyo": "yazıyor",
  "diyo": "diyor",
  "srıku": "son kullanma tarihi",
  "gekmis": "geçmiş",
  "yaxsidir": "iyidir",
  "sifaris": "sipariş",
  "gelib": "geldi",
  "pakrtlenmeyib": "paketlenmemiş",
  "axib": "akıp",
  "yarilanmisti": "yarılanmıştı",
  "2si": "2'si",
  "sarmış sınız": "sarmışsınız",
  "400 m": "400 ml",
  "350 m": "350 ml",
  "verececem": "vereceğim",
  "şidetle": "şiddetle",
  "kulladigim": "kullandığım",
  "ürünu": "ürünü",
  "gormemis": "görmemiş",
  "muhtesem": "muhteşem",
  "bəyəndim": "beğendim",
  "təşəkkürlər": "teşekkürler",
  "nası": "nasıl",
  "özür": "ürün",
  "teşekürler": "teşekkürler",
  "pantane": "pantene",
  "tesekkürler": "teşekkürler",
  "aileceekk": "ailece",
  "güel": "güzel",
  "her kəsə": "herkese",
  "tövsiyə edirəm": "tavsiye ederim",
  "təşəkkür edirəm": "teşekkür ederim",
  "süperr": "süper",
  "nett": "net",
   "mühtesem": "muhteşem",
  "alicam": "alacağım",
  "tasarruflu": "tasarruflu",
  "tatttışılmaz": "tartışılmaz",
  "şampunlar": "şampuanlar",
  "əla": "harika",
  "vəziyyətdə": "vaziyette",
  "gəlmişdi": "gelmişti",
  "almamışdı": "almamıştı",
  "paketlendigini": "paketlendiğini",
  "hepsıde": "hepsi de",
  "400yszdıgı": "400 yazdığı",
  "stok luk": "stokluk",
  "tskr": "teşekkür",
  "mumnunum": "memnunum",
  "ürünum": "ürünüm",
  "iyidi": "iyiydi",
  "orjinal": "orijinal",
  "gayet": "gayet",
  "stok yapcam": "stok yapacağım",
  "sarilmisti": "sarılmıştı",
  "eline": "elime",
  "yukseliyor": "yükseliyor",
  "hususi": "hususi",
  "vok": "çok",
  "gorselinde": "görselinde",
  "görünsün": "görünsün",
  "soylenemez": "söylenemez",
  "siparş": "sipariş",
  "1 sinde": "1'inde",
  "şanpuanlara": "şampuanlara",
  "karıştırılmıs": "karıştırılmış",
  "guze": "güzel",
  "urunde": "üründe",
  "kalitei": "kaliteli",
  "aldıgımız": "aldığımız",
  "teşekğler": "teşekkürler",
  "kirildi": "kırıldı",
  "özenliydibence": "özenliydi bence",
  "tremdyolda": "trendyol'da",
  "teşekkür ller": "teşekkürler",
  "cozum": "çözüm",
  "donusumlu": "dönüşümlü",
  "kullanacagım": "kullanacağım",
  "çokk": "çok",
   "olmus": "olmuş",
  "ulasmamıs": "ulaşmamış",
  "ulasmadıgı": "ulaşmadığı",
  "gıbıde": "gibi de",
  "tahmınımce": "tahminimce",
  "sonrada": "sonra da",
  "ınceleme": "inceleme",
  "sureci": "süreci",
  "ulasmadıgı": "ulaşmadığı",
  "ıadesı": "iadesi",
  "gormedım": "görmedim",
  "nasil": "nasıl",
  "boştur": "boştu",
  "çakma": "sahte",
  "greldi": "geldi",
  "kapaklari": "kapakları",
  "dagilmis": "dağılmış",
  "serisinide": "serisini de",
  "özelliyi": "özelliği",
  "saclarıma": "saçlarıma",
  "saçlariniz": "saçlarınız",
  "aeve": "seve seve",
  "şaşmam": "şaşmam",
  "kargolama": "kargolama",
  "paketlemesii": "paketlemesi",
  "güzell": "güzel",
  "beyenmishdim": "beğenmiştim",
  "saclari": "saçları",
  "yumusak": "yumuşak",
  "sampunun": "şampuanını",
  "almayi": "almayı",
  "kerar": "karar",
  "saclarin": "saçların",
  "dokulmesini": "dökülmesini",
  "azaldiginda": "azaldığında",
  "basarli": "başarılı",
  "pketlenmis": "paketlenmiş",
  "güzel di": "güzeldi",
  "saçıda": "saçı da",
  "aslada": "asla da",
  "stoklamayi": "stoklamayı",
  "sacalarim": "saçlarım",
  "kulaniyorum": "kullanıyorum",
  "vazgecilmezim": "vazgeçilmezim",
  "kullanımi": "kullanımı",
  "iyiki": "iyi ki",
  "almşm": "almışım",
  "almştm": "almıştım",
  "alacağim": "alacağım",
  "mükemmellimsi": "mükemmelsi",
    "yapıyo": "yapıyor",
  "kacirmiyorum": "kaçırmıyorum",
  "ykalr": "yakalır",
  "ber": "beri",
  "deniycemm": "deneyeceğim",
  "düştükce": "düştükçe",
  "dokudu": "dokusu",
  "kremı": "kremi",
  "yolrulmadan": "yorulmadan",
  "mükemmell": "mükemmel",
  "müko": "mükemmel",
  "dokuluyordu": "dökülüyordu",
  "gayret": "gayet",
  "vaz geçilmezim": "vazgeçilmezim",
  "bekledigim": "beklediğim",
  "pantene nin": "pantene'nin",
  "uzamasina": "uzamasına",
  "sagliyor": "sağlıyor",
  "yumusacik": "yumuşacık",
  "sacları": "saçları",
  "olmasıda": "olması da",
  "kalite li": "kaliteli",
  "panten": "pantene",
  "puanıda": "puanı da",
  "ındirimde": "indirimde",
  "baya": "bayağı",
  "99tl": "99 TL",
  "3lü": "3'lü",
  "umarim": "umarım",
  "cikar": "çıkar",
  "ucuk": "uçuk",
  "sizde": "siz de",
  "kokutuyo": "kokutuyor",
  "bantliydi": "bantlıydı",
  "almışttm": "almıştım",
  "yokmu": "yok mu",
  "problems": "problemleri",
  "yapıyo": "yapıyor",
  "ağırlaştırmıyo": "ağırlaştırmıyor",
  "ürün ü": "ürünü",
  "açmaktabir": "açmakta bir",
  "mükmnel": "mükemmel",
"basladm": "başladım",
  "yumusatiyor": "yumuşatıyor",
  "kapaklari": "kapakları",
  "akmasin": "akmasın",
  "paktlemesi": "paketlemesi",
  "falandı": "falandı",
  "üründe": "üründe",
  "özenl i": "özenli",
  "ambalajnmis": "ambalajlanmış",
  "şampuandı": "şampuandı",
  "gondermek": "göndermek",
  "ustumu": "üstümü",
  "ıade": "iade",
  "boylede": "böyle de",
  "dedırttı": "dedirtti",
  "sızı": "sizi",
  "ısını": "işini",
  "ınsanlar": "insanlar",
  "inşaallah": "inşallah",
  "teşekkürlerr": "teşekkürler",
    "saçi": "saçı",
  "fiyatta": "fiyata",
  "masallah": "maşallah",
  "orjenel": "orijinal",
  "paketlem": "paketleme",
  "yanliz": "yalnız",
  "cok": "çok",
  "gecc": "geç",
  "burdada": "burada da",
  "şaplam": "sağlam",
  "tşkederiz": "teşekkür ederiz",
  "harka": "harika",
  "kendiside": "kendisi de",
  "unutucaktık": "unutacaktık",
  "fec": "feci",
   "bantlamissiniz": "bantlamışsınız",
  "geçti olsa": "geç de olsa",
  "genede": "gene de",
  "heryerden": "her yerden",
  "beğendikçok": "beğendik çok",
  "gonderimler": "gönderimler",
  "foyatına": "fiyatına",
  "fiyetg": "fiyat",
  "fiatla": "fiyatla",
  "almıştim": "almıştım",
  "pantone": "pantene",
  "baküye": "bakü'ye",
  "kullandikdan": "kullandıktan",
  "giyat": "fiyat",
  "edir": "ediyor",
   "trendi": "trendyol",
  "güzəll": "güzel",
  "3tane": "3 tane",
  "serini": "serisini",
  "qayet": "gayet",
  "iyu": "iyi",
  "pantenenin": "pantene'nin",
  "degisiyor": "değişiyor",
  "yapti": "yaptı",
  "her zamn": "her zaman",
  "kullanıdığım": "kullandığım",
  "sapa sağlam": "sapasağlam",
   "panten": "pantene",
  "iyiydibeğendim": "iyiydi beğendim",
  "fiati": "fiyatı",
  "kampanyali": "kampanyalı",
  "trendyol dan": "trendyol'dan",
  "alişveriş": "alışveriş",
  "kalmiş": "kalmış",
  "yazmaktayim": "yazmaktayım",
  "pketleme": "paketleme",
  "patlamq": "patlama",
  "ürünümüz": "ürününüz",
  "sorunsun": "sorunsuz",
  "fiyatta": "fiyat da",
  "kutulanmis": "kutulanmış",
  "bantlanmis": "bantlanmış",
  "birsekilde": "bir şekilde",
  "satişlar": "satışlar",
  "sevdım": "sevdim",
  "verıldi": "verildi",
  "pantene den": "Pantene'den",
  "edicem": "edeceğim",
  "acma": "açma",
  "sansim": "şansım",
  "kepege": "kepeğe",
  "gelmis": "gelmiş",
  "kullanmayacagim": "kullanmayacağım",
  "göderildi": "gönderildi",
  "tessize": "teslim",
  "lim": "edilmedi",
  "gibiydi": "gibi",
  "sızmış": "sızmıştı",
  "hepsı": "hepsi",
  "iaede": "iade",
  "filan": "falan",
  "pntene": "pantene",
  "hedyesi": "hediyesi",
  "paketlemesinler": "paketlemesinden",
  "sanpuan": "şampuan",
  "magazalardan": "mağazalardan",
  "gibii": "gibi",
  "mükkemel": "mükemmel",
  "pentan": "pantene",
    "tavsuye": "tavsiye",
  "parlaklik": "parlaklık",
  "veriyir": "veriyor",
  "kunlandigim": "kullandığım",
  "adresşm": "adresim",
  "oantene": "pantene",
  "trendyoll": "Trendyol",
  "hepte": "hep de",
  "azcık": "azıcık",
  "pantenden": "pantene'den",
   "uladi": "ulaştı",
  "yari": "yarı",
  "yumuşacik": "yumuşacık",
  "öncede": "önce de",
  "bakimiile": "bakımı ile",
  "baketlenkisti": "paketlenmişti",
  "yapio": "yapıyor",
  "iyiydibeğendim": "iyiydi beğendim",
  "açıkcasi": "açıkçası",
  "normalin": "normal bir",
  "kokusunuda": "kokusunu da",
  "kısmınada": "kısmına da",
  "çobuk": "çabuk",
  "gorundugu": "göründüğü",
  "aynısı": "aynısı",
  "yazicam": "yazacağım",
  "saglık": "sağlık",
  "gönderiryorlar": "gönderiyorlar",
  "urunumude": "ürünümü de",
  "gonderebilirmisiniz": "gönderebilir misiniz",
  "yarisina": "yarısına",
  "kadr": "kadar",
  "bem": "nem",
  "siseler": "şişeler",
  "yarisi": "yarısı",
  "insallah": "inşallah",
  "normalşampuan": "normal şampuan",
  "alinca": "alınca",
  "anlatildigi": "anlatıldığı",
  "kalteli": "kaliteli",
  "surat": "Sürat",
  "kullanioz": "kullanıyoruz",
  "aliorum": "alıyorum",
  "salecocuk": " ",
  "dogal": "doğal",
  "sureci": "süreci",
  "fiyattı": "fiyatı",
  "çk": "çok",
  "güzel": "güzel",
  "sacı": "saçı",
  "ymuuş": "yumuş",
  "benegdim": "beğendim",
  "ba yıl dım": "bayıldım",

  "indiirmde": "indirimde",
  "geldkten": "geldikten",
  "paketlenmıstı": "paketlenmişti",
  "gülcelerim": "güncelleyeceğim",
  "di": "diydi",
  "kullanöaya": "kullanmaya",
  "şaahe": "şahane",
  "sapasağlam": "sapasağlam",
  "kasindiriyor": "kaşındırıyor",
  "nemlendrdi": "nemlendirdi",
  "ulduz": "yıldız",
  "haqqinda": "hakkında",
  "etdim": "ettim",
  "hamisi": "hepsi",
  "demək": "demek",
  "elə": "öyle",
  "nəysə": "neyse",
  "etmirəm": "etmiyorum",
  "deniyim": "deneyeyim",
  "yılkardır": "yıllardır",
  "değiştircem": "değiştireceğim",
  "phanthene": "pantene",
  "bayıldımm": "bayıldım",
  "sonundaa": "sonunda",
  "kullanıyorun": "kullanıyorum",
  "yumşacık": "yumuşacık",
  "fırma": "firma",
  "kaçırmıyor um": "kaçırmıyorum",
  "almnır": "alınır",
   "stok da": "stokta",
  "ıstedıgım": "istediğim",
  "gıbı": "gibi",
  "panten": "pantene",
  "paketlenmi": "paketlenmiş",
  "akma yapmamisti": "akma yapmamıştı",
  "memmunum": "memnunum",
  "pantone": "pantene",
  "yumusacık": "yumuşacık",
  "yapıuor": "yapıyor",
  "sacı": "saçı",
  "bıkac": "birkaç",
  "rengını": "rengini",
  "ozellıkle": "özellikle",
  "onerırım": "öneririm",
  "yumusacik": "yumuşacık",
  "memnumdum": "memnundum",
  "saclarim": "saçlarım",
  "yumusacik": "yumuşacık",
  "herbak": "herbal",
  "muazzım": "muazzam",
  "saçlari": "saçları",
  "kremınde": "kreminde",
  "yumusatıyor": "yumuşatıyor",
  "jolay": "kolay",
  "şampuaninıda": "şampuanını da",
  "deniyim": "deneyeyim",
  "kullanıodum": "kullanıyordum",
  "yumusacik": "yumuşacık",
  "by": "bu",
  "sktsi": "SKT'si",
  "sans": "şans",
    "kulalndigim": "kullandığım",
  "efsana": "efsane",
  "senelerfır": "senelerdir",
  "enfed": "enfes",
  "sampuanlar": "şampuanlar",
  "sülfatsiz": "sülfatsız",
  "örjinal": "orijinal",
  "sahlam": "sağlam",
  "birkısmı": "bir kısmı",
  "alacagınız": "alacağınız",
  "bittitken": "bittikten",
  "şampuansiz": "şampuansız",
   "hoşş": "hoş",
  "icerikli": "içerikli",
  "kulanmışım": "kullanmışım",
  "umud": "umut",
  "kutumdayim": "kutumdayım",
  "saçlarimla": "saçlarımla",
  "savaşim": "savaşım",
  "görmüs": "görmüş",
  "dustan": "duştan",
  "sampuanla": "şampuanla",
  "tanıstımmucize": "tanıştım mucize",
  "artik": "artık",
  "yumusacık": "yumuşacık",
  "cıkıyorum": "çıkıyorum",
  "yapıyo": "yapıyor",
  "yumşacık": "yumuşacık",
  "yumaşık": "yumuşacık",
  "saca": "saça",
  "yumusaklik": "yumuşaklık",
  "saçmalarıma": "saçlarıma",
  "heldi": "geldi",
  "teknkeşime": "teşekkürler",
  "sampuanlarindan": "şampuanlarından",
  "kopurtebiliyorum": "köpürtebiliyorum",
  "kabartmıyo": "kabartmıyor",
  "bağımlıyim": "bağımlıyım",
  "arindiriyor": "arındırıyor",
  "kullnadgm": "kullandığım",
  "kisin": "kışın",
  "saclarim": "saçlarım",
  "onarıcı olanini": "onarıcı olanını",
  "umarim": "umarım",
  "iyiydii": "iyiydi",
  "nenlendiriyor": "nemlendiriyor", "yapıyo": "yapıyor",
  "bıraktıgı": "bıraktığı",
  "acyip": "acayip",
  "gunu": "günü",
  "miiss": "miss",
  "kivircik": "kıvırcık",
  "yapti": "yaptı",
  "işcin": "için",
  "şampuandi": "şampuanıydı",
  "arganölü": "argan yağı",
  "coconut lısına": "hindistan cevizlisine",
  "gecmiş": "geçmiş",
  "mü": "mi",
  "basit": "basit",
  "yumushatiyordu": "yumuşatıyordu",
  "ana": "ama",
  "yara": "yara yaptı",
  "orjinaldir": "orijinaldir",
  "orjinalligi": "orijinalliği",
  "guven": "güven",
  "oldugunu": "olduğunu",
  "icerisinde": "içerisinde",
  "kuklanıyorum": "kullanıyorum",
  "iyidi": "iyiydi",
  "bişi": "bir şey",
  "saçlarimi": "saçlarımı",
  "sulfatfree": "sülfatfree",
  "mukemmell": "mükemmel",
  "kullanyor": "kullanıyor",
  "yumusacik": "yumuşacık",
  "kokuu": "kokusu",
  "düşdü": "düştü",
  "zamaman": "zamanlar",
  "kulladığım": "kullandığım",
  "kabarik": "kabarık",
  "kopurtebiliyorumm": "köpürtebiliyorum",
  "cesidinden": "çeşidinden",
  "tavsıyeyle": "tavsiyesiyle",
  "canlandirici": "canlandırıcı",
  "efsanee": "efsane",
  "kokuyordum": "kokuyordu",
  "geldim": "geldi",
  "yumusaq": "yumuşak",
  "qabardir": "kabardır",
  "sampuanim": "şampuanım",
  "kopuruyo": "köpürüyor",
  "iyidi": "iyiydi",
  "saclari": "saçları",
  "yumusacik": "yumuşacık",
  "sacta": "saçta",
  "mükü": "müthiş",
  "yapıyr": "yapıyor",
  "sülfatsz": "sülfatsız",
  "olmazsınnz": "olmazsınız",
  "kullanmsın": "kullanmasın",
  "yumşacık": "yumuşacık",
  "memnunun": "memnunum",
  "ilknkez": "ilk kez",
  "sulfatsiz": "sülfatsız",
  "tuzsuz": "tuzsuz",
  "icindekilerde": "içindekilerde",
  "icerikleri": "içerikleri",
  "almis": "almış",
  "yumusak": "yumuşak",
  "kullandikca": "kullandıkça",
  "hennuz": "henüz",
  "olmasina": "olmasına",
  "kirik": "kırık",
  "cikti": "çıktı",
   "teşekküeler": "teşekkürler",
  "yumusacik": "yumuşacık",
  "yorum yapicam": "yorum yapacağım",
  "saclarin": "saçların",
  "duzlestirip": "düzleştirip",
  "matlastiriyor": "matlaştırıyor",
  "gelmemistim": "gelmemiştim",
  "sertlestiriyo": "sertleştiriyor",
  "okadae": "o kadar",
  "kötükü": "kötü ki",
  "sacımı": "saçımı",
  "öner miyorum": "önermiyorum",
  "yapri": "yaptı",
  "iy deyildi": "iyi değildi",
  "ho kokulu": "hoş kokulu",
  "kopurme": "köpürme",
  "abartildigi": "abartıldığı",
  "sertleştiyordu": "sertleştiriyordu",
  "yasiyosunuz": "yaşıyorsunuz",
  "zamdir": "zamdır",
  "allah aşkına": "Allah aşkına",
  "kessinlikle": "kesinlikle",
  "yumşatmadı": "yumuşatmadı",
  "herhal": "herbal",
  "sertleştiyordu": "sertleştiriyordu",
  "yasiyosunuz": "yaşıyorsunuz",
  "zamdir": "zamdır",
  "allah aşkına": "Allah aşkına",
  "kessinlikle": "kesinlikle",
  "yumşatmadı": "yumuşatmadı",
  "herhal": "herbal",
  "mis": "mis gibi",
  "saçı": "saçımı",
  "watsonda": "Watson's'ta",
  "gratiste": "Gratis'te",
  "strec": "streç",
  "inanlmaz": "inanılmaz",
  "yumusak": "yumuşak",
  "kokuyorr": "kokuyor",
  "saçimda": "saçımda",
  "denememdim": "denememiştim",
  "güncellme": "güncelleme",
  "yapiyo": "yapıyor",
  "müthiş": "müthiş",
  "katıyo": "katıyor",
  "banyoyu": "banyo",
  "şerbeti": "şerbeti gibi",
  "kremle": "krem ile",
  "görmedim": "görmedim böyle",
  "şampuan": "şampuanı",
  "urun": "ürün",
  "saçıma": "saçımı",
  "frekans": "frekansı",
  "şampuabları": "şampuanları",
  "inanılı": "inanılır",
  "yumusacik": "yumuşacık",
  "arttiriyor": "artırıyor",
  "saçım da": "saçımda",
  "coagldi": "çoğaldı",
  "kesiikle": "kesinlikle",
  "deilim": "değilim",
  "şapıanı": "şampuanı",
  "şikayey": "şikayet",
  "orijinalliğinden": "orijinalliğinden (şüphe ettim)" ,
  "şekil": "şekilde",
  "teslimatta": "teslimat da",
  "problemimde": "problemim de",
  "ürünleri": "ürünleri (replika olabilir)",
  "kabuk": "kabuklara",
  "silinmiş": "silinmiş (tarihi)",
   "yaglandiriyor": "yağlandırıyor",
  "yumusatma": "yumuşatma",
  "sacta": "saçta",
  "sertlestirdi": "sertleştirdi",
  "sulfatsiz": "sülfatsız",
  "saclariniz": "saçlarınız",
  "saglamdı": "sağlamdı",
  "bakimlı": "bakımlı",
  "taraniyor": "taranıyor",
  "görselde ki": "görseldeki",
  "piyasan": "piyasadan",
  "köyü": "kötü",
  "kremiyle": "kremi ile",
  "birçoğumuz": "birçoğumuz",
  "bir çoğumuz": "birçoğumuz",
  "kılladığı": "kullandığı",
  "esım": "eşim",
  "harıkaymıs": "harikaymış",
  "ole": "öyle",
  "güze": "güzel",
  "paketlenmeşti": "paketlenmişti",
  "saçımıi": "saçımı",
  "survıvor": "Survivor",
  "bıkac": "birkaç",
  "paktelenmiş": "paketlenmiş",
  "yapıyo": "yapıyor",
  "dokulmuyor": "dökülmüyor",
  "sacımdaki": "saçımdaki",
  "sampuanim": "şampuanım",
  "kargoladi": "kargoladı",
  "yumusacık": "yumuşacık",
  "brgendım": "beğendim",
  "sampuanlarina": "şampuanlarına",
  "dokulmeme": "dökülmeme",
  "saclarinida": "saçlarını da",
  "olmasida": "olması da",
  "sacların": "saçların",
  "çokta": "çok da",
  "gidiyo": "gidiyor",
  "türkiyede": "Türkiye'de",
  "sevenerek": "severek",
  "yumuşacık yapıo": "yumuşacık yapıyor",
  "kullanmana": "kullanmanıza",
  "yapıyo": "yapıyor",
  "kullanmistim": "kullanmıştım",
  "saclari": "saçları",
      "suredir": "süredir",
  "kullaniorum": "kullanıyorum",
  "avakadolusunu": "avokadolusunu",
  "yapior": "yapıyor",
  "şampuanlar da": "şampuanlarda",
  "müthişş": "müthiş",
  "herkeze": "herkese",
  "kıllansan": "kullansan",
  "tüleren": "tüneyen",
  "gelem": "gelen",
  "güzl": "güzel",
  "yıllarıdr": "yıllardır",
  "kopuren": "köpüren",
  "miktari": "miktarı",
  "yumulacık": "yumuşacık",
   "teshekkürler": "teşekkürler",
  "durduracaga": "durduracağa",
  "saçımı çok iyi geldi": "saçıma çok iyi geldi",
  "bambulu": "bambulu (bambu özlü)",
  "gğzel": "güzel",
  "oöur": "olur",
  "umarımm": "umarım",
  "sampuanı": "şampuanı",
   "kullanıyorm": "kullanıyorum",
  "ind": "şimdilik",
  "sülfat siz": "sülfatsız",
  "mis gibi gibi": "mis gibi",
  "tebrıklet": "tebrikler",
  "arıyosanız": "arıyorsanız",
  "degerlendirilebilecek": "değerlendirilebilecek",
  "kullandığjm": "kullandığım",
  "iste": "işte",
  "ıdare": "idare",
  "qaliba": "galiba",
  "aldımğım": "aldığım",
  "birşeyde": "bir şey de",
  "kullaniyorym": "kullanıyorum",
  "azcigi": "azıcığı",
  "kopuruyor": "köpürüyor",
  "kulandığım": "kullandığım",
  "teşkkürler": "teşekkürler",
  "sapa salam": "sapasağlam",
  "harikaydi": "harikaydı",
  "sağsağlim": "sapasağlam",
  "kullandigimix": "kullandığımız",
  "kızkardeşimin": "kız kardeşimin",
  "uğrasmicam": "uğraşmayacağım",
  "aldatmasina": "aldatmasına",
  "uzerine": "üzerine",
  "parami": "paramı",
  "haklarina": "haklarına",
  "basvuracagim": "başvuracağım",
  "yasiyorumm": "yaşıyorum",
  "alışveriş yapmayin": "alışveriş yapmayın",
  "kepeğiö": "kepeğim",
  "upucuz": "uçuk ucuz",
  "orjinalini": "orijinalini",
  "kesinlikte": "kesinlikle",
  "hiş": "hiç",
  "saçim": "saçım",
  "dokiyor": "döküyor",
  "saçlqrda": "saçlarda",
  "aslından": "aslında",
  "oldukça uygun'a": "oldukça uyguna",
  "kuafordekiyle": "kuafördekiyle",
  "duzlestirdim": "düzleştirdim",
  "yikamadim": "yıkamadım",
  "gırtlağım": "boğazım",
  "sıyahtı": "siyahtı",
  "ürünlerın": "ürünlerin",
  "bununkıler": "bununkiler",
  "yesilorjinaldir": "yeşil orijinaldir",
  "yumusakcik": "yumuşacık",
  "farını": "faydasını",
  "tik ederim": "teşekkür ederim",
  "saclar": "saçlar",
  "isil": "ışıl",
  "sampuanlardan": "şampuanlardan",
  "avucla": "avuçla",
  "yikarken": "yıkarken",
  "urunle": "ürünle",
  "dimeticon": "dimethicone",
  "yağlandırmayi": "yağlandırmayı",
  "herbalın": "herbal’in",
  "şampuanı": "şampuan" ,
  "yağıda": "yağı da",
  "kullanamamm": "kullanamam",
  "saclaeim": "saçlarım",
  "bir kaç": "birkaç",
  "zamaninda": "zamanında",
  "kıllanı": "kullanıyorum",
  "yumusacik": "yumuşacık",
  "mis gibi gibi": "mis gibi",
  "dokunuyordu": "dökülüyordu",
  "memnünüm": "memnunum",
  "sapmuandan": "şampuandan",
  "kokuyorki": "kokuyor ki",
 "şampuanı": "şampuan",
  "yaglandirioir": "yağlandırıyor",
  "yumus": "yumuşaklık",
  "mis gibi gibi": "mis gibi",
  "onledigi": "önlediği",
  "kukusu": "kokusu",
  "sampunu": "şampuanı",
  "defe": "defa",
  "istifade etmisdim": "kullanmıştım",
  "ikincini": "ikincisini",
  "ucun": "için",
  "edirem": "ediyorum",
  "diqqetiniz": "dikkatiniz",
  "sag olun": "sağ olun",
  "vermeyebilur": "vermeyebilir",
  "paketləmişdilər": "paketlemişlerdi",
  "təşəkkür edirəm": "teşekkür ederim",
  "saçlarda ki": "saçlardaki",
  "gözel": "güzel",
  "şampuanı": "şampuan",
  "sertlestirmiyor": "sertleştirmiyor",
  "kumaşı": "ürünü",
  "beyenmedim": "beğenmedim",
  "iyade": "iade",
  "etim": "ettim",
  "mukemel": "mükemmel",
  "şampuanı": "şampuan",
  "tevsiye": "tavsiye",
  "maskesinide": "maskesini de",
  "diydi": "değildi",
  "herbal ın": "herbal’in",
  "yumuşacak": "yumuşacık",
  "yikayabiliyorsunuz": "yıkayabiliyorsunuz",
  "etkılı": "etkili",
  "her seyi": "her şeyi",
  "etkısıno": "etkisini",
  "çokmuşum": "çok uzun",
"ıste": "işte",
  "ıdare": "idare",
  "yumaşaklık": "yumuşaklık",
  "yapılmayam": "yapılmayan",
  "yikayabiliyorsunuz": "yıkayabiliyorsunuz",
  "etkılı": "etkili",
  "her seyi": "her şeyi",
  "etkısıno": "etkisini",
  "çokmuşum": "çok uzun",
  "pakeltlemesi": "paketlemesi",
  "yapılmıs": "yapılmış",
  "gorucez": "göreceğiz",
  "gunde": "günde",
  "dokulmeyi": "dökülmeyi",
  "kullamcam": "kullanacağım",
  "birakcak": "bırakacak",
  "yaparam": "yaparım",
  "aldigimdan": "aldığımdan",
  "hayalkırıklığına": "hayal kırıklığına",
  "gəlmədi": "gelmedi",
  "kobudlaşdırdı": "sertleştirdi",
  "günə": "güne",
  "qoyd": "koydu",
  "anlamad": "anlamadım",
   "saşında": "saçında",
  "orjinalliği": "orijinalliği",
  "şipariştekileri": "sipariştekileri",
  "üzerinde ki": "üzerindeki",
  "ikisinikarsılaştırana": "ikisini karşılaştırana",
  "ikisinikarsılaştırana": "ikisini karşılaştırana",
  "ibarelere": "ibarelere",
  "gonderiyosunuz": "gönderiyorsunuz",
  "kaldiğimi": "kaldığımı",
  "sacım": "saçım",
  "yaglanır": "yağlanır",
  "sacliyim": "saçlıyım",
  "degistirmemislerse": "değiştirmemişlerse",
  "icerigi": "içeriği",
  "alirim": "alırım",
  "pahaliya": "pahalıya",
  "şampunları": "şampuanları",
  "şampuanı": "şampuan",
  "beğenyorm": "beğeniyorum",
  "kullanacağın": "kullanacağım",
  "egzamali": "egzamalı",
  "senerlerdir": "senelerdir",
  "gerçek ten": "gerçekten",
  "parlatiyor": "parlatıyor",
  "yumsak": "yumuşak",
  "sevıyorum": "seviyorum",
  "ufakta": "ufak da",
  "beyeniyorum": "beğeniyorum",
  "kibi": "gibi",
  "saglikli": "sağlıklı",
  "özellike": "özellikle",
  "idestekleyici": "destekleyici",
  "s ürünü": "ürünü",
 "şampuanı": "şampuan",
  "sampuanlardan": "şampuanlardan",
  "kopurtmeye": "köpürtmeye",
  "değ": "değil",
  "şanpuan": "şampuan",
  "yumuşa": "yumuşaklık",
  "sampuandi": "şampuanıydı",
  "dokulmelerimi": "dökülmelerimi",
  "araklı": "aralıklı",
  "dokülmem": "dökülmem",
  "baglamiyorum": "bağlamıyorum",
  "vaz geçemediyim": "vazgeçemediğim",
  "kendıne": "kendine",
  "getirdı": "getirdi",
  "şampuanı": "şampuan",
  "yukseliyo": "yükseliyor",
  "sampuanlara": "şampuanlara",
  "kopuruyor": "köpürüyor",
  "şanpuan": "şampuan",
  "mis gibi gibi": "mis gibi",
  "dokusunu": "dokusu",
  "dokülmem": "dökülmem",
  "baglamiyorum": "bağlamıyorum",
  "araklı": "aralıklı",
  "sampuandi": "şampuanıydı",
  "dokulmelerimi": "dökülmelerimi",
  "vaz geçemediyim": "vazgeçemediğim",
  "kendıne": "kendine",
  "getirdı": "getirdi",
  "şampuanı": "şampuan",
  "mis gibi gibi": "mis gibi",
  "menunum": "memnunum",
  "saçlarımı": "saçımı",
  "kepeği": "kepek",
  "dökülmeyi": "dökülme",
  "sampuan": "şampuan",
  "bayiliyor": "bayılıyor",
  "tesekkğrler": "teşekkürler",
  "aldiğım": "aldığım",
  "ürunun": "ürünün",
  "memünum": "memnunum",
  "saclarimada": "saçlarıma da",
  "dokulmemi": "dökülmemi",
  "ucmus": "uçmuş",
  "karistiedi": "karıştırdı",
  "istiyorm": "istiyorum",
  "uguna": "uyguna",
  "yıkamadan": "yıkamada",
  "yapismadan": "yapışmadan",
  "nemlenmis": "nemlenmiş",
  "zamaninda": "zamanında",
  "kullanılıyorum": "kullanıyorum",
  "mükemel": "mükemmel",
  "şampuanı": "şampuan",
  "indirmdwyken": "indirimdeyken",
  "migrosta": "migros'ta",
  "cargo": "kargo",
  "kandırmakmayin": "kandırmayın",
  "orginal": "orijinal",
  "bahsedlen": "bahsedilen",
  "spraris": "sipariş",
"doldurulma": "doldurulmuş",
  "kulaklik": "kulaklık",
  "cikmadi": "çıkmadı",
  "ist": "İstanbul",
  "birdene": "bir tane",
  "qoyublar": "koymuşlar",
  "nede": "ne de",
  "gonderilmedi": "gönderilmedi",
  "magza": "mağaza",
  "kullanışli": "kullanışlı",
  "nkrmal": "normal",
  "aldıö": "aldım",
  "kulaniyo": "kullanıyor",
  "paketlenmemisdi": "paketlenmemişti",
  "clear ler": "clear'ler",
   "cıkmadı": "çıkmadı",
  "yakisikli": "yakışıklı",
  "iyiygeliuor": "iyi geliyor",
  "paketlemee": "paketleme",
  "uyugun": "uygun",
  "üzenle": "özenle",
  "teesekkurler": "teşekkürler",
"dükülmeye": "dökülmeye",
  "erkolar": "erkekler",
  "çiçegi": "çiçeği",
  "çok iyiye": "çok iyi",
  "gire": "göre",
  "içim": "için",
  "harikalarr": "harikalar",
  "begeniriz": "beğeniriz",
  "severeek": "severek",
  "stok yapiyoruz": "stok yapıyoruz",
  "olta": "oldukça",
  "kalitesininbaya": "kalitesini bayağı",
  "düsünen": "düşünen",
  "arkadaslar": "arkadaşlar",
  "posetle": "poşetle",
  "sakızlarn": "sakızların",
  "geldi": "verdi",
  "garcekten": "gerçekten",
  "gecti": "geçti",
  "kullanima": "kullanıma",
  "iyidi": "iyiydi",
  "kuknıgıruz": "kullanıyoruz",
  "kremıne": "kremine",
  "məhsuldur": "üründür",
  "super": "süper",
  "beyendim": "beğendim",
  "yaxşı": "iyi",
  "stokyapilmalik": "stok yapılmalık",
  "menün": "memnun",
  "sorumsuz": "sorunsuz",
  "hızlıydi": "hızlıydı",
"ürüni": "ürünü",
  "bende": "ben de",
  "geldı": "geldi",
  "orj": "orijinal",
  "dısarıya": "dışarıya",
  "içim": "için",
  "efsaneyaa": "efsaneye",
  "3in 1": "3 in 1",
  "yorumun": "yorumumun",
  "yotumun": "yorumumun",
  "yaptığlnız": "yaptığınız",
  "öyleye": "öylece",
  "gönderildiği": "gönderilmesi",
  "kafamiz": "kafamız",
 "hatadı": "hataydı",
  "stil": "site",
  "menthol lü": "mentollü",
  "on gönderiyorlar": "onu gönderiyorlar",
  "mml": "ml",
  "şişeşine": "şişesine",
  "kalitesiz sahte": "kalitesiz, sahte",
  "güzewl": "güzel",
  "kokuyprdu": "kokuyordu",
  "güsel": "güzel",
  "teşkkürler": "teşekkürler",
  "teseekkurler": "teşekkürler",
  " oğullarına": "oğullarıma",
  "oan": "ona",
  "capcanlı": "capcanlı",
  "felan": "falan",
  "güzək": "güzel",
  "udi": "idi",
  "tewekue": "teşekkür",
   "avantaj li": "avantajlı",
  "ela": "harika",
  "kulanıyor": "kullanıyor",
  "yapilmiş": "yapılmış",
  "teşekkür ler": "teşekkürler",
  "swvdim": "sevdim",
  "yaw": " ",
  "saçımı": "saçıma",
  "indirim de": "indirimde",
  "ulaşıyo": "ulaşıyor",
  "patpatli": "patpatlı",
  "goseldekiyle": "görseldekiyle",
  "gonderdiginiz": "gönderdiğiniz",
  "mis gibi gibi": "mis gibi",
  "düşsede": "düşse de",
  "paketlemesorunsuz": "paketleme sorunsuz",
  "gelsi": "geldi",
  "okdar": "o kadar",
  "teşekkürler ederim": "teşekkür ederim",
  "sagolsunlar": "sağ olsunlar",
"tasfiye": "tavsiye",
  "hıźlı": "hızlı",
  "iyi paketlemiş": "iyi paketlenmiş",
  "vermisdim": "vermiştim",
  "coxx": "çok",
  "paketleyib": "paketleyip",
  "yollamislar": "yollamışlar",
  "qoxusu": "kokusu",
  "qapagi": "kapağı",
  "bagli": "bağlı",
  "gelir": "geliyor",
  "kulanışlı": "kullanışlı",
  "dusunmeden": "düşünmeden",
  "ala bilirsiniz": "alabilirsiniz",
  "soylerim": "söylerim",
  "kullana bilir": "kullanabilir",
  "saglar": "sağlar",
 "fakkat": "fakat",
  "40 ı": "40’ı",
  "değils": "değilsiniz",
  "orjinali": "orijinali",
  "çözemedim": "çözemedim",
  "yaaww": " ",
  "şanpuanlar": "şampuanlar",
  "dağılmışdı": "dağılmıştı",
  "yokdu": "yoktu",
  "süne": "sine",
  "hüzel": "güzel",
  "ilken": "iken",
  "urum": "ürün",
  "şampuanı": "şampuan",
  "kuaförüyüm": "kuaförüm",
  "kardeşimn": "kardeşimin",
  "kepeke": "kepeğe",
  "gelyr": "geliyor",
  "ambalajlanmasi": "ambalajlanması",
  "degıller": "değiller",
  "duj": "duş",
  "kacrmak": "kaçırmak",
  "kacıncı": "kaçıncı",
  "alısım": "alışım",
  "bılmıyorum": "bilmiyorum",
  "gordıkce": "gördükçe",
  "1010": "10/10",
  "bəyəndi": "beğendi",
  "beyəndim": "beğendim",
  "karqo": "kargo",
  "əlimə": "elime",
  "çatdırıldı": "ulaştırıldı",
  "tökülməninin": "dökülmesinin",
  "qarşısını": "karşısını",
  "yemə": "yine",
  "allaxam": "alacağım",
  "teşekkurlər": "teşekkürler",
  "paketleleme": "paketleme",
  "stolk": "stok",
  "paketlemsi": "paketlemesi",
  "yollamissiniz": "yollamışsınız",
  "vaz geçilmezi": "vazgeçilmezi",
   "abım": "abim",
  "eksikaiz": "eksiksiz",
  "wampuanlar": "şampuanlar",
  "teklilere": "teklilerle",
  "diyete": "fiyata",
  "verdi": "geldi",
  "vazgeçilmezij": "vazgeçilmezi",
  "ön numara": "on numara",
  "şampo": "şampuan",
  "devam lı": "devamlı",
  "gercktn": "gerçekten",
  "kullanyrm": "kullanıyorum",
  "işal9": "inşallah",
  "içinde": "için de",
    "stok yapmalıık": "stok yapmalık",
  "kılir": "clear",
  "degistirmedik": "değiştirmedik",
  "claer": "clear",
  "kargolsma": "kargolama",
  "şeklide": "şekilde",
  "sampuann": "şampuan",
  "verdi": "geldi",
  "köpeğe": "kepeğe",
  "göründüğü": "göründüğü gibi",
  "paketlenmişt": "paketlenmişti",
  "korga": "kargo",
  "dısardan": "dışardan",
  "yağlanmayi": "yağlanmayı",
  "indirim deyken": "indirimdeyken",
  "gibi gibi": "gibi",
  "kullandığım": "kullandığı",
   "indırmdeydı": "indirimdeydi",
  "verdi": "geldi",
  "vsyr": "vs. (vesaire)",
  "gonül": "gönül",
  "şampuandı": "şampuandı",
  "harkia": "harika",
  "guzeldii": "güzeldi",
  "verdi": "geldi",
  "uykuna": "uygun",
  "yiksam": "yıkasam",
  "aksamina": "akşamına",
  "saclarimla": "saçlarımla",
  "gune": "güne",
  "gecebiliyorum": "geçebiliyorum",
  "dokulme": "dökülme",
  "catdirilmani": "çatdırılmasını",
  "acin": "açın",
  "mennun": "memnun",
  "yeteeli": "yeterli",
  "teşekkürlerr": "teşekkürler",
   "verdi": "geldi",
  "sapasağlam": "sapasağlam",
  "her dəfə": "her defa",
  "sifarisim": "siparişim",
  "sampuanlarim": "şampuanlarım",
  "verdi özenle": "geldi, özenle",
  "sroku keçmiş": "son kullanma tarihi geçmiş",
  "vaziyette": "durumda",
  "serçe parmağı derecede": "serçe parmağı kadar",
  "sanıyodum": "sanıyordum",
  "verdi ama": "geldi ama",
  "olcak ti": "olacaktı",
  "gonderildi": "gönderildi",
  "razi qaldim": "razı kaldım",
   "verdi": "geldi",
  "fiyatt": "fiyat",
  "ogluma": "oğluma",
  "sadace": "sadece",
  "korgoyu": "kargoyu",
  "gevenle": "güvenle",
  "verdi": "geldi",
  "kullandigi": "kullandığı",
  "verdi": "geldi",
  "paketlemeside": "paketlemesi de",
  "guz": "güzel",
  "urunm": "ürünüm",
  "verdi elime": "geldi, elime",
  "verdi hediyeniz": "geldi, hediyeniz",
  "içnde": "için de",
  "stoklanir": "stoklanır",
  "çabucanak": "çabucak",
  "özelne": "özenle",
  "hazirlanmis": "hazırlanmış",
  "ogluma": "oğluma",
  "hemdde": "hem de",
  "ssekilde": "şekilde",
  "ürunlerim": "ürünlerim",
  "verdi": "geldi",
  "begendiler": "beğendiler",
  "köpek": "kepek",
  "ürünün": "ürünü",
"oglumun": "oğlumun",
  "davamlı": "devamlı",
  "istifade etdiyi": "kullandığı",
  "hediyyəniz": "hediyeniz",
  "təşəkkür edəriz": "teşekkür ederiz",
  "i sevmiyor": "istemiyor",
  "hediyye": "hediye",
  "çon": "çok",
  "bıle": "bile",
  "hediyenize": "hediyeniz için",
  "teşekkürleer": "teşekkürler",
  "yapmalsnz": "yapmalısınız",
   "geldiginde": "geldiğinde",
  "şampuanı": "şampuan",
  "verdi": "geldi",
  "kullnadigimiz": "kullandığımız",
  "yakalyınca": "yakalayınca",
  "alis": "alış",
  "karim": "kârım",
  "düşünce": "düşünce",
  "yapa": "yapar",
  "yapdim": "yaptım",
  "hediyel": "hediye",
   "arkdsima": "arkadaşıma",
  "iui": "iyi",
  "verdi": "geldi",
  "gayrette": "gayet",
  "kullaniyo": "kullanıyor",
  "inridrimde": "indirimde",
  "eken": "iken",
  "walla": "valla",
  "öncede": "önce de",
  "inanin": "inanın",
  "aciyorum": "açıyorum",
  "kazanclar": "kazançlar",
  "kurulma": "kuruma",
  "verdi": "geldi",
  "şampuanı": "şampuan",
  "indrimden": "indirimden",
  "guzeller": "güzeller",
  "paketlenmiş ti": "paketlenmişti",
  "bur": "bir",
  "felde": "geldi",
  "verdi": "geldi",
   "hadiya ": "hediye",
   "üçün": "için",
  "ken": "iken",
  "kullıyor": "kullanıyor",
  "indirmli": "indirimli",
  "kullandigimiz": "kullandığımız",
  "kullandıgm": "kullandığım",
  "bayağı": "baya",
  "vericem": "vereceğim",
  "gyzel": "güzel",
  "paketlemesı": "paketlemesi",
  "mennum": "memnunum",
  "paletleme": "paketleme",
  "erkek kardeslerime": "erkek kardeşlerime",
  "ulaşdı": "ulaştı",
  "keçmiş": "geçmiş",
  "resimlerdede": "resimlerde de",
  "gazzedeki": "gazze’deki",
  "ulaşıp saçınızı diri yakıyorum": "ulaşmak istiyorum, saçım döküldü",
  "yumsacik": "yumuşacık",
  "oyğun": "uygun",
  "adet aldım": "adet aldım,",
  "kullandigi": "kullandığı",
  "3ünün": "3'ünün",
"kullandıgı": "kullandığı",
  "temızlık": "temizlik",
  "dayamadim": "dayanamadım",
   "görirmi": "gelir mi",
   "satislar": "satışlar",
   "daha uygun'a": "daha uyguna",
  "aldırma atıcıya": "aldım satıcıya",
  "tşkr": "teşekkür",
  "korunmus": "korunmuş",
  "yakalayinca": "yakalayınca",
  "artsin": "artsın",
  "yinr": "yine",
  "yapdım": "yaptım",
  "sızde": "siz de",
  "yapmştm": "yapmıştım",
  "baketleme": "paketleme",
  "buldukca": "buldukça",
  "fiyatti": "fiyattı",
  "alican": "alacağım",
  "gerekten": "gerçekten",
  "dusmustu": "düşmüştü",
  "yaptik": "yaptık",
  "ke": "ve",
  "fiata": "fiyata",
  "heralde": "herhalde",
  "şampuanlarım": "şampuanlar",
  "dokulmemisti": "dökülmemişti",
  "elim": "eşim",
  "kampayalar": "kampanyalar",
  "streçli": "streçli",
  "üstlubu": "üslubu",
  "yanlş": "yanlış",
  "olmadğı": "olmadığı",
  "oglama": "oğluma",
   "basarili": "başarılı",
  "oyuzden": "o yüzden",
  "kullndgm": "kullandığım",
  "topluiyi": "toplu iyi",
  "aliyrum": "alıyorum",
  "beyedik": "beğendik",
  "şaşmıcam": "şaşmayacağım",
  "stoklıcam": "stoklayacağım",
  "giti": "gitti",
  "laidm": "aldım",
  "kulandığımız": "kullandığımız",
  "indrimden": "indirimden",
  "uygunst": "uygundu",
  "verdi": "geldi",
  "icim": "için",
   "güzəldi": "güzeldi",
  "beğenmedm": "beğenmedim",
  "olsn": "olsun",
  "memnn": "memnun",
  "eləbil": "öyle bil",
  "üstündən": "üzerinden",
  "işlədiblər": "işlem yapılmış",
  "barı": "bari",
  "gıram": "gram",
  "kulanmıycam": "kullanmayacağım",
  "verdi": "geldi",
  "kullandığım": "kullandığım",
  "özlüsunu": "özlüsünü",
  "kullanamadim": "kullanamadım",
  "bosaltmislar": "boşaltmışlar",
  "normeldede": "normalde de",
  "dım": "aldım",
  "gozukuyor": "gözüküyor",
  "pazalı": "pahalı",
  "paketlemsi": "paketlemesi",
  "verdi şampuanı yatırımı": "şampuan yatırımını",
  "şampuanı": "şampuan",
  "verdi 3 günde verdi": "geldi, 3 günde ulaştı",
  "a bayılıyor": "aşırı bayılıyor",
  "yumusama": "yumuşama",
  "cok": "çok",
  "kepegını": "kepeğini",
  "fiyat da": "fiyata",
  "baskasiyla": "başkasıyla",
  "yağlanmayi": "yağlanmayı",
  "şampuanlarim": "şampuanlarım",
  "kullanamiyorum": "kullanamıyorum",
  "kullanıtorum": "kullanıyorum",
  "verdi": "geldi",
  "brğendik": "beğendik",
  "mehsul": "ürün",
  "catdirildi": "ulaştırıldı",
  "qiymetleri": "fiyatları",
  "serfeli": "uygun",
  "istifadede": "kullanımda",
  "azerbaycana": "azerbaycan’a","yumuşacıkk": "yumuşacık",
  "geld": "geldi",
  "karışıtı": "karşıtı",
  "tekrae": "tekrar",
  "göstermeye bilir": "göstermeyebilir",
  "kaldigim": "kaldığım",
  "tatlısı": "tatlı",
  "egzama iyi": "egzamaya iyi",
  "clearin": "Clear’ın",
  "ürünunü": "ürününü",
  "azaltiyor": "azaltıyor",
  "chok": "çok",
  "sachlarimi": "saçlarımı",
  "yumushacik": "yumuşacık",
    "cozdu": "çözdü",
  "yaglanmayida": "yağlanmayı da",
  "verdi": "geldi",
  "vaz geçmediğim": "vazgeçmediğim",
  "göstermeye bilir": "göstermeyebilir",
  "Clearın": "Clear'ın",
  "ürününu": "ürününü",
  "cabuk": "çabuk",
  "fiyatindan": "fiyatından",
  "akma sizma": "akma sızma",
  "guzelce": "güzelce",
   "kalitali": "kaliteli",
  "biyasada": "piyasada",
  "şampuanı": "şampuan",
  "kullanicam": "kullanacağım",
  "batliydi": "batılydı",
  "kepepe": "kepeğe",
  "verdi": "geldi",
  "verdi.": "geldi.",
  "verdi!": "geldi!",
  "verdi,": "geldi,",
  "dökülmemisti": "dökülmemişti",
  "verdi orijinal": "geldi, orijinal",
  "verdi yanındaki": "geldi, yanındaki",
  "teşekkür ler": "teşekkürler",
  "aeverek": "severek",
  "kullanıypruz": "kullanıyoruz",
  "orjinal": "orijinal",
  "doneminde": "döneminde",
  "orginaldır": "orijinaldir",
  "su kimi": "su gibi",
  "qeweydi": "güzeldi",
  "anca": "ancak",
  "mehsulların": "ürünlerin",
  "kotuydu": "kötüydü",
  "mnk": " ",
  "gönderseydinz": "gönderseydiniz",
  "krili": "kirli",
  "teşşekürler": "teşekkürler",
  "bekliyicektim": "bekleyecektim",
  "dalgın": "dalgalı",
"bilidk": "bildik",
  "indimleri": "indirimleri",
  "kutumu": "kutu mu",
  "markası": "marka",
  "kullandiğimiz": "kullandığımız",
  "kiz": "kız",
  "lil yox": "değil yok",
  "shampuan": "şampuan",
  "tirend yoldan": "trendyol’dan",
  "bırı": "biri",
  "yılardır": "yıllardır",
  "tapmayı": "toplamayı",
   "farketmedim": "fark etmedim",
    "bir dahikine": "bir dahakine",
     "fiya": "fiyat",
  "fiyatindan": "fiyatından",
  "gönül rahatligiyla": "gönül rahatlığıyla",
  "oneriririm": "öneririm",
  "teşekkür ler": "teşekkürler",
  "fıyatına": "fiyatına",
  "kaçirmadım": "kaçırmadım",
  "şampuanıma": "şampuanıma",
  "görecegimi": "göreceğimi",
  "super": "süper",
 "oz3nliydi": "özenliydi",
  "tessekur": "teşekkür",
  "onemlisi": "önemlisi",
  "yaxşıdır": "iyidir",
  "hamısı": "hepsi",
  "fıratı": "fiyatı",
  "ugundu": "uygundu",
  "ambalajlanmis": "ambalajlanmış",
  "tekkurler": "teşekkürler",
  "verdi": "geldi",
  "herşry": "her şey",
  "sikinti": "sıkıntı",
 "iki kayı": "iki katı",
  "verdi": "geldi",
  "sampunlar": "şampuanlar",
  "yarimcigdi": "yarımcıydı",
  "qeder": "kadar",
  "doldurublar": "doldurmuşlar",
  "bir sözlə": "bir sözle",
  "olunmamışdı": "olmamıştı",
  "sacida": "saçı da",
  "kargolanmidti": "kargolanmıştı",
  "kullanfigim": "kullandığım",
  "indiri deyken": "indirimdeyken",
  "yumusac i k": "yumuşacık",
  "mavı": "mavi",
  "mentellosunu": "mentollüsünü",
  "bırde": "bir de",
  "bn": "ben",
  "yapacagım": "yapacağım",
  "stoklamanizi": "stoklamanızı",
  "verdi": "geldi",
  "vazgeçmezim": "vazgeçilmezim",
  "şampuanlar": "şampuan",
  "mis gibi gibi": "mis gibi",
  "yapilmis": "yapılmış",
  "şapmuan": "şampuan",
  "qizim": "kızım",
  "kepekden": "kepekten",
  "eziyyet cekirdi": "eziyet çekiyordu",
  "istifadeden": "kullanımdan",
  "ferq": "fark",
  "şanpunu": "şampuanı",
  "gercektan": "gerçekten",
  "yılardır": "yıllardır",
  "hediyeler için teşekkürler": "hediyeler için teşekkür ederim",
  "yapyom": "yapıyorum",
  "ederm": "ederim",
  "o kadaarr": "o kadar",
  "yumusaciq etdi": "yumuşacık etti",
  "şekılde": "şekilde",
  "hedıyesınden": "hediyesinden",
  "var ol senn": "var ol sen",
  "razi qaldim": "razı kaldım",
  "hemende": "hemen de",
  "kulandim": "kullandım",
  "gosteririr": "gösterir",
  "ipək": "ipek",
  "kimi etdi": "gibi yaptı",
  "möhtəşəmdir": "müthiştir",
  "gün asira": "gün aşırı",
  "yuduğum": "yıkadığım",
  "3gun": "3 gün",
  "yaglanmadi": "yağlanmadı",
  "sizdiniz": "sizinle devam",
  "altım": "aldım",
  "verdi": "geldi",
  "indirim de": "indirimde",
  "paketeme": "paketleme",
  "ulaşdi": "ulaştı",
  "ulaştıgı": "ulaştığı",
  "yurdışından": "yurtdışından",
  "kaldik": "kaldık",
  "saçlarımada": "saçlarıma da",
  "yapıcam": "yapacağım",
  "verdi": "geldi",
  "bundanda": "bundan da",
  "aradigim": "aradığım",
  "pemp": "pembe",
  "olani": "olanı",
  "cicegiydi": "çiçeğiydi",
  "guzeel": "güzel",
  "stoklayalim": "stoklayalım",
  "çak": "çok",
  "aldımm": "aldım",
  "şampuanı": "şampuan",
  "yiğenime": "yeğenime",
  "gonderiyorlar": "gönderiyorlar",
  "çokarttım": "çıkarttım",
  "nerdeyse": "neredeyse",
  "gün ışında": "gün ışığında",
  "geçik": "gecikmiş",
  "çıkarıyosunuz": "çıkarıyorsunuz",
  "bekledigin": "beklediğin",
  "performansi": "performansı",
  "pakatlemesi": "paketlemesi",
  "denicem": "deneyeceğim",
 "saticaya": "satıcıya",
 "güzek": "güzel",
  "yunuşacık": "yumuşacık",
  "hepnkullandiğım": "hep kullandığım",
  "beyendim": "beğendim",
  "hediyyeler": "hediyeler",
  "teşşük": "teşekkür",
  "bayag": "bayağı",
  "kepeyim": "kepeğim",
  "fazlaydi": "fazlaydı",
  "paketlənmiş": "paketlenmiş",
  "tewekurler": "teşekkürler",
  "sızmamisti": "sızmamıştı",
  "kullanabılıyorum": "kullanabiliyorum",
  "güzek": "güzel",
  "yunuşacık": "yumuşacık",
  "hepnkullandiğım": "hep kullandığım",
  "beyendim": "beğendim",
  "hediyyeler": "hediyeler",
  "teşşük": "teşekkür",
  "bayag": "bayağı",
  "kepeyim": "kepeğim",
  "fazlaydi": "fazlaydı",
  "paketlənmiş": "paketlenmiş",
  "tewekurler": "teşekkürler",
  "sızmamisti": "sızmamıştı",
  "kullanabılıyorum": "kullanabiliyorum",
  "memnunumm": "memnunum",
  "elimede": "elime de",
  "şampuanı": "şampuan",
  "sapasağlam": "sapasağlam",
  "üçüde": "üçü de",
  "clerın": "Clear'ın",
  "küçükte": "küçük de",
  "şampuanı larini": "şampuanlarını",
  "şampuani": "şampuanı",
  "bildigim": "bildiğim",
  "geliiyor": "geliyor",
  "kullanip": "kullanıp",
  "mn": " ",
  "peketleme": "paketleme",
  "tşkur": "teşekkür",
  "ulastı": "ulaştı",
  "gediler": "geldiler",
  "yetiş ti": "yetişti",
  "hedyeniz": "hediyeniz",
  "haarikka": "harika",
  "eşimede": "eşime de",
  "köpekleri": "kepekleri",
  "ala bilirsiniz": "alabilirsiniz",
  "verdi": "geldi",
  "kndi": "kendi",
  "sampuanini": "şampuanını",
  "kaşıntim": "kaşıntım",
  "gösterio": "gösteriyor",
  "deng verdi": "denk geldi",
  "kuklanıcam": "kullanacağım",
  "şampuanlar yarın verdi": "şampuanlar yarın geldi",
  "demekli": "demek ki",
  "uretiliyorsa": "üretiliyorsa",
  "qeder": "kadar",
  "doldurulub": "doldurulmuş",
  "dı": "di",
  "heyifki": "ne yazık ki",
  "bura": "burada",
  "paylasmag olmur": "paylaşmak mümkün olmuyor",
  "nice": "güzel",
  "iyiye verdi": "iyi geldi",
  "gozeldi": "güzeldi",
   "onkadar": "o kadar",
  "yapsam": "yapabilsem",
  "stok yapcam": "stok yapacağım",
  "fiyatıda": "fiyatı da",
  "sevdıgım": "sevdiğim",
  "stokladm": "stokladım",
  "bayildi": "bayıldı",
  "eczabkım": "eczacım",
  "kullanıyom": "kullanıyorum",
  "harira": "harika",
  "helâl": "helal",
  "stokladıpım": "stokladığım",
  "ala bilirsiniz": "alabilirsiniz",
  "kndi": "kendi",
  "saclarimda": "saçlarımda",
  "sampuanini": "şampuanını",
  "kaşıntim": "kaşıntım",
  "gösterio": "gösteriyor",
  "deng verdi": "denk geldi",
  "kuklanıcam": "kullanacağım",
  "demekli": "demek ki",
  "uretiliyorsa": "üretiliyorsa",
  "boy da": "boyda",
  "şampuanı var": "şampuan var",
  "heyifki": "ne yazık ki",
  "bura": "burada",
  "video paylasmag olmur": "video paylaşmak mümkün olmuyor",
  "demek olarki": "demek oluyor ki",
  "qeder": "kadar",
  "doldurulub": "doldurulmuş",
  "etmisdim": "etmiştim",
  "hec": "hiç",
  "gormeden": "görmeden",
  "uçun": "için",
  "alsaydim": "alsaydım",
   "kargolanmis": "kargolanmış",
  "şampiyon": "şampuan",
  "saclarım": "saçlarım",
  "güüzel": "güzel",
  "iyidi": "iyiydi",
  "düşdük": "düştükçe",
  "yupyumuşaq": "yumuşacık",
  "yumşamlkl": "yumuşaklık",
  "şimdş": "şimdi",
  "ıcuza": "ucuza",
  "ceyizime": "çeyizime",
  "geldi geldi": "geldi",
  "çokda": "çok da",
  "teşejjür": "teşekkür",
  "saçımıda": "saçımı da",
  "tşklersalam": "teşekkürler",
  "güzel ler": "güzeller",
  "saclarimı": "saçlarımı",
  "verdi": "geldi",
  "gözk": "çok",
  "sanponlar": "şampuanlar",
  "patdayib": "patlayıb",
  "yaxilib": "yakşı olub",
  "bir birine": "birbirine",
  "pis paketleyibler": "kötü paketlemişler",

"verdi": "geldi",
  "internetten": "internet üzerinden",
  "şampuanı akmış verdi": "şampuan akmıştı, geldi",
  "mehsularim": "ürünlerim",
  "yoxdu": "yoktu",
  "buu": "bu",
  "yollasydnz": "yollasaydınız",
  "insalah": "inşallah",
  "zamanlar da": "zamanlarda",
  "un kokusu": "ürün kokusu",
  "edılebılır": "edilebilir",
  "geç verdi": "geç geldi",
  "verdi": "geldi",
  "trentyola": "trendyolaa",
  "iyikidr": "iyiki de",
  "belə": "bile",
  "və": "ve",
  "xətası": "hatası",
  "hepnizə": "hepinize",
  "konjlmamışdı": "konulmamıştı",
  "yaglandirmiyor": "yağlandırmıyor",
  "sağliyor": "sağlıyor",
  "dovenin": "Dove’un",
  "şampuanlarıni": "şampuanlarını",
  "yaninda": "yanında",
  "yapildi": "yapıldı",
  "cokgüxel": "çok güzel",
  "takiptyim": "takipteyim",
  "indirme": "indirim",
  "yumşaq": "yumuşak",
  "görüntüyə": "görüntüye",
  "sahib": "sahip",
  "kaldm": "kaldım",
  "bayildik": "bayıldık",
  "alıyorum": "alıyorum",
  "edrım": "ederim",
  "şampuanlarin": "şampuanların",
  "uurn": "ürün",
  "dgl": "değil",
  "alabilirisniz": "alabilirsiniz",
  "oldigu": "olduğu",
  "zamanfiyati": "zaman fiyatı",
  "alabilirsinzi": "alabilirsiniz",
  "gelecegini": "geleceğini",
  "demədim": "demedim",
  "cesiti": "çeşidi",
  "saclarımı": "saçlarımı",
  "dolgyn": "dolgun",
  "pametleme": "paketleme",
  "gozül": "güzel",
  "almanizi": "almanızı",
  "tovsüye": "tavsiye",
  "saglıklı": "sağlıklı",
  "yaxşi": "iyi",
  "məmnun": "memnun",
  "hamsi": "hepsi",
  "qat": "daha",
 "onceden": "önceden",
  "aldim": "aldım",
  "şok": "çok",
  "süreklü": "sürekli",
  "sapasağlam": "sapasağlam",
  "lee": "ile",
  "kullaniyruz": "kullanıyoruz",
  "kalacaginiz": "kalacağınız",
  "birdə": "bir de",
  "hədiyyələrin": "hediyelerin",
  "elədi": "oldu",
  "sərf": "kârlı",
  "etiyor": "ediyor",
  "almaq": "almak",
  "nəzərinizdə": "düşüncenizde",
  "olsun": "olsun",
  "yapsinlar": "yapsınlar",
  "cunku": "çünkü",
  "henz": "henüz",
  "kullnmaya": "kullanmaya",
  "baslamadm": "başlamadım",
  "yapdi": "yaptı",
  "güsell": "güzel",
  "sorunuz": "sorunsuz",
  "ellime": "elime",
  "guncellemem": "güncellemem",
  "ortulu": "örtülü",
  "buneyaa": "bu ne ya",
  "karqom": "kargom",
  "sampunum": "şampuanım",
  "qiriq": "kırık",
  "aqmis": "akmış",
  "aqzi": "ağzı",
  "qiymuslar": "koymuşlar",
  "yumushatdi": "yumuşattı",
  "sachimi": "saçımı",
  "kurutdukdan": "kuruttuktan",
  "sachlara": "saçlara",
  "yaglaniyorsa": "yağlanıyorsa",
  "şu ama": "şu ana",
  "vav": " ",
  "köpürüyo": "köpürüyor",
  "hosuma": "hoşuma",
  "istiyim": "isteyeyim",
  "kabardı": "kabarıyor",
  "kabarmasini": "kabarmasını",
  "etkisini": "etkisini",
  "sacimla": "saçımla",
  "basa cikmayi": "başa çıkmayı",
  "şampaunla": "şampuanla",
  "yikadiktan": "yıkadıktan",
  "saclarimin": "saçlarımın",
  "sagladi": "sağladı",
  "saclarim": "saçlarım",
  "acikcasi": "açıkçası",
  "gorunuyor": "görünüyor",
  "bakim": "bakım",
  "kullanimdan": "kullanımdan",
  "gormeye": "görmeye",
  "gorundu": "göründü",
  "çitiş": "şişip",
  "gosteriyo": "gösteriyor",
  "sampuanım": "şampuanım",
  "yıpranmıs": "yıpranmış",
  "tı": "tı.",
  "birkac": "birkaç",
  "sampuanda": "şampuanda",
  "arsında": "arasında",
  "görunmesini": "görünmesini",
  "verdiki": "verdi ki",
  "ettin": "ettim",
  "şapuandan": "şampuandan",
  "hyqluronic": "hyaluronic",
  "yumasak": "yumuşak",
  "sekil": "şekil",
  "kopür": "köpür",
  "gibi gibi": "gibi",
  "kalin": "kalın",
  "yag": "yağ",
  "sagladi": "sağladı",
  "sl": "SLS",
  "sampuanimi": "şampuanımı",
  "kullanıyodum": "kullanıyordum",
  "mahvetmişti": "mahvetmişti.",
  "kivirciklar": "kıvırcıklar",
  "belirginlestiriyor": "belirginleştiriyor",
  "deneyib": "deneyip",
  "deyişiklik": "değişiklik",
  "oldğunu": "olduğunu",
  "gosteriyo": "gösteriyor",
  "sampuanım": "şampuanım",
  "yıpranmıs": "yıpranmış",
  "birkac": "birkaç",
  "sampuanda": "şampuanda",
  "arsında": "arasında",
  "görunmesini": "görünmesini",
  "verdiki": "verdi ki",
  "ettin": "ettim",
  "şapuandan": "şampuandan",
  "hyqluronic": "hyaluronic",
  "yumasak": "yumuşak",
  "sekil": "şekil",
  "kopür": "köpür",
  "kalin": "kalın",
  "yag": "yağ",
  "sagladi": "sağladı",
  "sl": "SLS",
  "sampuanimi": "şampuanımı",
  "kullanıyodum": "kullanıyordum",
  "mahvetmişti": "mahvetmişti.",
  "kivirciklar": "kıvırcıklar",
  "belirginlestiriyor": "belirginleştiriyor",
  "deneyib": "deneyip",
  "deyişiklik": "değişiklik",
  "oldğunu": "olduğunu",
  "saclarımı": "saçlarımı",
  "iceriğinde": "içeriğinde",
  "şampuanı": "şampuan",
  "muh te şem": "muhteşem",
  "hyqluronic": "hyaluronic",
  "kullanöadım": "kullanmadım",
  "kargo sırketıne": "kargo şirketine",
  "agirlastirmiyor": "ağırlaştırmıyor",
  "yaglaniyor": "yağlanıyor",
  "rofleli": "röfleli",
  "ındirim": "indirim",
  "yıpratmistim": "yıpratmıştım",
  "acilmiyodu": "açılmıyordu",
  "dökülüyodu": "dökülüyordu",
  "geliyor gelmez": "gelir gelmez",
  "saçımı olan": "saçı olan",
  "değiştirmeyeceim": "değiştirmeyeceğim",
  "verdi": "geldi",
  "verdi muhteşem": "geldi, muhteşem",
  "kullanmaya": "kullanmaya",
  "saçlarımda": "saçlarımda",
   "sayesınde": "sayesinde",
  "çol": "çok",
  "verdi": "geldi",
  "kullanildiginda": "kullanıldığında",
  "almasını": "açılmasını",
  "yıprandi": "yıprandı",
  "dolu": "dolayı",
  "saçıma": "saçımı",
  "kullanırim": "kullanırım",
  "e daha ne": "daha ne",
  "kepeĝi": "kepeği",
  "li": "li.",
  "sulfatsız": "sülfatsız",
  "yikamada": "yıkamada",
  "canliligini": "canlılığını",
  "eyerki": "eğer ki",
  "a sahipseniz": "a sahipseniz",
  "ten": "yüzünden",
  "exzamaya": "egzamaya",
   "saçlarımi": "saçlarımı",
  "kullanimimdan": "kullanımımdan",
  "cansiz": "cansız",
  "gorunen": "görünen",
  "durulaniyor": "durulanıyor",
  "saçlarımda ki": "saçlarımdaki",
  "stoklayacağı": "stoklayacağım",
  "yapışmadan": "yapılmadan",
  "dove un": "dove’un",
  "şampunun": "şampuanın",
  "üzərində": "üzerinde",
  "tərkibində": "içeriğinde",
  "olmağı": "olduğu",
  "normnal": "normal",
  "guzel": "güzel",
  "yaglandirir": "yağlandırır",
  "olmadi": "olmadı",
  "sonrdan": "sonradan",
  "basladi": "başladı",
  "biraktim": "bıraktım",
  "kullanmayi": "kullanmayı",
  "donem": "dönem",
  "sampuanlarini": "şampuanlarını",
  "verdi": "geldi",
  "mukkemmel": "mükemmel",
   "kendi̇siimee": "kendime",
  "aldim": "aldım",
  "harikkaa": "harika",
  "kampanyasi": "kampanyası",
  "aldi": "aldı",
  "fiyatperformans": "fiyat performans",
  "yumaşıcık": "yumuşacık",
  "yapiyorlarki": "yapıyorlar ki",
   "indiirmden": "indirimden",
  "azaltti": "azalttı",
  "alicaktimki": "alacaktım ki",
  "stokluycam": "stoklayacağım",
  "dagilmadan": "dağılmadan",
  "h8ssi": "hissi",
  "yapistirmiyor": "yapıştırmıyor",
  "fiyatıda": "fiyatı da",
  "saclara": "saçlara",
  "kesfetseydim": "keşfetseydim",
  "denebunu": "denebunu'da",
  "yazlik": "yazlık",
  "stok yapacagim": "stok yapacağım",
   "sağlamdi": "sağlamdı",
  "tellidüz": "telli düz",
  "geç verdi": "geç geldi",
  "fiyat da": "fiyata",
  "saşları": "saçları",
  "gibi gibi": "gibi",
  "xoşuma": "hoşuma",
  "olaraq": "olarak",
  "sevdiyim": "sevdiğim",
  "gezel": "güzel",
  "satıcı ya": "satıcıya",
   "kafanizda": "kafanızda",
  "yumusatiyo": "yumuşatıyor",
  "kasıntı": "kaşıntı",
  "temizliyo": "temizliyor",
  "güncelliyeceğim": "güncelleyeceğim",
  "takki": "tahriş",
  "kargox": "kargo",
  "alamaya": "almaya",
  "döküme": "dökülme",
  "yarasiz": "yararsız",
  "veziyete": "vaziyete",
  "dusub": "düşüp",
  "ekseriyeti": "çoğu",
  "kampaynya": "kampanya",
  "üründu": "üründü",
  "etmeyecegim": "etmeyeceğim",
  "r ezil lik": "rezillik",
    "saç tipidi": "saç tipiydi",
  "düşmədi": "düşmedi",
  "doktu": "döktü",
  "sertleltirdi": "sertleştirdi",
  "verdi": "geldi",
  "ebatıda": "ebatı da",
  "internerden": "internetten",
  "bekluyonu z": "bekliyorsunuz",
  "saçmlarmı": "saçlarımı",
  "şampuanlarıni": "şampuanlarını",
  "sevdiyim": "sevdiğim",
   "sapa sağlam": "sapasağlam",
  "təşəkkür edirəm": "teşekkür ederim",
  "sapsağlam": "sapasağlam",
  "hədiyyələr": "hediyeler",
  "də": "de",
  "saçımı": "saçlarımı",
  "gelecegini": "geleceğini",
  "dusunmemistim": "düşünmemiştim",
  "eşlisinden": "öncekinden",
  "şükür": "şükürler olsun",
  "begendik": "beğendik",
  "ceyiz": "çeyiz",
  "mis gibi gibi": "mis gibi",
  "yakaladum": "yakaladım",
  "gostrrir": "gösterir",
  "sampuanda": "şampuanda",
  "buradannalıyoruz": "buradan alıyoruz",
  "əladı": "eladı",
  "tövsiyə": "tavsiye",
  "şampuaann": "şampuanın",
  "deyilim": "değilim",
  "herşeye": "her şeye",
  "koşa": "kısa",
  "tanesını": "tanesini",
  "verdi": "geldi",
  "iyi fiyat da": "iyi fiyata",
  "yapacam": "yapacağım",
  "gzuel": "güzel",
    "verdi": "geldi",
  "eladı": "aldı",
  "edirəm": "ederim",
  "gzuel": "güzel",
  "sapa sağlam": "sapasağlam",
  "uygunduydu": "uygundu",
  "sevdim": "sevdim",
  "kullandımda": "kullandığımda",
  "olcam": "olacağım",
  "düşünuyorum": "düşünüyorum",
  "hayırli": "hayırlı",
  "isler": "işler",
  "inidirümde": "indirimde",
  "paketlemis": "paketlemiş",
  "istedigim": "istediğim",
  "gitmiyo": "gitmiyor",
   "koksuna": "kokusuna",
  "indirmdeyken": "indirimdeyken",
  "alıyorm": "alıyorum",
  "doveun": "dove'un",
  "iyisii": "iyisi",
  "cok": "çok",
  "artmis": "artmış",
  "indirimddn": "indirimden",
  "ettiriyo": "ettiriyor",
   "yumos": "yumoş",
  "kapaklilar": "kapaklılar",
  "üründün": "üründü",
  "saçlarımiza": "saçlarımıza",
  "oluyr": "oluyor",
  "kulanimda": "kullanımda",
  "geliyr": "geliyor",
  "indirimee": "indirme",
  "t ederim": "teşekkür ederim",
  "mükkemeldi": "mükemmeldi",
  "kapmaya": "kampanyaya",
  "hediyeleriz": "hediyeleriniz",
  "3 lü": "3'lü",
  "indirme": "indirim",
  "gibi gibi": "mis gibi",
  "sorunsuzz": "sorunsuz",
  "dov": "dove",
  "alacam": "alacağım",
  "dıyat": "fiyat",
  "trenyol": "trendyol",
  "güxel": "güzel",
    "satıvıya": "satıcıya",
  "hediyedi": "hediye için",
  "tedkkur": "teşekkür",
  "kullanmiyorum": "kullanmıyorum",
  "iyidi": "iyiydi",
  "şampuanom": "şampuanım",
  "cokta": "çok da",
  "fıyatı": "fiyatı",
  "gırdıgınde": "girdiğinde",
  "yapyorum": "yapıyorum",
  "beyenerek": "beğenerek",
  "təşəkkür edirəm": "teşekkür ederim",
  "elimene": "elime",
  "güzeel": "güzel",
  "uniliverden": "unilever'den",
  "replika olabilir": "replika değildir",
  "tşk": "teşekkür",
  "alsin": "alsın",
  "iyiki": "iyi ki",
  "fotograftaki": "fotoğraftaki",
  "patlamıstı": "patlamıştı",
  "sampuanın": "şampuanın",
  "doğu": "dove",
  "pirl": "pırıl",
  "yqpdi": "yaptı",
  "cox": "çok",
  "etdi": "etti",
  "kaliite": "kalite",
  "verdi": "geldi",
  "derimm": "derim",
  "edirəm": "ederim",
  "etmişdim": "etmiştim",
  "paketlərə": "paketlere",
  "gəlib": "gelip",
  "çıxdı": "çıktı",
  "bıttıkce": "bittikçe",
  "kullanıyorumdum": "kullanıyordum",
  "indirimdeyden": "indirimdeyken",
  "iyki": "iyi ki",
 "minareler": "mineraller",
  "sagliklı": "sağlıklı",
  "elimdeykdi": "elimdeydi",
  "hissetim": "hissettim",
  "tellerinini": "tellerini",
  "uzadın": "uzasın",
  "tereddüt etmeden": "tereddüt etmeden",
  "saclarida": "saçları da",
  "endirimə": "indirim",
  "düşdükcə": "düştükçe",
  "stoklayıram": "stokluyorum",
  "elinenize": "elinize",
  "her zamn": "her zaman",
  "uyuyun": "uygun",
  "yumos yumas": "yumuş yumuş",
  "paketelenmiş": "paketlenmiş",
  "ver dir": "verdi",
  "urunlerin": "ürünlerin",
  "oncedenden": "önceden",
  "guzl": "güzel",
  "ılgılı": "ilgili",
  "ler": "ler",
  "sitok": "stok",
  "esi": "işi",
  "d3": "de",
  "sapasağlam": "sapasağlam",
  "kullandığığım": "kullandığım",
  "teşekürler": "teşekkürler",
  "univer": "unilever",
  "məhsulları": "ürünleri",
  "hədiyyə": "hediye",
  "b3n": "ben",
  "hediyelerde": "hediyeler de",
  "alisverisim": "alışverişim",
  "saçlarima": "saçlarıma",
  "gibi gibi": "gibi",
  "diydi": "idi",
  "indirmeyi": "indirimde",
  "hwdiyeniz": "hediyeniz",
  "sattiginiz": "sattığınız",
  "urunede": "ürüne de",
  "parayada": "paraya da",
  "döktü": "döküldü",
  "oketleme": "paketleme",
  "eşimde": "eşim de",
  "iyidi": "iyiydi",
  "teşeklürler": "teşekkürler",
  "yumuşatıyo": "yumuşatıyor",
  "gayat": "gayet",
  "indirim i": "indirim",
  "stiklamaya": "stoklamaya",
  "calisiyorum": "çalışıyorum",
  "bantlanmasi": "bantlanması",
  "pat pata": "patpata",
  "kullanmışdım": "kullanmıştım",
  "yene": "yine",
  "ali if": "alım için",
  "detayıda": "detayı da",
  "indirim li.": "indirimli",
  "aldik": "aldık",
  "iyiki": "iyi ki",
  "ediyor": "etkiliyor",
  "tovsüye": "tavsiye",
  "qızıma": "kızıma",
  "razmer": "beden",
  "ede bilersiz": "edebilirsiniz",
  "yaglanmiyo": "yağlanmıyor",
  "aldugim": "aldığım",
  "çok bebeğinden": "çok beğendim",
  "fiyatda": "fiyatta",
  "kalitel": "kaliteli",
  "beğedim": "beğendim",
  "saçaları": "saçları",
  "yumuşacıq": "yumuşacık",
  "möhteşem": "muhteşem",
  "çatıb": "çatıbı",
  "dəmir": "demir",
  "kullandi": "kullandı",
  "alış veriş": "alışveriş",
  "stoklarim": "stoklarım",
  "begendiik": "beğendik",
  "saclarimiza": "saçlarımıza",
  "uygunada": "uyguna da",
  "yumuşakcık": "yumuşacık",
  "stoklayin": "stoklayın",
  "kalitelii": "kaliteli",
  "şampuanı": "şampuan",
  "aşklar": "arkadaşlar",
    "insalah": "inşallah",
  "sampuanları": "şampuanları",
  "bayildik": "bayıldık",
  "teşekküler": "teşekkürler",
  "sçlarım": "saçlarım",
  "yalanmıyor": "yağlanmıyor",
  "bayıldıgım": "bayıldığım",
  "mis gibi gibi": "mis gibi",
  "ulasmadi": "ulaşmadı",
  "oykot": "boykot",
  "ümidi": "ümitle",
  "yatağı": "yaptığı",
  "hedef ozelikte": "hedef özellikte",
  "saçli": "saçlı",
  "aldırtın": "aldırttın",
  "sıtok": "stok",
  "bıtdıkce": "bittikçe",
  "edırem": "ederim",
  "stoklıyorum": "stokluyorum",
  "ürün ler": "ürünler",
  "kullandığımız": "kullandığımız",
  "pakeylenmişki": "paketlenmiş ki",
   "ütünler": "ürünler",
  "teşekürederim": "teşekkür ederim",
  "ayriyeten": "ayrıyeten",
  "iyiki": "iyi ki",
  "bir birinden": "birbirinden",
  "tesekkurler": "teşekkürler",
  "kazanclariniz": "kazançlarınız",
  "beğenek": "beğenerek",
  "taniki": "tabii ki",
  "terentyoldan": "trendyol’dan",
  "saticidan": "satıcıdan",
  "artiq": "artık",
  "sifarishimdir": "siparişimdir",
  "sifarisimde": "siparişimde",
  "defekinde": "defasında",
  "chox": "çok",
  "kargolanmisdi": "kargolanmıştı",
  "sampunlarin": "şampuanların",
  "gozeldir": "güzeldir",
  "paketləmə": "paketleme",
  "tiretyol": "Trendyol",
  "hediyyede": "hediye de",
  "ipek kibi": "ipek gibi",
  "kullanişli": "kullanışlı",
  "birinkn": "birinin",
  "düşüyo": "düşüyor",
  "superistifade": "süper istifade",
  "yum şacıq": "yumuşacık",
  "elifırcuyum": "elidorcuyum",
  "tesekurler": "teşekkürler",
  "üniliver": "unilever",
  "cok": "çok",
  "endirime": "indirim",
  "gibi gibi": "gibi",
  "herxaman": "her zaman",
  "twkler": "teşekkürler",
  "rahatliyla": "rahatlığıyla",
  "ipek kimi": "ipek gibi",
  "kimi": "gibi",
  "geldiu": "geldi",
  "tewekkur": "teşekkür",
  "şampuanim": "şampuanım",
  "pakette": "paketteki",
  "taranıyo": "taranıyor",
  "aaliyorm": "alıyorum",
  "fşyarı": "fiyatı",
  "uygunada": "uygun fiyata",
  "alin": "alın",
  "fiyatq": "fiyata",
  "sekılde": "şekilde",
  "fiyata": "fiyata",
  "oarlak": "parlak",
  "marca": "marka",
  "seve": "severek",
  "stoklıyorum": "stokluyorum",
  "diyecem": "diyeceğim",
  "şampuanı": "şampuan",
  "kullanıyoz": "kullanıyoruz",
  "firmayıda": "firmayı da",
  "sanpon": "şampuan",
  "qiymeti": "fiyatı",
  "yaxsinolar": "iyi olsunlar",
  "güzelidi": "güzeldi",
  "aldigimzi": "aldığımız",
  "verdi": "geldi",
  "alıb": "alıp",
  "depolayıram": "depoluyorum",
  "iyidi": "iyiydi",
  "indirim deyken": "indirimdeyken",
   "bikaç": "birkaç",
  "memnunum": "memnunum",
  "verdi": "geldi",
  "qiymət": "fiyat",
  "alısveriş": "alışveriş",
  "cünkü": "çünkü",
  "cabucakta": "çabucakta",
  "ggeldi": "geldi",
  "ürümü": "ürünü",
  "kapimdaydi": "kapımdaydı",
  "patpatlı": "pat-patlı",
  "yaptığın": "yaptığın",
  "paletlemesi": "paketlemesi",
  "tş": "teşekkür",
  "birbasa": "birbaşa",
  "verdi": "geldi",
  "trend yol": "trendyol",
  "yakaladim": "yakaladım",
  "mükəmməl": "mükemmel",
  "beyənirəm": "beğeniyorum",
  "denicegim": "deneyeceğim",
 "muthiss": "müthiş",
  "ayreten": "ayrıca",
  "verdi": "geldi",
  "iligli": "ilgili",
  "alıdığım": "aldığım",
  "paketlenmiştim": "paketlenmişti",
  "kalacam": "kalacağım",
  "hatirliycam": "hatırlayacağım",
  "önericem": "önereceğim",
  "siitenizi": "sitenizi",
  "hayrt": "hayır",
  "tarakta": "tarak da",
  "almıs": "almış",
  "kullniyorum": "kullanıyorum",
  "tesekkurelr": "teşekkürler",
   "telekürler": "teşekkürler",
  "geldigimde": "geldiğimde",
  "sagol": "sağ ol",
  "hediue": "hediye",
  "bir kaçtane": "birkaç tane",
  "nsl": "nasıl",
  "miş ha": "miş ha!",
  "memmun": "memnun",
  "paril": "parıl",
  "parliyor": "parlıyor",
  "yipranmisti": "yıpranmıştı",
  "çok oldum": "çok mutlu oldum",
  "viteminleri": "vitaminleri",
  "açılmasını": "alınmasını",
  "2side": "ikisi de",
  "verecegiz": "vereceğiz",
  "başarılıki": "başarılı ki",
  "öncelike": "öncelikle",
  "kopurmesine": "köpürmesine",
  "gibi gibi": "gibi",
  "saclarundaki": "saçlarındaki",
  "gibidevkokuyir": "gibi de kokuyor",
  "elidorr": "elidor",
  "2set": "2 set",
  "kaç sarılmıştı": "kargo sarılmıştı",
  "cabucak": "çabucak",
  "acıldı": "açıldı",
  "düzlestirdim": "düzleştirdim",
  "kadarda": "kadar da",
  "yakalanılıcak": "yakalanabilecek",
  "dökülmeyü": "dökülmeyi",
  "geçemicem": "geçemeyeceğim",
  "özen liydi": "özenliydi",
   "gonul": "gönül",
  "parladıı": "parladı",
  "yumuşacık yapttı": "yumuşacık yaptı",
  "ürünün": "ürün",
  "ünilevere": "unilever'e",
  "diaydi": "idi",
  "fyt": "fiyat",
  "sarınıb": "sarınıp",
  "hediyyeleriniz": "hediyeleriniz",
  "yapdı": "yaptı",
  "yumusavık": "yumuşacık",
  "yapıo": "yapıyor",
  "kullndığım": "kullandığım",
  "yumuş yapıyor": "yumuşacık yapıyor",
  "teeşkkürler": "teşekkürler",
  "kabarikligi": "kabarmalığı",
  "ediyo": "ediyor",
  "uzamasindada": "uzamasında da",
  "təşəkkür edirəm": "teşekkür ederim",
  "məhsul": "ürün",
   "duzlestirmeye": "düzleştirmeye",
  "yardimci": "yardımcı",
  "oarlak": "parlak",
  "îçin": "için",
  "teşekurler": "teşekkürler",
  "paket deyi": "paket deydi",
  "oaketleme": "paketleme",
  "edilmedi": "eklemişler",
  "ndirimde": "indirimde",
  "tülermemi": "kabarıklığımı",
  "kullandığımşampuanı": "kullandığım şampuanı",
  "hediyelerinizçanta": "hediyeleriniz çanta",
  "temizleme pamukları": "temizleme pamukları",
  "verdihe": "verdi",
  "sarınıb": "sarınılmış",
  "oaketleme": "paketleme",
  "hediyelerinşz": "hediyeleriniz",
  "oarlak": "parlak",
  "îçin": "için",
  "teşekurler": "teşekkürler",
  "deyi": "diye",
  "ndirimde": "indirimde",
  "kulkandim": "kullandım",
  "verdihepsi": "verdi, hepsi",
  "kurlu": "kuru",
  "güzeldi": "güzeldi",
  "gibioluyor": "gibi oluyor",
  "saçımıyumusavık": "saçımı yumuşacık",
  "verdiayriyetten": "verdi, ayriyetten",
  "göründüğügibi": "göründüğü gibi",
  "sarılıkolili": "sarılı kolili",
  "bandlı": "bantlı",
  "poşetesarılmıs": "poşete sarılmış",
  "şampuanısaçlarım": "şampuanı, saçlarım",
  "makyajçantası": "makyaj çantası",
  "saçserumu": "saç serumu",
  "stokluyorm": "stokluyorum",
  "olustu": "oluştu",
  "hediyelerinizçanta": "hediyeleriniz çanta",
  "içinde": "için de",
  "verdihe": "verdi",
  "içinde": "için de",
  "tarkakta": "tarakta",
  "verdiginde": "girdiğinde",
  "hediyes": "hediyesi",
  "verdiginde": "girdiğinde",
  "muaazam": "muazzam",
  "gonderdığınız": "gönderdiğiniz",
  "mını": "minik",
  "memnunumkaldık": "memnun kaldık",
  "saticidan": "satıcıdan",
  "alisveris": "alışveriş",
  "kargolamasi": "kargolaması",
  "catlak": "çatlak",
  "varsin": "varsın",
   "bembe": "pembe",
  "elıdorun": "elidorun",
  "stokda": "stokta",
  "korgo": "kargo",
  "kullamıyorum": "kullanmıyorum",
  "kampanya dan": "kampanyadan",
    "ireal": "ideal",
     "yapilmisti": "yapılmıştı",
  "heald": "head",
  "shoulders": "shoulders",
  "kullndğm": "kullandığım",
  "kçok": "çok",
  "görseldeki": "görseldeki",
  "gibi gibi": "gibi",
  "yumuşakck": "yumuşacık",
  "sapasağlam": "sapa sağlam",
  "paketlenmişki": "paketlenmiş ki",
  "yipranmis": "yıpranmış",
  "flas": "flash",
  "flaş": "flash",
  "yararlandimi": "yararlandım",
  "il": "ilk",
  "alacam": "alacağım",
   "yken": "iken",
  "şampuanı": "şampuan",
  "gibi gibi": "gibi",
  "a teşekkürler": "teşekkürler",
  "memnunuma": "memnunum",
  "verdi teşekkür ederim": "verildi, teşekkür ederim",
  "verdi teşekkürler": "verildi, teşekkürler",
  "hediyesizdii": "hediyesizdi",
  "gönderimişti": "gönderilmişti",
  "yapcam": "yapacağım",
  "yapicam": "yapacağım",
  "teskkurler": "teşekkürler",
  "sevə işlədirəm":  "severek kullanıyorum",
  "təşəkkür": "teşekkür ",
  "edirəm": "ederim",
  "bayagi": "bayağı",
   "karkodlarından": "barkodlarından",
  "kuttuk": "kontrol ettik",
  "ambalajlanmisti": "ambalajlanmıştı",
  "şampuanı": "şampuan",
  "fiyat trendyolda": "fiyatı Trendyol’da",
  "kargoda": "kargoyla",
  "flaş": "flash",
  "6tane": "6 tane",
  "ıyıydı": "iyiydi",
  "rahatlıgı": "rahatlığı",
  "alısverıs": "alışveriş",
  "güzelli": "güzelliği",
  "alcam": "alacağım",
  "yapcam": "yapacağım",
  "stokliyorum": "stokluyorum",
  "siparız": "sipariş",
  "itinalı": "özenli",
 "karkodlarından": "barkodlarından",
  "kuttuk": "kontrol ettik",
  "ambalajlanmisti": "ambalajlanmıştı",
  "şampuanı": "şampuan",
  "trendyolda": "Trendyol'da",
  "burdan": "buradan",
  "kargoda": "kargoyla",
  "flaş": "flash",
  "6tane": "6 tane",
  "ıyıydı": "iyiydi",
  "rahatlıgı": "rahatlığı",
  "alısverıs": "alışveriş",
  "güzelli": "güzelliği",
  "alcam": "alacağım",
  "yapcam": "yapacağım",
  "stokliyorum": "stokluyorum",
  "siparız": "sipariş",
  "tskrler": "teşekkürler",
  "tartisilmaz": "tartışılmaz",
  "stoklamalım": "stoklayalım",
   "kullanmistik": "kullanmıştık",
   "gönderdiğimiz": "gönderdiğiniz",
  "teşekkür ler": "teşekkürler",
  "coc": "çok",
  "indrime": "indirim",
  "quzel": "güzel",
  "tesekurler": "teşekkürler",
  "hədiyyə": "hediye",
  "olmamişti": "olmamişti",
  "keratın": "keratin",
  "hedıyesı": "hediyesi",
  "ćok": "çok",
  "olarakta": "olarak da",
  "kat": "çok",
  "indorime": "indirim",
  "işün": "için",
  "derım": "derim",
  "kacirmadim": "kaçırmadım",
  "şanpuanlarım": "şampuanlarım",
  "kargolamasi": "kargolaması",
  "içindee": "içinde",
  "teşekkürederim": "teşekkür ederim",
  "memnumum": "memnunum",
  "teşekkürllər": "teşekkürler",
  "almiştim": "almıştım",
  "başladim": "başladım",
  "saçlarim": "saçlarım",
  "teskkur": "teşekkür",
  "yapmisiniz": "yapmışsınız",
  "tskrlr": "teşekkürler",
  "girecegim": "gireceğim",
  "yayında": "yanında",
  "yumusaticilar": "yumuşatıcılar",
  "bayıldım": "bayıldım",
  "jestinize": "jestinize",
  "göndermişiler": "göndermişler",
  "tarakta": "tarak da",
  "tatlis": "tatlı",
  "tüşükkürler": "teşekkürler",
  "olarakta": "olarak da",
  "reddetdi": "reddetti",
  "kılları": "kıl",
  "kaldırmaca": "kandırmaca",
  "ehrhalde": "herhalde",
  "kiteli": "kaliteli",
  "gözeldi": "güzeldi",
  "en iyidi": "en iyisiydi",
  "tartakları": "fırçaları",
  "kanamalarimi": "kanamalarımı",
  "araliklari": "aralıkları",
  "olucak": "olacak",
  "sorunsjz": "sorunsuz",
  "2 cift": "2 çift",
  "verdi": "geldi",
  "simdiye": "şimdiye",
  "almadigima": "almadığıma",
  "pismanim": "pişmanım",
  "firca": "fırça",
  "temizligide": "temizliği de",
  "firca": "fırça",
  "sertligide": "sertliği de",
  "iyimis": "iyiymiş",
  "dislerimin": "dişlerimin",
  "fıçası": "fırçası",
  "bulamıyodum": "bulamıyordum",
  "cokda": "çok da",
  "firca": "fırça",
  "dış fırçası": "diş fırçası",
   "degismeyen": "değişmeyen",
  "firça": "fırça",
  "kapasitesı": "kapasitesi",
  "edılır": "edilir",
  "fircaydi": "fırçaydı",
  "gibi gibi": "gibi",
    "fircayi": "fırçayı",
  "curugum": "çürüğüm",
  "firca": "fırça",
  "tskkurler": "teşekkürler",
  "yapmis": "yapmış",
  "piyasay": "piyasaya",
  "gecerım": "geçerim",
  "bulundurduklarim": "bulundurduklarım",
  "miş": "mis",
  "fircalaniyor": "fırçalanıyor",
  "aynisi": "aynısı",
  "firca": "fırça",
  "teşrkkğrler": "teşekkürler",
    "dis fircasi": "diş fırçası",
  "ondirime": "indirime",
  "görüldügü": "görüldüğü",
  "gelfi": "geldi",
  "glmiyor": "gelmiyor",
  "fiyatıda": "fiyatı da",
  "öznele": "özenle",
  "sipariş im": "siparişim",
  "aynisi": "aynısı",
  "gorseldekinin": "görseldekilerin",
    "titreşimli": "titreşimli",
  "satıcıyada": "satıcıya da",
  "67 aydır": "6-7 aydır",
  "mereti": " ",
  "dhshsshs": "",
  "benve": "bence",
  "aralarini": "arasını",
  "fırçalaniyor": "fırçalanıyor",
  "indorime": "indirime",
  "hedef": "hediye",
  "hedıyesı": "hediyesi",
  "hədiyyə": "hediye",
  "tatlis": "tatlı",
  "tırışları": "tırnakları",
  "tskrlr": "teşekkürler",
  "dislerin": "dişlerin",
  "aralarini": "aralarını",
  "mükəmməl": "mükemmel",
  "kargolamasi": "kargolaması",
  "stokliyorum" : "stokluyorum",
   "ındırımda": "indirimde",
  "stogumuzu": "stoğumuzu",
  "heqiqeten": "hakikaten",
  "sertligi": "sertliği",
  "healt": "health",
  "teslımat": "teslimat",
  "verdi": "geldi",
  "markanun": "markanın",
   "kacmaz": "kaçmaz",
    "fircaliyor": "fırçalıyor",
  "birma": "bir marka",
  "balitesi": "kalitesi",
  "fırçasi": "fırçası",
  "tartişilmaz": "tartışılmaz",
  "fıtça": "fırça",
  "kullanıslı": "kullanışlı",
  "görümdüğü": "görüldüğü",
  "öznele": "özenle",
  "fıası": "fırçası",
  "yalam": "yalan",
  "raflarında ki": "raflardaki",
  "edldi": "edildi",
  "yildizida": "yıldızı da",
  "verdiğimi": "verdiğim",
  "gönderihassas": "gönderi hassas",
  "biras": "biraz",
  "dycek": "diyecek",
  "gonderdiniz": "gönderdiniz",
  "başarılıı": "başarılı",
  "kesinliklee": "kesinlikle",
    "kullanımmamış": "kullanılmamış",
  "kuşşanmaya": "kullanmaya",
  "muğlaka": "mutlaka",
  "kesinlikke": "kesinlikle",
  "killi": "kıllı",
  "firca": "fırça",
  "verdi": "geldi",
  "kullansihim": "kullanırım",
  "görüntüdügü": "göründüğü",
  "yımuşak": "yumuşak",
  "kullanıslı": "kullanışlı",
   "dis fircasi": "diş fırçası",
  "eks yumuşak": "ekstra yumuşak",
  "yılardır": "yıllardır",
  "verdi": "geldi",
  "fırça dan": "fırçadan",
  "degdirince": "değdirince",
  "işe": "ise",
  "kargo'nun": "kargonun",
  "3 sefere": "3 seferdir",
  "teşekkürr": "teşekkür",
  "firca": "fırça",
  "beğendm": "beğendim",
  "marke": "marka",
   "begendım": "beğendim",
  "kaliteloi": "kaliteli",
  "harıka": "harika",
  "fırçasıı": "fırçası",
  "kullanysli": "kullanışlı",
  "tavsiya": "tavsiye",
  "azalmasiyla": "azalmasıyla",
  "yukselen": "yükselen",
  "ettigim": "ettiğim",
  "firca": "fırça",
  "hassa":"hassas",
  "dis ellerine": "diş etlerine",
  "fiayti": "fiyatı",
  "artiyor": "artıyor",
  "verdi": "geldi",
  "alicak": "alacak",
  "istedıgım": "istediğim",
  "yollamıslar": "yollamışlar",
  "wimdiye": "şimdiye",
  "digim": "diğim",
  "acıktı": "açıktı",
  "tas": "taş",
  "alnabiliyor": "alınabiliyor",
  "fırçalardan": "fırçalardan",
  "dilslerim": "dişlerim",
  "arasina": "arasına",
  "giriyoe": "giriyor",
  "fırca": "fırça",
  "complate": "complete",
  "iyimiş": "iyiymiş",
  "fircasi": "fırçası",
  "kisminin": "kısmının",
  "dolayi": "dolayı",
  "fircayi": "fırçayı",
  "hıçbır": "hiçbir",
  "alabılırsınız": "alabilirsiniz",
  "gğxl": "güzel",
  "beğendım": "beğendim",
   "keyfiyyetli": "keyfiyetli",
  "iyibegendik": "iyi beğendik",
  "gelisiyoruz": "gelişiyoruz",
  "garcektan": "gerçekten",
  "sertorta": "sert orta",
  "dişetimi": "diş etimi",
  "sor yok": "sorun yok",
"proscıence": "proscience",
  "verdi": "geldi",
  "ugrasmak": "uğraşmak",
  "istemedigim": "istemediğim",
  "musteri": "müşteri",
  "iletisime": "iletişime",
  "gectigimde": "geçtiğimde",
  "urunlerim": "ürünlerim",
  "urunleri": "ürünleri",
  "degisim": "değişim",
  "ettigimde": "ettiğimde",
  "olmadigi": "olmadığı",
  "gonder diniz": "gönderdiniz",
  "soyledik": "söyledik",
  "ipananin": "ipana'nın",
  "kapaginin": "kapağının",
  "altindaki": "altındaki",
  "acildi": "açıldı",
  "aldiklarimda": "aldıklarımda",
  "farklı verdi": "farklı geldi",
  "iyide": "iyi de",
  "verdı": "geldi",
  "dis": "diş",
  "aksamlari": "akşamları",
  "sabahlari": "sabahları",
  "beyazlarici": "beyazlatıcı",
  "rahatlatiyor": "rahatlatıyor",
  "ezcanede": "eczanede",
  "anımda": "anında",
  "gızel": "güzel",
  "surwklı": "sürekli",
  "uyun": "uzun",
  "dis": "diş",
  "acısından": "açısından",
  "verdi": "geldi",
  "diş renimi": "diş rengimi",
  "aciyor": "açıyor",
  "rahatlatiyor": "rahatlatıyor",
  "harik": "harika",
  "hızlı verdi": "hızlı geldi",
  "sizi kalmıyor": "sızı kalmıyor",
  "iyidi": "iyisi",
  "dış etlerinizde": "diş etlerinizde",
  "bulamazsiniz": "bulamazsınız",
  "özenlı": "özenli",
  "kullanicam": "kullanacağım",
   "mecunu": "macunu",
  "3ocak": "3 Ocak",
  "s t t": "skt",
  "bitir": "bitti",
  "nədi": "neydi",
  "skt si": "skt'si",
  "yalniş": "yanlış",
  "yapdı yaptı": "yaptı",
  "gösterim": "görünüm",
  "bitir": "bitmiş",
  "skt": "skt",
  "eskidan": "eskiden",
  "yime": "yine",
  "kutularin": "kutuların",
  "verdi": "geldi",
  "gönderdiği": "gönderdiği",
  "beğendik": "beğendik",
  "verdi": "geldi",
  "göreceğim": "göreceğim",
  "kullandığımız": "kullandığımız",
  "ili": "iki",
  "sağ olun": "sağolun",
  "memnununv": "memnunum",
  "göztereceğim": "göstereceğim",
  "yolamişsınız": "yollamışsınız",
  "kiaa sürede": "kısa sürede",
  "kullansaydim": "kullansaydım",
  "kullanim": "kullanım",
  "trh": "tarihi",
  "yedek aldığımdan": "yedek aldığım için",
  "soylemisti": "söylemişti",
  "replika": "replika",
  "profesionel": "profesyonel",
  "internet'e": "internette",
  "vermıstım": "vermüştüm",
  "gonderin": "gönderin",
  "alakasu": "alakası",
  "arkadaşımım": "arkadaşımın",
  "oglumda": "oğlumda",
  "dis": "diş",
  "hassaiyette": "hassasiyette",
  "aldigimi": "aldığımı",
  "onerdi": "önerdi",
  "kalıcılığı": "kalıcılığı",
  "kırmızı şeritli": "kırmızı şeritli",
  "yorumsuz": "yorum yok",
  "bereketsiz verdi": "bereketsiz geldi",
  "dış macunu": "diş macunu",
  "tatdi": "tadı",
  "verdi": "geldi",
  "bir çok": "birçok",
  "dizimizde": "dişimizde",
  "tavsıyesı": "tavsiyesi",
  "şimidlik": "şimdilik",
  "begendigim": "beğendiğim",
  "dışlerimizi": "dişlerimizi",
   "ınsallah": "inşallah",
   "kullandığınım": "kullandığım",
  "tadıda": "tadı da",
  "su an": "şu an",
  "rahatlatiyor": "rahatlatıyor",
  "tavsiyrmizdir": "tavsiyemizdir",
  "paketlemede": "paketlemede",
  "güzel di": "güzeldi",
  "etlerşni": "etlerini",
  "dis": "diş",
  "sizlatmadan": "sızlatmadan",
  "verdi": "geldi",
  "aldiklarimla": "aldıklarımla",
  "paketleti": "paketleri",
  "etmyrm": "etmiyorum",
  "guvenmeyin": "güvenmeyin",
  "acilen": "acilen",
  "basladı": "başladı",
  "mahcumu": "macunu",
  "dış": "diş",
  "dis": "diş",
  "çök": "çok",
  "yildan": "yıldan",
  "iyidi": "iyiydi",
  "sizlamalarina": "sızlamalarına",
  "bayaga": "bayağa",
  "sensondyn": "sensodyne",
  "far": "fırça",
  "ozellikle": "özellikle",
  "oneririm": "öneririm",
  "sevidiğim": "sevdiğim",
  "müşderim": "müşterim",
  "vermişdi": "vermişti",
  "čok": "çok",
  "fotorağraftakile": "fotoğraftakilerle",
  "tesekkurler": "teşekkürler",
  "uyusmuyor": "uyuşmuyor",
  "orjinalligine": "orijinalliğine",
  "açıklarmısınız": "açıklar mısınız",
  "tertemis": "tertemiz",
  "fenadeğil": "fena değil",
  "alınabilr": "alınabilir",
  "dis": "diş",
  "dış": "diş",
  "diw macunu": "diş macunu",
  "beywendim": "beğendim",
  "etdiyimiz": "ettiğimiz",
  "tafsiye": "tavsiye",
  "uygu n": "uygun",
  "yanliş": "yanlış",
  "dagitirken": "dağıtırken",
  "nezaman": "ne zaman",
  "yollandı": "yollandı",
  "fiat": "fiyat",
  "ucızmuş": "ucuzmuş",
  "guze k": "güzel",
  "aalesef": "maalesef",
  "verdi": "geldi",
  "farli": "farklı",
  "karişiq": "karışık",
  "orjinel": "orijinal",
  "gönderimiş": "gönderilmiş",
  "dişetlerine": "diş etlerine",
  "tessekurler": "teşekkürler",
  "dişmacunu": "diş macunu",
  "gazel": "güzel",
  "kalite si": "kalitesi",
  "ma iyi": "ama iyi",
  "setlik": "sertlik",
  "fıŕça": "fırça",
  "fiyatlı": "fiyatlı",
  "keyfiyyetli": "kaliteli",
  "agritmadi": "ağrıtmadı",
  "teşek": "teşekkür",
  "adı": "aldım",
  "fırcası": "fırçası",
  "acitmiyor": "acıtmıyor",
  "adlım": "aldım",
"kullancam": "kullanacağım",
  "tasviye": "tavsiye",
  "firca": "fırça",
  "gönermişler": "göndermişler",
  "ihtitaçlardan": "ihtiyaçlardan",
    "disariya": "dışarıya",
  "teşklr": "teşekkürler",
  "ıygun": "uygun",
   "favorı": "favori",
  "muqemmel": "mükemmel",
  "çabuk verdi": "çabuk geldi",
  "coxda": "çok da",
   "tesekkurlee": "teşekkürler",
  "almiwdj": "almıştı",
  "tskler": "teşekkürler",
"kullanamıycam": "kullanamayacağım",
  "bilki": "belki",
    "qldim": "aldım",
  "tesekkurler": "teşekkürler",
  "çokuyg": "çok uyguna",
  "hediyeicin": "hediye için",
  "gayetgüzel": "gayet güzel",
  "diç": "diş",
  "memnhmuz": "memnunuz",
  "fircalari": "fırçaları",
  "ıcıza": "ucuza",
  "bişeydi": "bir şeydi",
   "sıgnal": "signal",
  "baxisdan": "bakıştan",
  "aliniri": "alınır",
  "güzəl": "güzel",
  "anlaminda": "anlamında",
  "sevmedigim": "sevmediğim",
  "sipariwim": "siparişim",
  "olmuwdu": "olmuştu",
  "bawqa": "başka",
  "koymuwsuz": "koymuşsunuz",
  "döküyo": "dökülüyor",
  "kutudaydi": "kutudaydı",
  "geldigi": "geldiği",
  "fircalar": "fırçalar",
"degilde": "değil de",
  "güle": "güzel",
  "aladım": "aldım",
  "basligi": "başlığı",
  "nhassas": "hassas",
  "denicez": "deneyeceğiz",
  "dıs": "diş",
  "fırcam": "fırçam",
  "yanındada": "yanında da",
  "yolladıhınız": "yolladığınız",
  "tesekkurler": "teşekkürler",
  "keşfettigim": "keşfettiğim",
  "işçi": "işçilik",
  "ellerini": "etlerini",
  "yumşak": "yumuşak",
  "bəyənirik": "beğeniyoruz",
  "superdi": "süperdi",
  "bakidan": "bakü'den",
  "zakaz": "sipariş",
  "etdik": "ettik",
  "yumsaqdir": "yumuşaktır",
  "yğzden": "yüzden",
  "yumşak": "yumuşak",
  "gayette": "gayet de",
  "4tane": "4 tane",
  "dişlerini": "dişini",
  "ünilevere": "unilever’e",
  "dii": "diş",
  "indirimda": "indirimde",
  "43 ye": "43 tl'ye",
  "yogunluktan": "yoğunluktan",
   "fırçayi": "fırçayı",
  "killari": "kılları",
  "tesekkurler": "teşekkürler",
  "yumuşakk": "yumuşak",
  "hashas": "hassas",
  "düşler": "dişler",
  "fitcasi": "fırçası",
  "dîş": "diş",
  "verdi": "geldi",
  "yumuşacıkhediyeniz": "yumuşacık, hediyeniz",
  "dahanda": "daha da",
"etdik": "ettik",
  "yaxşıdı": "iyidir",
  "kutusunuda": "kutusunu da",
  "bakimi": "bakımı",
   "guzel": "güzel",
  "almiştm": "almıştım",
 "heidye": "hediye",
  "yumuşaq": "yumuşak",
  "damağı": "damak",
  "heç": "hiç",
  "incitmir": "incitmiyor",
  "təmizliyir": "temizliyor",
  "endirimlə": "indirimle",
  "alir": "alır",
  "bunlari": "bunları",
  "yumuwacik": "yumuşacık",
  "kulandigim": "kullandığım",
  "hassasa": "hassassa",
  "pajet": "paket",
  "dil eti": "diş eti",
  "gibide": "gibi de",
  "ıstwyıgım": "istediğim",
  "fiyatli": "fiyatlı",
  "bir kaç": "birkaç",
  "dishi": "dişi",
  "səliqəli": "düzenli",
  "formada": "şekilde",
  "qablaşdırılıb": "paketlenmiş",
  "buldugu": "bulduğu",
  "disfircasi": "diş fırçası",
  "yılladır": "yıllardır",
  "tecihim": "tercihim",
  "wn": "en",
  "aldigimiz": "aldığımız",
    "güncellicem": "güncelleyeceğim",
  "beyazlatiyor": "beyazlatıyor",
  "uygun a": "uyguna",
  "gürseldeki": "görseldeki",
  "kaldımteşekkür": "kaldım teşekkür",
  "ön numara": "on numara",
  "palstikler": "plastikler",
  "olsaydi": "olsaydı",
  "formanın": "firmanın",
  "j0öğk": "çok",
  "digil": "değil",
  "deği": "değil",
"uygun a": "uyguna",
  "yerin": "yerine",
  "dimi": "değil mi",
  "iadede": "iade de",
  "inş": "inşallah",
  "sap kismi": "sap kısmı",
  "orijinali ile": "orijinaliyle",
  "pembeyesil": "pembe yeşil",
  "gondermis": "göndermiş",
  "fotografla": "fotoğrafla",
  "teshir": "teşhir",
  "onemsemiyor": "önemsemiyor",
  "firçasi": "fırçası",
  "uğraşamiyacağim": "uğraşamayacağım",
  "diyom": "diyorum",
  "idade": "iade",
  "umarik": "umarım",
  "fircalardan": "fırçalardan",
   "fiş": "diş",
  "kaldımteşekkür": "kaldım teşekkür",
  "ön numara": "on numara",
  "palstikler": "plastikler",
  "olsaydi": "olsaydı",
  "formanın": "fırmanın",
  "j0öğk": "çok",
  "digil": "değil",
  "deği": "değil",
  "umurak": "umarım",
  "yumusal": "yumuşak",
  "tesekürler": "teşekkürler",
  "halle": "hale",
  "yumuşaklıgı": "yumuşaklığı",
  "reklerden": "renklerden",
  "kullandigm": "kullandığım",
  "temizlendiginden": "temizlendiğinden",
  "olabiliyorsnz": "olabiliyorsunuz",
  "forçalıyorum": "fırçalıyorum",
  "yumcacık": "yumuşacık",
  "freş": "fresh",
   "tamizliyor": "temizliyor",
   "fır": "fırça",
  "məncə": "bence",
  "rahatligiyla": "rahatlığıyla",
  "favorimm": "favorim",
  "duruyo": "duruyor",
    "gonderıyorlar": "gönderiyorlar",
  "tarıhı": "tarihi",
  "cvpta": "cevapta",
  "degıl": "değil",
  "siparisim": "siparişim",
  "edicek": "edecek",
   "dişerime": "dişlerime",
  "memnnm": "memnunum",
  "begendim": "beğendim",
  "market e": "markete",
  "güzle": "güzel",
  "duş eti": "diş eti",
  "hassasiyet i": "hassasiyeti",
  "uygunfiyat": "uygun fiyat",
  "idal": "ideal",
  "stokladigi": "stokladığı",
  "merketlerde": "marketlerde",
  "verdi": "geldi",
  "yanlislik": "yanlışlık",
  "odemek": "ödemek",
  "ise": "işe",
  "anlayişiniz": "anlayışınız",
  "yakişir": "yakışır",
  "getirmisler": "getirmişler",
  "aldigimiz": "aldığımız",
  "hickimseye": "hiç kimseye",
   "yqapayım": "yapayım",
  "verdi": "geldi",
  "sipris": "sipariş",
  "etmistim": "etmiştim",
  "ugrasmamak": "uğraşmamak",
  "kalmadigi": "kalmadığı",
  "uğraştiyorlar": "uğraştırıyorlar",
  "guzele": "güzele",
  "beğenmedin": "beğenmedim",
  "ürunun": "ürünün",
  "inidirmli": "indirimli",
  "kullanacagim": "kullanacağım",
  "perf": "performans",
  "inş": "inşallah",
  "azərbaycana": "azerbaycan'a",
  "geç kalmış": "geç kalmıştı",
  "teşekkürlerö": "teşekkürler",
  "beyyndim": "beğendim",
  "iyidi": "iyiydi",
  "stoklamıstım": "stoklamıştım",
  "karıkoca": "karı koca",
  "tad": "tat",
  "vermıyor": "vermiyor",
   "kqmpanyada": "kampanyada",
  "beyazlatiyor": "beyazlatıyor",
  "verdi": "geldi",
  "paketlenmişdir": "paketlenmiştir",
  "ıyiydi": "iyiydi",
  "edirim": "ederim",
  "möhtəşəm": "muhteşem",
  "paketlənmə": "paketlenme",
  "beyazlatıyl": "beyazlatıyor",
    "fiatı": "fiyatı",
 "istedigi": "istediği",
  "herkesede": "herkese de",
  "avantaji": "avantajı",
  "setcaldim": "set aldım",
  "fiatı": "fiyatı",
  "etkinli": "etkinliği",
  "beyazlatiyor": "beyazlatıyor",
  "bembayaz": "bembeyaz",
  "beyazlatiyor": "beyazlatıyor",
  "kullandğm": "kullandığım",
  "kullanmda": "kullanmada",
  "dışardaki": "dışarıdaki",
  "kullanıldığım": "kullandığım",
  "ipanin": "ipana'nın",
  "sevdiğimidir": "sevdiğimdir",
  "gorunce": "görünce",
  "ılgılenenler": "ilgilenler",
  "kaçırmasın": "kaçırmasın",
  "beyazlatıyır": "beyazlatıyor",
  "dişleirmde": "dişlerimde",
  "diş etleirmde": "diş etlerimde",
  "pess": "pes",
  "beyazlatiyor": "beyazlatıyor",
  "datıcıdan": "satıcıdan",
  "hassasiyetide": "hassasiyeti de",
  "indirimdecaldim": "indirimde aldım",
  "disde": "dişte",
  "lekere": "lekelere",
  "soguk": "soğuk",
  "ındırımden": "indirimden",
   "beyazlatiyo": "beyazlatıyor",
  "ugun": "uygun",
  "geldigi": "geldiği",
  "fotograflardaki": "fotoğraflardaki",
  "aciklamadaki": "açıklamadaki",
  "beyazladi": "beyazladı",
  "iyidi": "iyiydi",
  "i": "iyi",
  "zamana": "zaman",
  "beyazlatmasi": "beyazlatması",
  "gelince": "gelince",
  "dışler": "dişler",
  "sasirdim": "şaşırdım",
  "bulamiyosun": "bulamıyorsun",
  "bulsanda": "bulsan da",
  "aliyoruz": "alıyoruz",
  "beyazlikta": "beyazlıkta",
  "ustune": "üstüne",
  "tanimam": "tanımam",
  "dislerimi": "dişlerimi",
  "sizlatan": "sızlatan",
  "fircalandiginda": "fırçalandığında",
  "dislerinize": "dişlerinize",
  "guveniyorsaniz": "güveniyorsanız",
  "yetisdi": "yetişti",
    "değşmem": "değişmem",
  "uyqun": "uygun",
   "kullanin": "kullanın",
  "heyet": "gayet",
  "ferahlıgın": "ferahlığını",
   "teslimm": "teslim",
  "disler": "dişler",
  "sorunsuzgeldi": "sorunsuz geldi",
   "buyutu": "boyutu",
  "disler": "dişler",
  "macun u": "macunu",
    "bitikçe": "bittikçe",
  "beyazlasiyor": "beyazlaşıyor",
  "temilik": "temizlik",
  "macunlari": "macunları",
  "beyazlatıyo": "beyazlatıyor",
  "akici": "akıcı",
  "değilim": "değil",
  "güzle": "güzel",
  "beyazlatmasıda": "beyazlatması da",
  "kaldigini": "kaldığını",
  "soyledi": "söyledi",
  "kullandıgımız": "kullandığımız",
  "fircalama": "fırçalama",
  "beyazlarmada": "beyazlatmada",
  "dislerime": "dişlerime",
  "dışarda": "dışarıda",
  "market lerden": "marketlerden",
  "ipanayi": "ipanayı",
  "ortodondik": "ortodontik",
  "fotoğrafdaki": "fotoğraftaki",
  "parlatici": "parlatıcı",
  "eserlerinden": "lekelerinden",
  "paraniz": "paranız",
  "indiirme": "indirime",
  "gönderi": "gönderim",
  "ed ildi": "edildi",
  "beyazlatuyor": "beyazlatıyor",
  "defedi": "defa",
  "aliram": "alırım",
  "bsxib": "bakıp",
  "bir nece dege": "bir defa",
  "istifade": "istifade",
  "etmisem": "etmişim",
  "yaxsidi": "iyidi",
  "fiyati": "fiyatı",
  "uzuza": "ucuza",
  "ağardici": "beyazlatıcı",
  "məhsultəşəkkürlər": "ürün, teşekkürler",
  "guzelfakat": "güzel fakat",
  "aldiginiz": "aldığınız",
  "etkilo": "etkili",
  "fircalayamam": "fırçalayamam",
  "degistirdim": "değiştirdim",
  "geçicemsatıcıya": "geçeceğim, satıcıya",
  "fotoğrafta ki": "fotoğraftaki",
  "kaldın": "kaldım",
  "güzeldii": "güzeldi",
  "attim": "attım",
  "stokladığ": "stokladığım",
  "veridim": "verdim",
  "garkettim": "fark ettim",
  "sehirdeymisiz": "şehirdeymişiz",
  "təxmini": "təxminən",
  "geç gəlsədə": "geç gəlsə də",
  "sifarişim": "sifarişım",
  "razıyam": "razıyam",
  "bakarmısınız": "bakar mısınız",
  "gonderıldı": "gönderildi",
  "gözlü": "güzel",
  "görselde ki": "görseldeki",
  "uru n": "ürün",
  "bakü'ye": "Bakü'ye",
  "verdi": "geldi",
  "orn": "örnek",
  "kamlumbağa": "kaplumbağa",
 "üzün": "uzun",
  "ezilmis": "ezilmiş",
  "kapagi": "kapağı",
  "değildi": "değildi",
  "u tan maz lar": "utanmazlar",
  "hak etkiliyor": "hak ediyor",
  "geliyor gelmez": "gelir gelmez",
  "tiryakisiyim": "tiryakisiyim",
  "sariliklarina": "sarılıklarına",
  "görunum": "görünüm",
  "verecem": "vereceğim",
  "yenə": "yine",
   "isdik": "istedik",
  "beyazlatmasıda": "beyazlatması da",
  "disleri": "dişleri",
  "beyazlatiyor": "beyazlatıyor",
  "temuzlik": "temizlik",
  "hissediyorsun": "hissediyorsunuz",
  "parllatıyor": "parlatıyor",
  "patlattı": "parlattı",
  "verdi": "geldi",
  "pataya": "fiyata",
  "yapayom": "yapayım",
  "haketmiyor": "hak etmiyor",
  "gondermis": "göndermiş",
  "gönderdiğiniz": "gönderdiğiniz",
  "tarıhı": "tarihi",
   "çekilmesi": "çekilmesi",
  "qutu": "kutu",
  "əzilmiş": "ezilmiş",
  "məhsullarda": "ürünlerde",
  "eyni": "aynı",
  "iyun": "haziran",
  "7dən": "7’sinde",
  "etdim": "ettim",
  "iyunun": "haziranın",
  "28i": "28’inde",
  "fotorafdakinin": "fotoğraftakinin",
  "almağa": "almaya",
  "kulandiğim": "kullandığım",
  "nedenirki": "ne denir ki",
  "patlatıyor": "parlatıyor",
  "sevkiyat": "teslimat",
  "iyidpp9ir": "iyidir",
  "özenliydı": "özenliydi",
  "eksra": "ekstra",
  "hatali": "hatalı",
   "kulanmadım": "kullanmadım",
  "kulanıca": "kullanınca",
  "patlamisti": "patlamıştı",
    "waov": "iyi",
  "beyazlaticagini": "beyazlatacağını",
  "kutı": "kutu",
  "yamulmuştı": "yamulmuştu",
  "aldığımda": "aldığımda",
  "severak": "severek",
  "alib": "alıp",
  "hicde": "hiç de",
  "dish": "diş",
  "makule": "makul",
  "uretim": "üretim",
  "iyidiş": "iyi diş",
  "verdi": "geldi",
  "ha b ve gün": "hafta bir gün",
  "doldurma": "sahte",
  "bandıjelatini": "bandı jelatini",
  "girdikçe": "girdikçe",
  "alıyom": "alıyorum",
   "berektli": "bereketli",
  "manitamda": "manitan da",
  "tavisye": "tavsiye",
  "macuların": "macunların",
  "marca": "marka",
  "stokladıgım": "stokladığım",
  "gordukce": "gördükçe",
  "kattigini": "kattığını",
  "aldıp": "alıp",
  "çookk": "çok",
  "kulnmadım": "kullanmadım",
  "kullnıyorum": "kullanıyorum",
  "grice": "girince",
  "bılıyorum": "biliyorum",
  "bunu": "bununu",
  "gibi gibi": "gibi",
  "alınn": "alın",
  "aldırınn": "aldırın",
  "devam lı": "devamlı",
  "memmunun": "memnunum",
  "kampanya ya": "kampanyaya",
  "verdi": "geldi",
  "tesekür": "teşekkür",
  "denk verdi": "denk geldi",
  "teshekkurler": "teşekkürler",
  "ichin": "için",
  "uurnler": "ürünler",
  "gondermislerdi": "göndermişlerdi",
  "eladi": "geldi",
  "dökümme": "dökülme",
  "uyguma": "uyguna",
  "tesekkurlet": "teşekkürler",
  "mehsuldu": "üründür",
  "hədiyyənizə": "hediyenize",
  "təşəkkürlət": "teşekkürler",
  "kabi": "kabı",
  "macunum": "macunumdur",
  "mehsulu": "ürünü",
  "ailelikce": "ailecek",
  "vwrdim": "verdim",
  "kendjmde": "kendimde",
  "guzel": "güzel",
  "məmnunam": "memnunum",
  "indirimdə": "indirimde",
  "sipariş verdiyim": "sipariş verdiğim",
  "birbaşa": "doğrudan",
  "vaxtında": "zamanında",
  "formada": "şekilde",
  "ulaşdırıldı": "ulaştırıldı",
   "mohteseemm": "muhteşem",
  "neçe": "birkaç",
  "ayriliqda": "ayrıca",
  "olacagsiz": "olacaksınız",
  "beyazlatiyor": "beyazlatıyor",
  "dişları": "dişleri",
  "kasinlikle": "kesinlikle",
  "göndərmisiniz": "göndermişsiniz",
  "sıkıyom": "sıkıyorum",
  "begendim": "beğendim",
  "verdi": "geldi",
  "diydi": "dişiydi",
  "yarı ya": "yarıya",
  "4 lu": "4'lü",
  "inanlımaz": "inanılmaz",
  "kem kapıldı": "hemen kapıldı",
  "unfulurum": "unuturum",
  "mavimi": "mavi mi",
  "renklimi": "renkli mi",
  "aldiğimda": "aldığımda",
   "qalib": "kalmış",
  "kutulari": "kutuları",
  "ozensiz": "özensiz",
  "macunue": "macunu",
  "grice": "görünce",
  "ulaşt ı": "ulaştı",
   "defasinda": "defasında",
  "stokladigimiz": "stokladığımız",
  "kullanıcam": "kullanacağım",
  "mağlum": "malum",
  "dişlerimdeki": "dişlerimde ki",
  "farkı": "farkı",
  "girer girmez": "girer girmez",
  "bir kaç": "birkaç",
  "sıkıntısız ulaşiyor": "sıkıntısız ulaşıyor",
  "indırıme": "indirime",
  "dusunce": "düşünce",
  "fşyatla": "fiyatla",
  "beyalatmasından": "beyazlatmasından",
    "herzman": "her zaman",
  "alıyorumm": "alıyorum",
  "kulandıgım": "kullandığım",
  "hedileriniz": "hediyeleriniz",
  "siparish": "sipariş",
   "mukemller": "mükemmel",
  "bembehaz": "bembeyaz",
  "gibi gibi": "gibi",
  "macuun": "macun",
  "kargolanmiştı": "kargolanmıştı",
  "signal e": "Signal'e",
  "aldın": "aldım",
  "yanında ki": "yanındaki",
  "den": "memnunum",
  "on numara": "on numara",
  "denk verdi": "denk geldi",
  "kullandım": "kullanıyorum",
  "efsane": "efsane!",
 "onual": "onu al",
  "favorin": "favorim",
  "məcunudu": "macunuydu",
  "istifade": "istifadə",
  "edirik": "ediyoruz",
  "paket leme": "paketleme",
  "beyazlarması": "beyazlatması",
  "mükoo": "müko",
  "alış veriş": "alışveriş",
  "beyzlatiyor": "beyazlatıyor",
  "bayada": "baya da",
  "aklima": "aklıma",
  "pembemor": "pembe mor",
  "üründegil": "ürün değil",
  "begenildigi": "beğenildiği",
  "kullandğmz": "kullandığımız",
  "diğim": "dediğim",
  "yaxwi": "iyi",
  "paketlərin": "paketlerin",
  "için de": "içinde",
  "hızlə": "hızla",
  "bayada": "baya da",
  "agizda": "ağızda",
  "stoklamışta": "stoklamış da",
  "dislerim": "dişlerim",
  "yapicagim": "yapacağım",
  "fiuata": "fiyata",
   "stoklansınn": "stoklansın",
  "degişim": "değişim",
  "ediliyo": "ediliyor",
  "cok": "çok",
  "inidirimde": "indirimde",
  "bulamiyorum": "bulamıyorum",
  "beyazlattigina": "beyazlattığına",
  "ulasildi": "ulaşıldı",
  "hediyyeller": "hediyeler",
  "pakelenmişti": "paketlenmişti",
  "diydi": "idi",
  "tovsiyye": "tavsiye",
  "indirimdekaçırmak": "indirimde kaçırmak",
  "foyat": "fiyat",
  "bununu": "bunu",
  "çok iy": "çok iyi",
  "2 ci": "2. ci",
  "paketlenmeside": "paketlenmesi de",
  "iy yapılmışdı": "iyi yapılmıştı",
  "kullnıyoruz": "kullanıyoruz",
  "hiss ediyorum": "hissediyorum",
  "çübki": "çünkü",
  "endirimdeyken": "indirimdeyken",
  "stok cuyum": "stokçuyum",
  "stolkluyorum": "stokluyorum",
  "diydi": "idi",
  "özellikler": "özellikle",
  "1hafta": "1 hafta",
  "4lü": "4'lü",
  "patlatması": "parlatması",
  "kesinlkle": "kesinlikle",
  "beyazlatti": "beyazlattı",
  "macunlarini": "macunlarını",
  "kopurmuyor": "köpürmüyor",
  "syhtem": "system",
  "dislerimde": "dişlerimde",
  "fiyatıda": "fiyatı da",
  "olmasini": "olmasını",
  "agiz": "ağız",
  "beyaxlik": "beyazlık",
  "sagliyo": "sağlıyor",
  "markasini": "markasını",
  "cay": "çay",
  "icen": "içen",
  "tawsiye": "tavsiye",
  "gibi gibi": "gibi",
  "diş te": "dişte",
  "histe": "hissi de",
  "güzelde": "güzel de",
  "sistem": "System",
  "sarariyor": "sararıyor",
  "kullandigimda": "kullandığımda",
  "sararmalarin": "sararmaların",
  "dışı": "dışı",
  "macunun": "macunun",
  "beslendiğim": "beğendiğim",
  "sararmisti": "sararmıştı",
  "kesinlik": "kesinlikle",
  "dişlerim beyazladı": "dişlerim beyazlaştı",
  "ağız da": "ağızda",
  "4 lu": "4'lü",
  "dişleri mi": "dişlerimi",
  "cay": "çay",
  "birr": "bir",
  "4lü": "4'lü",
  "geliyor gelmez": "gelir gelmez",
  "6 ededlik": "6 adetlik",
  "1eded": "1 adet",
  "kem verdi": "hem verdi",
  "satis": "satış",
  "belle": "belli",
  "nə": "ne",
  "gəlmədi": "gelmedi",
  "ha bir de": "ha, bir de",
  "edər": "eder",
  "kullanıyoryz": "kullanıyoruz",
  "yanındaki ki": "yanındaki",
  "indirim giriyir": "indirim giriyor",
  "istofade": "istifade",
  "güzel ler": "güzeller",
  "firsadim": "fırsatım",
  "paketlenmish": "paketlenmiş",
  "ekonomi": "ekonomik",
  "haruka": "harika",
  "yakalim": "yakaladım",
  "gondermısler": "göndermişler",
  "hediyyeye": "hediyeye",
  "curuyen": "çürüyen",
  "yapisina": "yapısına",
  "sag olun": "sağ olun",
  "almsm": "almışım",
   "endirimdəykən": "indirimdeyken",
  "her kes": "herkes",
  "sag olun": "sağ olun",
  "wekilde": "şekilde",
  "cati": "çatdı",
  "azerbaycana": "Azerbaycan'a",
  "diw": "diş",
  "tovsiye": "tavsiye",
  "alamaga": "almaya",
  "temizleyir": "temizliyor",
  "qanatmir": "kanatmıyor",
  "dorunsuz": "sorunsuz",
  "kalsbalık": "kalabalık",
  "telrkür": "teşekkür",
  "tavdiye": "tavsiye",
  "keyfiyyətli": "kaliteli",
  "məhsullardi": "ürünlerdi",
  "atiram": "atıyorum",
  "qeweng": "güzel",
  "kopullenir": "köpürür",
  "mutleq": "mutlaka",
  "stoklayacagim": "stoklayacağım",
  "bantlamislar": "bantlamışlar",
  "şəkildə": "şekilde",
  "pariltatiyor": "parlatıyor",
  "bayildim": "bayıldım",
  "iyikide": "iyi ki de",
  "harikasa": "harikaysa",
  "özenliydii": "özenliydi",
  "indirim deydi": "indirimdeydi",
  "iyidi": "iyiydi",
  "hediyesj": "hediyesi",
  "foyat": "fiyat",
  "uygu a": "uyguna",
  "signall": "Signal",
  "teskler": "teşekkürler",
  "kutular da": "kutular da",
  "ben bakuda": "ben bakü'de",
   "hak etkiliyor": "hak ediyor",
  "iyimi": "iyi ki",
  "gerçektende": "gerçekten de",
  "fiyatıda": "fiyatı da",
  "uzın": "uzun",
  "cocugumda": "çocuğum da",
  "bununu": "bunu",
  "aimak": "almak",
  "istesim": "istedim",
  "foyata": "fiyata",
  "icleri": "içleri",
    "maacun": "macun",
  "gibvi": "gibi",
  "paketlenmish": "paketlenmiş",
  "keyfiyyeti": "kalitesi",
  "çekınmeden": "çekinmeden",
  "tewwekurler": "teşekkürler",
  "gerkecekten": "gerçekten",
  "farkedeceksiniz": "fark edeceksiniz",
  "edeck": "edecek",
  "dueyd": "düzeyde",
  "gelmisti": "gelmişti",
  "şübhe": "şüphe",
  "kojusu": "kokusu",
  "büzülme": "buruşma",
"gelmed": "gelmedi",
  "keyfiyyeti": "kalitesi",
  "çekınmeden": "çekinmeden",
  "tewwekurler": "teşekkürler",
  "gerkecekten": "gerçekten",
  "farkedeceksiniz": "fark edeceksiniz",
  "sarıldığından": "sarılığından",
  "geliyor gelmez": "gelir gelmez",
  "bir kaç": "birkaç",
  "fiyatıda": "fiyatı da",
  "bununu": "bunu",
  "itaberen": "itibaren",
  "sasirti": "şaşırttı",
  "hisside": "hissi de",
  "dişlerimde ki": "dişlerimdeki",
  "ferahlı": "ferahlık",
  "çok güzelaz": "çok güzel, az",
  "dişlerimde ki": "dişlerimdeki",
  "yasamamdı": "yaşamadım",
  "beyazladıgını": "beyazladığını",
  "dislerim": "dişlerim",
  "disetim": "di şetim",
  "hasas": "hassas",
  "olusturmadi": "oluşturmadı",
  "benzetsemde": "benzetsem de",
  "hossettim": "hissettim",
  "farkettim": "fark ettim",
  "kulanmaya": "kullanmaya",
  "edrim": "ederim",
  "anlıcaksınız": "anlayacaksınız",
  "gibvi": "gibi",
  "tewwekurler": "teşekkürler",
  "kullandigimda": "kullandığımda",
  "diyebilirm": "diyebilirim",
  "diş ettim": "diş etim",
  "paketlenmish": "paketlenmiş",
  "beyazladıgını": "beyazladığını",
  "karincalanmi": "karıncalanma",
  "beuaz": "beyaz",
  "bıraktıpı": "bıraktığı",
  "seffaf": "şeffaf",
  "dislerdeki": "dişlerdeki",
  "dislerim": "dişlerim",
  "sağlığımıda": "sağlığımı da",
   "yöndende": "yönünden de",
  "paketkenmısıtı": "paketlenmişti",
  "esnasında ki": "esnasındaki",
  "farkettim": "fark ettim",
  "dişetlerindeki": "diş etlerindeki",
  "fircalamada": "fırçalamada",
  "agizimda": "ağzımda",
  "biraktigi": "bıraktığı",
  "borakıyor": "bırakıyor",
  "diş ettim": "diş etim",
  "bayazlanam": "beyazlatma",
  "hak etkiliyor": "hak ediyor",
  "kulanimdaki": "kullanımdaki",
  "bıraktıgi": "bıraktığı",
  "iyikide": "iyi ki de",
  "etkiliyor": "etkiliyor",
  "çoj": "çok",
  "demli çay icebilirim": "demli çay içebilirim",
  "o sorunuda": "o sorunu da",
  "kallteli": "kaliteli",
  "diş macunlarinda": "diş macunlarında",
  "birakmasi": "bırakması",
  "ağizda": "ağızda",
  "karşiladi": "karşıladı",
  "mithiş": "müthiş",
   "bununu": "bunun",
  "fircalamadan": "fırçalamadan",
  "beyazladigini": "beyazladığını",
  "farkediyorsunuz": "fark ediyorsunuz",
  "hak etkiliyor": "hak ediyor",
  "agzinizda": "ağzınızda",
  "fircalanada": "fırçalamada",
  "dislerimdeki": "dişlerimdeki",
  "ferahlıtıcı": "ferahlatıcı",
  "belliydi": "belirgindi",
  "kulanıyordum": "kullanıyordum",
  "tanene": "tanesini",
  "alcaksınız": "alacaksınız",
  "attm": "attım",
  "sıparişimle": "siparişimle",
  "həpsi": "hepsi",
  "yollamissinjz": "yollamışsınız",
  "dusunmuyorum": "düşünmüyorum",
  "agiz": "ağız",
  "ne yazıkki": "ne yazık ki",
  "begenmedik": "beğenmedik",
   "bor": "bir",
  "gözləməyin": "gözlemeyin",
  "degistirsemde": "değiştirsem de",
  "agzinda": "ağzında",
  "bir da": "bir daha",
  "dokunmaniz": "dokunmanız",
  "deniycem": "deneyeceğim",
  "özensiz verdi": "özensizdi",
  "indirimda": "indirimde",
  "dislem": "dişlerim",
   "fırçaladigim": "fırçaladığım",
  "bitdikce": "bittikçe",
  "verdi": "geldi",
  "5a": "5'e",
  "terchimdirr": "tercihimdir",
  "kullandımız": "kullandığımız",
  "2 li.": "2'li",
  "verdi": "geldi",
  "sevdiğinm": "sevdiğim",
  "denk verdi": "denk geldi",
  "eddigim": "ettiğim",
  "ise yarar": "işe yarar",
   "gusel": "güzel",
  "fiyatperformans": "fiyat performans",
  "akmadökülme": "akma dökülme",
  "guncel": "güncel",
  "gozle": "gözle",
  "gorulur": "görülür",
  "olrak": "olarak",
  "dede": "de",
  "yazarim": "yazarım",
  "beyazlatmiyomus": "beyazlatmıyormuş",
 "temuzliyor": "temizliyor",
  "bununu": "bunu",
  "dempatation": "temptation",
  "kokularıda": "kokuları da",
  "arac": "araç",
  "okucakti": "olucaktı",
   "posetinin": "poşetinin",
  "posetini": "poşetini",
  "acilmis": "açılmış",
  "icindeki": "içindeki",
  "acik": "açık",
  "pismanlik": "pişmanlık",
  "bununu": "bunu",
  "anlayışlari": "anlayışları",
  "kurye": "kuryeyle",
  "gitmis": "gitmiş",
  "arasdira": "araştırabilir",
  "bilirmisiniz": "bilir misiniz",
  "etkiliyor": "etkilemiyor",
  "basqa": "başka",
   "alırmıyım": "alır mıyım",
    "ikisine": "ikisi de",
  "kaliciligi": "kalıcılığı",
  "akslarla": "Axe'larla",
  "tane": "taneye",
  "atilmiş": "atılmış",
  "kökü": "kokusu",
  "fiyatıda": "fiyatı da",
  "turubcu": "turuncu",
  "igrenc": "iğrenç",
  "ihiyyiyiyiy": "iyi",
   "alabilir siniz": "alabilirsiniz",
  "cooper santral": "copper santal",
  "en iyidi": "en iyisiydi",
  "deodarttanta": "deodoranlardan",
  "3 üde": "3'ü de",
  "civarındayı": "civarındaydı",
  "şampuan amedspor 47": "şampuan, Amedspor 47",
  "3 ününde": "3'ünün de",
  "bununu": "bunu",
  "muk": "mükemmel",
  "içşn": "için",
  "fotoraftaki": "fotoğraftaki",
  "denk verdi": "denk geldi",
  "bgndık": "beğendik",
  "oldnı": "olanı",
  "iyiymis": "iyiymiş",
  "digerlero": "diğerleri",
  "güzeldiruu": "güzeldir",
  "en iyidi": "en iyisiydi",
 "devamke": "devam",
  "2 si": "2'si",
  "2 günde verdi": "2 günde geldi",
  "üçünüde": "üçünü de",
  "çok yaşasamda": "çok yaşasam da",
  "kokulular": "kokular",
  "arac içi": "araç içi",
  "tutacagi": "tutacağı",
"kokuyolar": "kokuyorlar",
"kalıcığı": "kalıcılığı",
  "konular": "kokular",
  "bergama": "bergamot",
  "axe ın": "axe'ın",
  "gerekiyo": "gerekiyor",
   "diydi": "idi",
  "siparisimiz": "siparişimiz",
  "yesil": "yeşil",
  "kirmizi": "kırmızı",
  "sktgmda": "sıktığımda",
  "acılir": "açılır",
  "eddim": "ettim",
  "dakka": "dakika",
  "tatlı": "tatlı",
  "esıme": "eşime",
  "begendı": "beğendi",
  "indirim deyken": "indirimdeyken",
  "kullanılandım": "kullandım",
  "carsmelli": "karamelli",
  "tempetion": "temptation",
  "kokuyoo": "kokuyor",
  "begendi": "beğendi",
  "kokuyormus": "kokuyormuş",
  "sıktıgi": "sıktığı",
  "arkadşıma": "arkadaşıma",
  "koksu": "kokusu",
  "oarfüm": "parfüm",
  "sağlik": "sağlık",
   "beğenditeşekkür": "beğendi teşekkür",
  "sevdigimiz": "sevdiğimiz",
  "verdi": "geldi",
  "hrika": "harika",
  "kosusu": "kokusu",
  "kokuo": "kokuyor",
  "ağir": "ağır",
  "oğlymma": "oğluma",
  "parfumden": "parfümden",
    "ozenliyli": "özenliydi",
  "kaldımhemen": "kaldım hemen",
  "kalicisiz": "kalıcısız",
  "içersinde": "içerisinde",
  "çarpa": "çarparak",
  "yolluyolar": "yolluyorlar",
  "asirii": "aşırı",
  "mümkün": "mükemmel",
  "tam dolayı": "tam dolu",
  "yarı dolayı": "yarı dolu",
  "denecekz": "deneyeceğiz",
  "takim": "takım",
  "hak etkiliyor": "hak ediyor",
  "hulllaniyor": "kullanıyor",
  "ederiiz": "ederiz",
  "kutulama": "kutulama",
  "sapa sağlam": "sapasağlam",
   "verdi gönderilen": "geldi, gönderilen",
  "verdi teşekkürler": "geldi, teşekkürler",
  "kokularindan": "kokularından",
  "efsanevibir": "efsanevi bir",
  "çocuklarima": "çocuklarıma",
  "müqq": "mükemmel",
  "çarpa": "çarparak",
  "tesekkurler": "teşekkürler",
  "kokuuor": "kokuyor",
   "tşklr": "teşekkürler",
  "cokiyi": "çok iyi",
  "kalmamis": "kalmamış",
  "malıı": "malı",
  "ürin": "ürün",
  "bununu": "bunu",
  "paketlanmiş": "paketlenmiş",
  "yıllrdır": "yıllardır",
  "gargo": "kargo",
  "qaliciligida": "kalıcılığı da",
  "qoxusuda": "kokusu da",
  "kokuho": "koku",
   "gelime": "elime",
  "sevdıgı": "sevdiği",
  "ger": "her",
  "kicam": "kullanacak",
  "koynuşlardı": "koymuşlardı",
  "içinde": "için de",
  "beğenildi": "beğenildiği",
  "özenliydii": "özenliydi",
  "ıcınde": "için de",
  "sadecik": "sadece",
  "snrki": "sonraki",
    "verdi": "geldi",
  "yaninada": "yanına da",
  "bitane": "bir tane",
  "nurdan": "buradan",
  "uniliver": "unilever",
  "sapasağlam": "sapa sağlam",
  "kool": "cool",
  "begeniyoruz": "beğeniyoruz",
  "gediye": "hediye",
  "hediyede": "hediye de",
  "toplanımlarda": "toplu alımlarda",
  "pakettleme": "paketleme",
  "düsunulmüs": "düşünülmüş",
  "içinde": "için de",
  "geldi": "geldi",
  "herzman": "rexona her zaman",
  "verdi": "geldi",
  "her hangi": "herhangi",
  "saolun": "sağ olun",
  "kakusu": "kokusu",
  "nişanlımı": "nişanlıma",
  "diydi": "iyiydi",
  "için de": "içinde",
  "sevdigimiz": "sevdiğimiz",
  "herzman": "her zaman",
  "verdi": "geldi",
  "her hangi": "herhangi",
  "saolun": "sağ olun",
  "kakusu": "kokusu",
  "nişanlımı": "nişanlıma",
  "diydi": "iyiydi",
  "için de": "içinde",
  "sevdigimiz": "sevdiğimiz",
  "kulla iyordu": "kullanıyordu",
  "suparisim": "siparişim",
  "fiyasaya": "piyasaya",
  "uygu": "uygun",
  "dolayı": "doluydu",
  "deodorant ı": "deodorantı",
  "helâl": "helal",
  "teşekkrler": "teşekkürler",
  "hızlıdı": "hızlıydı",
  "saolsunlar": "sağ olsunlar",
  "gonderilmisti": "gönderilmişti",
  "yollamişlar": "yollamışlar",
  "girmisken": "girmişken",
  "atokladim": "stokladım",
  "kullanılabilcek": "kullanılabilecek",
  "gondermenizde": "göndermenizde",
  "hepsının": "hepsinin",
  "kalici": "kalıcı",
  "olsuun": "olsun",
  "hadiye": "hediye",
  "teşşekkür": "teşekkür",
  "kokusna": "kokusuna",
  "bayildii": "bayıldı",
  "bşr": "bir",
  "vazgecemediği": "vazgeçemediği",
  "deoderanti": "deodorantı",
  "vazgecilmezi": "vazgeçilmezi",
  "paketlemeydi": "paketlemesiydi",
  "csntasi": "çantası",
  "gayrt": "gayet",
  "gekdi": "geldi",
  "tras ": "traş",
  "ayretten": "ayrıca",
  "gönderildş": "gönderildi",
  "azamandada": "zamanda da",
  "geldii": "geldi",
  "küçüm": "küçük",
  "mıtlu": "mutlu",
  "almayin": "almayın",
  "sikarsiniz": "sıkarsınız",
  "yazmasi": "yazması",
  "yanlış": "yanlış",
  "orjinal": "orijinal",
  "gramsjı": "gramajı",
  "sipiş": "süpürüş",
  "olmazi": "olmazı",
  "fiyatlarin": "fiyatların",
   "teşekkürederiz": "teşekkür ederiz",
  "degerlelmndirmek": "değerlendirmek",
  "emegi": "emeği",
  "geçenherkese": "geçen herkese",
  "4aydi": "4 aydır",
  "istifadə": "kullanıyor",
  "başağrısı": "baş ağrısı",
  "sevdiyi": "sevdiği",
  "bayildim": "bayıldım",
  "xl": "",
  "rahatlıyla" : "rahatlığıyla" ,
  "cocuklarimda": "çocuklarımda",
  "deodorant ı": "deodorantı",
  "kargolanmıstı": "kargolanmıştı",
  "işin aldım": "için aldım",
  "uyğun": "uygun",
  "baya" : "bayağı",
  "almisin": "almışsım",
  "aloveralı": "aloe veralı",
  "kalite li.": "kaliteli",
  "number": "numara",
  "one": "bir",
  "qeşengdı": "güzeldi",
  "müqemmel": "mükemmel",
  "parlaq": "parlak",
  "yumusaciq": "yumuşacık",
  "olfu": "oldu",
  "helede": "hala da",
  "yoxdur": "yoktur",
  "extra": "ekstra",
  "kaldigimix": "kaldığımız",
  "sıkmıstı": "sıkmıştı",
  "şəkilde": "şekilde",
  "sakin": "sakın",
  "dəfə ": "defa",
  "1010": "10/10",

}

def fix_typos_and_remove_stopwords(text, typo_dict, stopwords_set):
    words = text.split()
    cleaned_words = []
    for word in words:
        corrected = typo_dict.get(word, word)
        if corrected not in stopwords_set:
            cleaned_words.append(corrected)
    return ' '.join(cleaned_words)

def clean_text(text, typo_dict, stopwords_set):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    text = text.lower()
    text = remove_word_repeats(text)
    text = fix_missing_space_after_dot(text)
    text = fix_merged_number_words(text)
    text = remove_punctuation_noise(text)
    text = normalize_unicode(text)
    text = fix_typos_and_remove_stopwords(text, typo_dict, stopwords_set)
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^\w\sçğıöşüÇĞİÖŞÜ.,!?']", "", text)
    text = re.sub(r"\n|\r", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"(.)\1{2,}", r"\1", text)
    text = re.sub(r"(\.)\1{2,}", r"\1", text)
    return text



total = len(df_final_perceptual)

for i in range(total):
    if i % 100 == 0:
        print(f"İşlenen yorum: {i}/{total}")

    comment = df_final_perceptual.iloc[i]["comment_clean"]

    if not isinstance(comment, str) or comment.strip() == "":
        continue

    cleaned = clean_text(comment, typo_corrections1, final_stopwords)
    df_final_perceptual.iloc[i, df_final_perceptual.columns.get_loc("comment_clean")] = cleaned

df_final_perceptual = df_final_perceptual.dropna(subset=["comment_clean"])
df_final_perceptual = df_final_perceptual[df_final_perceptual["comment_clean"].str.strip() != ""]


df_final_perceptual = df_final_perceptual.dropna(subset=["comment_clean"])
df_final_perceptual = df_final_perceptual[df_final_perceptual["comment_clean"].str.strip() != ""]

df_final_perceptual = df_final_perceptual.reset_index(drop=True)
df_final_perceptual["id"] = df_final_perceptual.index + 25319

df_sql = df_sql.drop(columns=["comment_text", "comment_clean"], errors="ignore")

conn2 = sqlite3.connect(db_path)
cursor = conn2.cursor()
conn2.commit()

conn2 = sqlite3.connect(db_path)
df_sql = pd.read_sql_query("SELECT * FROM Reviews", conn2)
conn2.close()

# 3. Yeni yorumların ID’sini 25319’dan başlatalım
df_final_perceptual = df_final_perceptual.reset_index(drop=True)
df_final_perceptual["id"] = df_final_perceptual.index + 25319

# 4. Eski ve yeni verileri birleştir (concat)
df_combined = pd.concat([df_sql, df_final_perceptual], ignore_index=True)

# 5. Sadece comment_clean dolu olanları tut
df_combined = df_combined[df_combined["comment_clean"].notna()]
df_combined = df_combined[df_combined["comment_clean"].str.strip() != ""]

# 6. Yeni veritabanına yaz
db_path_final = base_drive_path + "reviews3.db"
conn_final = sqlite3.connect(db_path_final)
df_combined.to_sql("Reviews", conn_final, if_exists="replace", index=False)
conn_final.close()

print("✅ Tüm veriler comment_text'siz ve temiz yorumlarla ekdata2.db'ye yazıldı.")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-19-3764209829>:140: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_advanced_exclude = df_perceptual[df_perceptual["comment_clean"].str.contains(pattern, case=False, na=False)]
<ipython-input-19-3764209829>:141: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_final_perceptual = df_perceptual[~df_perceptual["comment_clean"].str.contains(pattern, case=False, na=False)]


İşlenen yorum: 0/670
İşlenen yorum: 100/670
İşlenen yorum: 200/670
İşlenen yorum: 300/670
İşlenen yorum: 400/670
İşlenen yorum: 500/670
İşlenen yorum: 600/670
✅ Tüm veriler comment_text'siz ve temiz yorumlarla ekdata2.db'ye yazıldı.


In [ ]:
##Manuel etiketleme kodu

import pandas as pd
df = pd.read_csv("etiketSIZ2.csv")

if "label" not in df.columns:
    df["label"] = None

unlabeled_df = df[df["label"].isnull()].reset_index(drop=True)

# Etiketlenmiş sayacı
pozitif_sayisi = df["label"].value_counts().get(1, 0)
negatif_sayisi = df["label"].value_counts().get(0, 0)

# Geçmiş için index tut
history = []

try:
    i = 0
    while i < len(unlabeled_df):
        row = unlabeled_df.iloc[i]
        row_id = row["id"]

        print(f"\n[{i+1}/{len(unlabeled_df)}] ID: {row_id}")
        print("Yorum:", row["comment_clean"])
        label = input("Etiket gir (0=Negatif, 1=Nötr, 2=Pozitif, b=geri): ").strip()

        if label == "b":
            if history:
                last_id = history.pop()
                df.loc[df["id"] == last_id, "label"] = None
                i -= 1
                print("Geri dönüldü.")
            else:
                print("Geri gidilecek bir şey yok.")
            continue

        elif label in {"0", "1"}:
            df.loc[df["id"] == row_id, "label"] = int(label)
            history.append(row_id)
            if label == "0":
                negatif_sayisi += 1
            elif label == "1":
                pozitif_sayisi += 1
        else:
            print(" Atlandı.")

        if i % 10 == 0:
            print("\n Etiket Sayıları:")
            print(f"  0 → {df['label'].value_counts().get(0, 0)} adet")
            print(f"  0 → {negatif_sayisi} adet")
            print(f"  1 → {pozitif_sayisi} adet")


        if i % 50 == 0:
            df.to_csv("etiketLI_veri2.csv", index=False)
            print("Ara kayıt yapıldı.")

        # Otomatik durma koşulu
        if negatif >= 500
            print("yeterli. Otomatik durduruluyor.")
            break

        i += 1

except KeyboardInterrupt:
    print("\n Elle durduruldu. Kaydediliyor...")

df.to_csv("etiketLI_veri2.csv", index=False)
print("\n Etiketleme tamamlandı veya durduruldu. 'etiketLI_veri2.csv' olarak kaydedildi.")


FileNotFoundError: [Errno 2] No such file or directory: '1yildiz_yorumlar_labeled.csv'